### Generating fake 700 values to add to the hr_dataset

### V10+ [shit load of version's _failed_ and _successfully failed_]

**successfully failed** - they created the dataset but failed to make sense, there were logic gaps which had made the data not 'realsitic' and can't perform any analysis as cumilativly they failed to make sense and the realtionship with all columns with the 'wrong column' was screwed up!

In [ ]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker


class ConsultancyEmployee:
    def __init__(self, faker, reference_date=datetime(2024, 1, 1)):
        self.fake = faker
        self.reference_date = reference_date
        self._init_department_config()
        self._init_location_config()
        self._init_specialization_config()
        self._init_certification_config()
        self.assigned_unique_positions = set()

    def _init_department_config(self):
        self.department_weights = {
            'Product Architecture': 0.2,
            'Design & UX': 0.2,
            'Engineering Delivery': 0.3,
            'Product Strategy': 0.15,
            'Client Services': 0.15
        }
        
        # Separate regular positions from unique executive positions
        self.departments = {
            'Product Architecture': {
                'entry': ['Junior Solution Architect', 'System Analyst I', 'Technical Documentation Specialist'],
                'mid': ['Solution Architect', 'Enterprise Architect', 'Integration Specialist'],
                'senior': ['Principal Architect', 'Lead Solution Architect', 'Senior Enterprise Architect']
            },
            'Design & UX': {
                'entry': ['UI Designer I', 'UX Researcher I', 'Design System Specialist'],
                'mid': ['Senior UX Designer', 'Design Lead', 'Design Strategist'],
                'senior': ['Principal Designer', 'Lead Design Strategist', 'Senior Design Lead']
            },
            'Engineering Delivery': {
                'entry': ['Associate Developer', 'Junior DevOps', 'QA Engineer I'],
                'mid': ['Tech Lead', 'Senior Developer', 'Cloud Architect'],
                'senior': ['Principal Engineer', 'Engineering Lead', 'Development Director']
            },
            'Product Strategy': {
                'entry': ['Product Analyst', 'Business Analyst I', 'Market Researcher'],
                'mid': ['Product Strategist', 'Senior Product Manager', 'Domain Expert'],
                'senior': ['Principal Product Manager', 'Lead Product Strategist', 'Senior Strategy Lead']
            },
            'Client Services': {
                'entry': ['Project Coordinator', 'Client Success Associate', 'Engagement Analyst'],
                'mid': ['Engagement Manager', 'Client Partner', 'Program Manager'],
                'senior': ['Senior Program Director', 'Principal Client Partner', 'Senior Delivery Lead']
            }
        }

        # Define unique executive positions separately
        self.executive_positions = {
            'Product Architecture': ['Chief Architect', 'Head of Architecture'],
            'Design & UX': ['Head of Design', 'Chief Experience Officer'],
            'Engineering Delivery': ['CTO', 'VP of Engineering'],
            'Product Strategy': ['Head of Product', 'Chief Product Officer'],
            'Client Services': ['Head of Client Services', 'Chief Customer Officer']
        }

        self.level_weights = {
            'Product Architecture': {'entry': 0.3, 'mid': 0.5, 'senior': 0.2},
            'Design & UX': {'entry': 0.4, 'mid': 0.4, 'senior': 0.2},
            'Engineering Delivery': {'entry': 0.5, 'mid': 0.3, 'senior': 0.2},
            'Product Strategy': {'entry': 0.3, 'mid': 0.4, 'senior': 0.3},
            'Client Services': {'entry': 0.2, 'mid': 0.5, 'senior': 0.3}
        }

    def _select_position(self, department, level):
        if level == 'senior' and len(self.assigned_unique_positions) < len([pos for positions in self.executive_positions.values() for pos in positions]):
            # Determine if this senior employee should get an executive position
            available_exec_positions = set(self.executive_positions[department]) - self.assigned_unique_positions
            if available_exec_positions and random.random() < 0.2:  # 20% chance for eligible senior employees
                position = random.choice(list(available_exec_positions))
                self.assigned_unique_positions.add(position)
                return position
        
        # Regular position assignment
        return random.choice(self.departments[department][level])

    def _init_location_config(self):
        self.location_data = {
            'London': {'country': 'UK', 'region': 'EMEA', 'cost_multiplier': 1.2, 'timezone': 'GMT', 'market_demand': 0.9},
            'New York': {'country': 'USA', 'region': 'Americas', 'cost_multiplier': 1.3, 'timezone': 'EST', 'market_demand': 1.0},
            'San Francisco': {'country': 'USA', 'region': 'Americas', 'cost_multiplier': 1.35, 'timezone': 'PST', 'market_demand': 1.1},
            'Singapore': {'country': 'Singapore', 'region': 'APAC', 'cost_multiplier': 1.15, 'timezone': 'SGT', 'market_demand': 0.95},
            'Berlin': {'country': 'Germany', 'region': 'EMEA', 'cost_multiplier': 1.1, 'timezone': 'CET', 'market_demand': 0.85}
        }
        self.location_weights = {
            'Product Architecture': {'London': 0.3, 'New York': 0.2, 'San Francisco': 0.2, 'Singapore': 0.15, 'Berlin': 0.15},
            'Design & UX': {'London': 0.25, 'New York': 0.3, 'San Francisco': 0.2, 'Singapore': 0.15, 'Berlin': 0.1},
            'Engineering Delivery': {'London': 0.2, 'New York': 0.25, 'San Francisco': 0.25, 'Singapore': 0.15, 'Berlin': 0.15},
            'Product Strategy': {'London': 0.3, 'New York': 0.25, 'San Francisco': 0.2, 'Singapore': 0.15, 'Berlin': 0.1},
            'Client Services': {'London': 0.3, 'New York': 0.2, 'San Francisco': 0.2, 'Singapore': 0.2, 'Berlin': 0.1}
        }

    def _init_specialization_config(self):
        self.specializations = {
            'Product Architecture': ['Cloud Infrastructure', 'Microservices', 'System Integration', 'API Design', 'Data Architecture', 'Security Architecture'],
            'Design & UX': ['Mobile Design', 'Web Applications', 'Design Systems', 'User Research', 'Service Design', 'Design Thinking'],
            'Engineering Delivery': ['Full Stack', 'Frontend', 'Backend', 'DevOps', 'Cloud Native', 'Mobile Development'],
            'Product Strategy': ['Digital Transformation', 'Innovation Strategy', 'Go-to-Market', 'Product Discovery', 'Market Analysis'],
            'Client Services': ['Agile Delivery', 'Digital Strategy', 'Change Management', 'Portfolio Management', 'Strategic Planning']
        }

    def _init_certification_config(self):
        self.certification_types = {
            'Product Architecture': ['AWS Solutions Architect', 'Azure Architect', 'TOGAF', 'Google Cloud Architect'],
            'Design & UX': ['Google UX Design', 'Interaction Design Foundation', 'Human Factors International'],
            'Engineering Delivery': ['AWS Developer', 'Kubernetes', 'Azure Developer', 'Scrum Master'],
            'Product Strategy': ['Product Management (PMP)', 'Agile Product Management', 'PSPO'],
            'Client Services': ['Prince2', 'PMP', 'Agile Project Management', 'Scrum Master']
        }

    def _select_department(self):
        return random.choices(
            list(self.department_weights.keys()),
            weights=list(self.department_weights.values())
        )[0]

    def _select_level(self, department):
        weights = self.level_weights[department]
        return random.choices(
            list(weights.keys()),
            weights=list(weights.values())
        )[0]

    def _select_location(self, department):
        weights = self.location_weights[department]
        return random.choices(
            list(weights.keys()),
            weights=list(weights.values())
        )[0]

    def _initialize_core_attributes(self):
        """Initialize core employee attributes that influence other calculations."""
        is_outlier = random.random() < 0.05
        self.current_department = self._select_department()
        self.current_level = self._select_level(is_outlier)
        self.hire_date = self._generate_hire_date(self.current_level, is_outlier)
        self.current_location = self._select_location(self.current_department)

    def _generate_hire_date(self, level, is_outlier):
        """Generate realistic hire date based on level and experience requirements."""
        experience_ranges = {
            'entry': (0, 2),
            'mid': (2, 6),
            'senior': (5, 12)
        }
        
        if is_outlier:
            experience_ranges = {
                'entry': (2, 4),
                'mid': (5, 8),
                'senior': (10, 15)
            }
        
        min_years, max_years = experience_ranges[level]
        days_ago = random.randint(min_years * 365, max_years * 365)
        return self.reference_date - timedelta(days=days_ago)

    def _get_position(self):
        """Get position title based on department and level."""
        return random.choice(self.departments[self.current_department][self.current_level])

    def _generate_location_data(self):
        """Generate location-related data including remote work arrangements."""
        location_info = self.location_data[self.current_location]
        
        return {
            'city': self.current_location,
            'country': location_info['country'],
            'region': location_info['region'],
            'remote_work_ratio': self._calculate_remote_ratio(),
            'travel_percentage': self._calculate_travel_percentage()
        }

    def _generate_financial_metrics(self):
        """Generate financial metrics including salary and billing rates."""
        base_salary = self._calculate_base_salary()
        
        return {
            'base_salary': base_salary,
            'billing_rate': self._calculate_billing_rate(base_salary),
            'utilization_target': self._calculate_utilization_target(),
            'actual_utilization': self._calculate_actual_utilization()
        }

    def _calculate_base_salary(self):
        """Calculate base salary considering multiple factors."""
        # Base salary ranges by level
        base_ranges = {
            'entry': (65000, 85000),
            'mid': (95000, 145000),
            'senior': (150000, 250000)
        }
        
        # Department multipliers
        dept_multipliers = {
            'Product Architecture': 1.25,
            'Design & UX': 1.15,
            'Engineering Delivery': 1.20,
            'Product Strategy': 1.18,
            'Client Services': 1.10
        }
        
        # Calculate base salary
        min_salary, max_salary = base_ranges[self.current_level]
        base = random.uniform(min_salary, max_salary)
        
        # Apply multipliers
        location_multiplier = self.location_data[self.current_location]['cost_multiplier']
        dept_multiplier = dept_multipliers[self.current_department]
        experience_multiplier = self._calculate_experience_multiplier()
        
        final_salary = base * location_multiplier * dept_multiplier * experience_multiplier
        
        # Round to nearest thousand
        return round(final_salary, -3)

    def _calculate_experience_multiplier(self):
        """Calculate experience multiplier based on years of experience."""
        years_experience = (self.reference_date - self.hire_date).days / 365
        return 1 + (min(years_experience, 15) * 0.025)

    def _calculate_billing_rate(self, base_salary):
        """Calculate billing rate based on cost and market factors."""
        # Basic cost calculation
        annual_cost = base_salary * 2.5  # Including overhead
        utilization_target = self._calculate_utilization_target() / 100
        annual_billable_hours = 2080 * utilization_target
        
        base_rate = annual_cost / annual_billable_hours
        
        # Apply market adjustments
        market_multipliers = {
            'Product Architecture': 1.4,
            'Design & UX': 1.3,
            'Engineering Delivery': 1.35,
            'Product Strategy': 1.45,
            'Client Services': 1.25
        }
        
        location_demand = self.location_data[self.current_location]['market_demand']
        
        final_rate = base_rate * market_multipliers[self.current_department] * location_demand
        
        # Round to nearest 50
        return round(final_rate / 50) * 50

    def _calculate_utilization_target(self):
        """Calculate utilization target considering role and responsibilities."""
        base_targets = {
            'Product Architecture': 70,
            'Design & UX': 75,
            'Engineering Delivery': 85,
            'Product Strategy': 65,
            'Client Services': 80
        }
        
        level_adjustments = {
            'entry': 10,
            'mid': 0,
            'senior': -15
        }
        
        target = base_targets[self.current_department] + level_adjustments[self.current_level]
        
        # Add realistic variation
        target += random.uniform(-3, 3)
        
        return round(min(max(target, 50), 90))

    def _calculate_actual_utilization(self):
        """Calculate actual utilization with realistic variation from target."""
        target = self._calculate_utilization_target()
        variation = random.uniform(-15, 5)  # Tendency to under-utilize rather than over-utilize
        
        return round(min(max(target + variation, 50), 95), 1)

    def _generate_expertise_data(self):
        """Generate expertise-related data including specializations and certifications."""
        return {
            'primary_specialization': self._select_specialization(),
            'secondary_specialization': self._select_specialization(),
            'industry_expertise': self._select_industry_expertise(),
            'certifications': self._assign_certifications()
        }

    def _select_specialization(self):
        """Select specialization based on department."""
        specializations = {
            'Product Architecture': [
                'Cloud Infrastructure', 'Microservices', 'System Integration',
                'API Design', 'Data Architecture', 'Security Architecture'
            ],
            'Design & UX': [
                'Mobile Design', 'Web Applications', 'Design Systems',
                'User Research', 'Service Design', 'Design Thinking'
            ],
            'Engineering Delivery': [
                'Full Stack', 'Frontend', 'Backend', 'DevOps',
                'Cloud Native', 'Mobile Development'
            ],
            'Product Strategy': [
                'Digital Transformation', 'Innovation Strategy', 'Go-to-Market',
                'Product Discovery', 'Market Analysis'
            ],
            'Client Services': [
                'Agile Delivery', 'Digital Strategy', 'Change Management',
                'Portfolio Management', 'Strategic Planning'
            ]
        }
        
        return random.choice(specializations[self.current_department])

    def _select_industry_expertise(self):
        """Select industry expertise areas."""
        industries = [
            'FinTech', 'HealthTech', 'E-commerce', 'Enterprise Software',
            'Social Impact', 'Consumer Tech', 'EdTech', 'Industrial Tech'
        ]
        
        num_industries = random.randint(2, 4)
        return random.sample(industries, k=num_industries)

    def _assign_certifications(self):
        """Assign relevant certifications based on role and level."""
        available_certs = self.certification_types.get(self.current_department, [])
        
        # Number of certifications based on level
        cert_counts = {
            'entry': (0, 2),
            'mid': (1, 3),
            'senior': (2, 4)
        }
        
        min_certs, max_certs = cert_counts[self.current_level]
        num_certs = random.randint(min_certs, min(max_certs, len(available_certs)))
        
        return random.sample(available_certs, k=num_certs)

    def _calculate_project_complexity(self, level, is_outlier):
        complexity_weights = {
            'entry': [0.5, 0.3, 0.15, 0.05, 0],     # Mostly simple projects
            'mid': [0.1, 0.3, 0.4, 0.15, 0.05],     # Mix of complexities
            'senior': [0, 0.1, 0.3, 0.4, 0.2]       # More complex projects
        }
        
        if is_outlier:
            complexity_weights = {
                'entry': [0.2, 0.3, 0.3, 0.15, 0.05],
                'mid': [0, 0.15, 0.25, 0.4, 0.2],
                'senior': [0, 0, 0.2, 0.4, 0.4]
            }
        
        return np.random.choice(range(1, 6), p=complexity_weights[level])

    def _calculate_flight_risk(self, is_outlier):
        """
        Calculate flight risk with more realistic distribution:
        - Most employees (60%) should be low-medium risk (20-50%)
        - Some (30%) medium-high risk (50-70%)
        - Few (10%) very high risk (70-85%)
        - Extremely rare (1-2%) critical risk (85-95%)
        """
        # Base risk starts lower
        base_risk = 20
        
        # Core factors
        years_in_role = (self.reference_date - self.hire_date).days / 365
        engagement_score = self._generate_engagement_score(is_outlier)
        performance_score = self._generate_performance_score(self.current_level, is_outlier)
        utilization = self._calculate_actual_utilization()
        
        # Critical triggers that could significantly increase risk
        if years_in_role > 4 and self.current_level != 'senior':
            base_risk += np.random.normal(15, 3)  # Add variation
        
        if engagement_score < 6:
            base_risk += np.random.normal(20, 4)
        elif engagement_score < 7:
            base_risk += np.random.normal(10, 2)
            
        if performance_score > 4.5:  # High performers may be poached
            base_risk += np.random.normal(12, 3)
        elif performance_score < 3.0:  # Low performers may be dissatisfied
            base_risk += np.random.normal(15, 3)
            
        # Market factors
        market_demand = self.location_data[self.current_location]['market_demand']
        if market_demand > 0.9:  # Hot markets increase risk
            base_risk += np.random.normal(8, 2)
            
        # Role-specific factors
        high_demand_positions = {
            'Solution Architect': 12,
            'Cloud Architect': 15,
            'Tech Lead': 10,
            'Senior Developer': 8,
            'Senior UX Designer': 7,
            'Product Strategist': 6
        }
        position_risk = high_demand_positions.get(self._get_position(), 0)
        base_risk += np.random.normal(position_risk, 2) if position_risk > 0 else 0
        
        # Apply some random variation
        base_risk += np.random.normal(0, 3)
        
        # Cap maximum risk and ensure minimum
        return round(min(max(base_risk, 15), 95))

    def _calculate_promotion_readiness(self, level, hire_date, is_outlier):
        """
        Calculate promotion readiness with more realistic distribution:
        - Most (50%) should be in 20-50% range (developing)
        - Some (30%) in 50-70% range (ready soon)
        - Few (15%) in 70-85% range (ready now)
        - Very few (5%) above 85% (overdue)
        """
        if level == 'senior':
            return round(random.uniform(0, 30), 1)
            
        # Core factors
        years_in_role = (self.reference_date - hire_date).days / 365
        performance_score = self._generate_performance_score(level, is_outlier)
        
        # Base score starts lower
        base_score = 20
        
        # Experience factor (diminishing returns after 3 years)
        experience_score = min(years_in_role * 8, 24)
        
        # Performance impact (exponential for high performers)
        performance_factor = 0
        if performance_score >= 4.5:
            performance_factor = 25
        elif performance_score >= 4.0:
            performance_factor = 15
        elif performance_score >= 3.5:
            performance_factor = 8
            
        # Skills and certifications
        cert_count = len(self._assign_certifications())
        skill_factor = min(cert_count * 3, 12)  # Cap skill contribution
        
        # Project complexity impact
        project_complexity = self._calculate_project_complexity(level, is_outlier)
        complexity_factor = project_complexity * 3
        
        # Calculate total score
        total_score = base_score + experience_score + performance_factor + skill_factor + complexity_factor
        
        # Add small random variation
        total_score += np.random.normal(0, 2)
        
        # Ensure realistic distribution
        if total_score > 85:  # Only 5% should be here
            total_score = min(total_score, 92)
        
        return round(min(max(total_score, 15), 92), 1)

    def _calculate_innovation_score(self, level, department):
        base_innovation = {
            'Product Architecture': 7.5,    # High technical innovation
            'Design & UX': 8.0,            # Creative solutions
            'Engineering Delivery': 7.0,    # Technical implementation
            'Product Strategy': 8.5,        # Business innovation
            'Client Services': 6.5          # Process innovation
        }
        
        level_factors = {
            'entry': 0.7,    # Learning phase
            'mid': 1.0,      # Established
            'senior': 1.3    # Leading innovation
        }
        
        performance_score = self._generate_performance_score(level, False)
        performance_factor = (performance_score - 3) * 0.2
        
        project_complexity = self._calculate_project_complexity(level, False)
        complexity_factor = (project_complexity - 3) * 0.15
        
        # Fixed certification count calculation
        cert_count = len(self._assign_certifications())
        certification_factor = cert_count * 0.1
        
        innovation = (base_innovation[department] * level_factors[level] * 
                    (1 + performance_factor + complexity_factor + certification_factor))
        
        return round(min(max(innovation + random.uniform(-0.5, 0.5), 1), 10), 1)

    def _assign_office(self, department):
        weights = {
            'Product Architecture': [0.3, 0.2, 0.2, 0.15, 0.15],
            'Design & UX': [0.25, 0.3, 0.2, 0.15, 0.1],
            'Engineering Delivery': [0.2, 0.25, 0.25, 0.15, 0.15],
            'Product Strategy': [0.3, 0.25, 0.2, 0.15, 0.1],
            'Client Services': [0.3, 0.2, 0.2, 0.2, 0.1]
        }
        
        office_key = np.random.choice(
            list(self.office_locations.keys()),
            p=weights[department]
        )
        return self.office_locations[office_key]
    
    def _calculate_remote_ratio(self):
        base_remote = {'entry': 20, 'mid': 30, 'senior': 40}[self.current_level]
        return round(min(max(base_remote + random.uniform(-10, 10), 0), 100), 1)

    def _calculate_travel_percentage(self):
        base_travel = {'entry': 10, 'mid': 20, 'senior': 30}[self.current_level]
        dept_modifier = {
            'Product Architecture': 0,
            'Design & UX': -5,
            'Engineering Delivery': -10,
            'Product Strategy': 15,
            'Client Services': 20
        }[self.current_department]
        return round(min(max(base_travel + dept_modifier + random.uniform(-5, 5), 0), 100), 1)


    def _assign_industry_expertise(self):
        # Ensure we don't try to sample more than available
        num_industries = random.randint(2, min(4, len(self.industry_expertise)))
        return random.sample(self.industry_expertise, k=num_industries)

    def _generate_performance_score(self, level, is_outlier):
        base_scores = {
            'entry': np.random.normal(3.5, 0.4),
            'mid': np.random.normal(3.8, 0.3),
            'senior': np.random.normal(4.1, 0.2)
        }
        
        score = base_scores[level]
        if is_outlier:
            score = random.choice([2.0, 4.8])
        
        return round(min(max(score, 1), 5), 1)
    
    def _generate_project_metrics(self):
        """Generate project-related metrics and statistics."""
        is_outlier = random.random() < 0.05
        
        # Calculate number of active projects based on level and department
        base_projects = {
            'entry': (1, 2),
            'mid': (2, 3),
            'senior': (2, 4)
        }
        
        # Department modifiers for project count
        dept_modifiers = {
            'Product Architecture': 0,
            'Design & UX': -1,
            'Engineering Delivery': 1,
            'Product Strategy': 0,
            'Client Services': 1
        }
        
        min_projects, max_projects = base_projects[self.current_level]
        modifier = dept_modifiers[self.current_department]
        
        # Adjust project range based on department modifier
        min_projects = max(1, min_projects + modifier)
        max_projects = max(min_projects, max_projects + modifier)
        
        active_projects = random.randint(min_projects, max_projects)
        
        # Generate individual project data
        projects = []
        for _ in range(active_projects):
            project = {
                'complexity': self._calculate_project_complexity(self.current_level, is_outlier),
                'status': random.choice(['Planning', 'In Progress', 'Final Stage']),
                'duration_months': random.randint(3, 12),
                'team_size': random.randint(3, 10)
            }
            projects.append(project)
        
        # Calculate average project metrics
        avg_complexity = round(sum(p['complexity'] for p in projects) / len(projects), 1)
        avg_duration = round(sum(p['duration_months'] for p in projects) / len(projects), 1)
        avg_team_size = round(sum(p['team_size'] for p in projects) / len(projects), 1)
        
        return {
            'active_projects': active_projects,
            'avg_project_complexity': avg_complexity,
            'avg_project_duration': avg_duration,
            'avg_team_size': avg_team_size,
            'projects_on_time': self._calculate_projects_on_time(active_projects, is_outlier),
            'project_satisfaction': self._generate_client_satisfaction(
                self.current_level, 
                self.current_department, 
                projects,
                self._generate_performance_score(self.current_level, is_outlier)
            )
        }

    def _calculate_projects_on_time(self, active_projects, is_outlier):
        """Calculate percentage of projects delivered on time."""
        if is_outlier:
            return random.choice([100, 60])
            
        base_on_time = {
            'entry': 80,
            'mid': 85,
            'senior': 90
        }
        
        dept_modifiers = {
            'Product Architecture': -5,
            'Design & UX': 0,
            'Engineering Delivery': -3,
            'Product Strategy': 2,
            'Client Services': 5
        }
        
        on_time_pct = base_on_time[self.current_level] + dept_modifiers[self.current_department]
        # Add some random variation
        on_time_pct += random.uniform(-5, 5)
        
        return round(min(max(on_time_pct, 60), 100), 1)

    def _generate_development_metrics(self):
        """Generate development and growth-related metrics."""
        is_outlier = random.random() < 0.05
        
        return {
            'training_hours': self._calculate_training_hours(self.current_level, is_outlier),
            'mentorship_hours': self._calculate_mentorship_hours(self.current_level),
            'knowledge_sharing_score': self._calculate_knowledge_sharing(self.current_level, is_outlier),
            'promotion_readiness': self._calculate_promotion_readiness(
                self.current_level, 
                self.hire_date, 
                is_outlier
            )
        }

    def _generate_risk_metrics(self):
        """Generate risk and retention-related metrics."""
        is_outlier = random.random() < 0.05
        
        engagement_score = self._generate_engagement_score(is_outlier)
        flight_risk = self._calculate_flight_risk(is_outlier)
        
        return {
            'engagement_score': engagement_score,
            'flight_risk': flight_risk,
            'retention_risk': 'High' if flight_risk > 70 else 'Medium' if flight_risk > 40 else 'Low'
        }

    def _generate_engagement_score(self, is_outlier):
        """Generate employee engagement score."""
        if is_outlier:
            return random.choice([4.0, 9.5])
        
        base_engagement = {
            'entry': np.random.normal(7.5, 0.8),
            'mid': np.random.normal(7.8, 0.7),
            'senior': np.random.normal(8.2, 0.6)
        }
        
        score = base_engagement[self.current_level]
        
        # Adjust based on department
        dept_adjustments = {
            'Product Architecture': 0.2,
            'Design & UX': 0.3,
            'Engineering Delivery': 0.1,
            'Product Strategy': 0.4,
            'Client Services': 0.2
        }
        
        score += dept_adjustments[self.current_department]
        
        return round(min(max(score, 1), 10), 1)

    def _generate_performance_metrics(self, is_outlier):
        """Generate performance metrics with controlled outlier distribution"""
        if is_outlier:
            # Outliers are high performers (95th percentile or above)
            performance_score = round(random.uniform(4.5, 5.0), 2)
            innovation_score = round(random.uniform(90, 100), 1)
            delivery_quality = round(random.uniform(95, 100), 1)
        else:
            # Normal distribution for regular employees
            performance_score = round(random.normalvariate(3.7, 0.4), 2)
            performance_score = max(min(performance_score, 4.5), 2.5)  # Cap normal range
            
            innovation_score = round(random.normalvariate(75, 8), 1)
            innovation_score = max(min(innovation_score, 90), 50)
            
            delivery_quality = round(random.normalvariate(82, 7), 1)
            delivery_quality = max(min(delivery_quality, 95), 60)
        
        return {
            'performance_score': performance_score,
            'innovation_score': innovation_score,
            'delivery_quality': delivery_quality
        }

    def _generate_project_data(self):
        """Generate random project data."""
        statuses = ['Planning', 'In Progress', 'Final Stage']
        return {
            'complexity': random.randint(1, 5),
            'status': random.choice(statuses)
        }

    def _generate_client_satisfaction(self, level, department, projects, performance_score):
        """Generate client satisfaction score based on multiple factors"""
        if not projects:
            return None
        
        # Base satisfaction weighted by department expertise
        dept_base_satisfaction = {
            'Product Architecture': 4.2,
            'Design & UX': 4.0,
            'Engineering Delivery': 4.1,
            'Product Strategy': 4.0,
            'Client Services': 4.3
        }
        
        # Calculate weighted satisfaction based on multiple factors
        total_weight = 0
        weighted_satisfaction = 0
        
        for project in projects:
            project_weight = 1.0
            base = dept_base_satisfaction[department]
            
            # Project complexity factor
            complexity = project['complexity']
            complexity_factor = {
                1: 0.1,    # Simple projects easier to satisfy
                2: 0.05,
                3: 0,
                4: -0.05,
                5: -0.1    # Complex projects harder to satisfy
            }
            
            # Project status factor
            status_factor = {
                'Planning': 0.05,      # Early stage optimism
                'In Progress': 0,      # Neutral
                'Final Stage': -0.05   # Delivery pressure
            }
            
            # Calculate project-specific satisfaction
            project_satisfaction = base
            project_satisfaction += complexity_factor[complexity]
            project_satisfaction += status_factor[project['status']]
            
            # Performance impact
            performance_impact = (performance_score - 3.5) * 0.2
            project_satisfaction += performance_impact
            
            # Level-based adjustments
            level_factor = {
                'entry': -0.1,    # Less experience
                'mid': 0,         # Baseline
                'senior': 0.1     # More experience
            }
            project_satisfaction += level_factor[level]
            
            # Add slight randomization for realism
            project_satisfaction += random.uniform(-0.1, 0.1)
            
            # Accumulate weighted satisfaction
            weighted_satisfaction += project_satisfaction * project_weight
            total_weight += project_weight
        
        final_satisfaction = round(weighted_satisfaction / total_weight, 1)
        return min(max(final_satisfaction, 1.0), 5.0)

    def _generate_internal_contribution(self, level):
        base_contribution = {
            'entry': (1, 3),
            'mid': (3, 5),
            'senior': (4, 7)
        }
        min_contrib, max_contrib = base_contribution[level]
        return random.randint(min_contrib, max_contrib)

    def _generate_delivery_quality(self, level, is_outlier):
        base_quality = {
            'entry': np.random.normal(85, 5),
            'mid': np.random.normal(90, 4),
            'senior': np.random.normal(93, 3)
        }
        
        quality = base_quality[level]
        if is_outlier:
            quality = random.choice([75, 98])
            
        return round(min(max(quality, 70), 100), 1)

    def _calculate_training_hours(self, level, is_outlier):
        base_hours = {
            'entry': (40, 60),
            'mid': (30, 50),
            'senior': (20, 40)
        }
        
        if is_outlier:
            return random.choice([10, 100])
            
        min_hours, max_hours = base_hours[level]
        return random.randint(min_hours, max_hours)

    def _calculate_mentorship_hours(self, level):
        # Negative hours indicate receiving mentorship, positive hours indicate giving mentorship
        mentorship_ranges = {
            'entry': (-30, -10),
            'mid': (-10, 20),
            'senior': (20, 40)
        }
        min_hours, max_hours = mentorship_ranges[level]
        return random.randint(min_hours, max_hours)

    def _calculate_knowledge_sharing(self, level, is_outlier):
        """
        Calculate knowledge sharing score with more realistic distribution:
        Entry: 3-6 range (learning phase)
        Mid: 5-8 range (active contribution)
        Senior: 7-9 range (leadership in sharing)
        Extremely rare to have very low scores (company culture wouldn't allow it)
        """
        # Base ranges adjusted to be more realistic
        base_sharing = {
            'entry': (3, 6),
            'mid': (5, 8),
            'senior': (7, 9)
        }
        
        if is_outlier:
            if random.random() < 0.8:  # 80% chance of high outlier
                return round(random.uniform(8.5, 9.5), 1)
            return round(random.uniform(2.5, 3.5), 1)  # 20% chance of low outlier
        
        min_share, max_share = base_sharing[level]
        
        # Generate score using beta distribution for more realistic clustering
        if level == 'entry':
            score = np.random.beta(4, 3) * (max_share - min_share) + min_share
        elif level == 'mid':
            score = np.random.beta(5, 3) * (max_share - min_share) + min_share
        else:  # senior
            score = np.random.beta(6, 2) * (max_share - min_share) + min_share
        
        return round(score, 1)
    def generate_employee(self, department=None, level=None, is_outlier=False):
        self.current_department = department or self._select_department()
        self.current_level = level or self._select_level(self.current_department)
        self.hire_date = self._generate_hire_date(self.current_level, is_outlier)
        self.current_location = self._select_location(self.current_department)
        
        return {
            'employee_id': self.fake.unique.random_number(digits=6),
            'full_name': self.fake.name(),
            'department': self.current_department,
            'position': self._select_position(self.current_department, self.current_level),
            'level': self.current_level,
            'hire_date': self.hire_date,
            **self._generate_location_data(),
            **self._generate_financial_metrics(),
            **self._generate_expertise_data(),
            **self._generate_performance_metrics(is_outlier),
            **self._generate_project_metrics(),
            **self._generate_development_metrics(),
            **self._generate_risk_metrics(),
            'manager_id': None,
            'total_comp': None,
            'team_lead_projects': 0,
            'direct_reports': 0
        }


class ConsultancyDataGenerator:
    def __init__(self, num_records, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        self.num_records = num_records
        self.employee_generator = ConsultancyEmployee(self.fake)
        self.departments = list(self.employee_generator.departments.keys())
        self.levels = ['entry', 'mid', 'senior']
        self.department_weights = self.employee_generator.department_weights
        self.level_weights = self.employee_generator.level_weights
        
        # Setting company ratios based on company size
        self.senior_ratio = 0.03 if num_records >= 1000 else 0.05
        self.mid_ratio = 0.10 if num_records >= 1000 else 0.12 if num_records >= 100 else 0.08
        self.max_senior_managers = max(1, int(num_records * self.senior_ratio))
        self.max_mid_managers = max(2, int(num_records * self.mid_ratio))
        self.max_direct_reports_senior = 10 if num_records >= 1000 else 8 if num_records >= 100 else 6
        self.max_direct_reports_mid = 8 if num_records >= 1000 else 6 if num_records >= 100 else 4

    def _select_department(self):
        # Ensure weights are valid probabilities (positive and sum to 1)
        weights = list(self.department_weights.values())
        if not all(w > 0 for w in weights) or abs(sum(weights) - 1) > 1e-9:
            # Normalize weights if they're invalid
            total = sum(weights)
            weights = [w/total if w > 0 else 0.01 for w in weights]
            weights = [w/sum(weights) for w in weights]  # Renormalize after fixing zeros
        
        return random.choices(list(self.department_weights.keys()), weights=weights)[0]

    def _select_level(self, department):
        # Ensure weights are valid probabilities
        weights = list(self.level_weights[department].values())
        if not all(w > 0 for w in weights) or abs(sum(weights) - 1) > 1e-9:
            # Normalize weights if they're invalid
            total = sum(weights)
            weights = [w/total if w > 0 else 0.01 for w in weights]
            weights = [w/sum(weights) for w in weights]
        
        return random.choices(list(self.level_weights[department].keys()), weights=weights)[0]

    def _assign_managers(self, df):
        if df.empty:
            return df
            
        df['is_manager'] = False
        df['management_level'] = None
        df['direct_reports'] = 0
        
        for dept in self.departments:
            dept_mask = df['department'] == dept
            if not any(dept_mask):
                continue
                
            dept_indices = df[dept_mask].index
            
            # Ensure valid masks for manager selection
            senior_mask = (df['level'] == 'senior') & dept_mask & \
                        df['position'].str.contains('Director|Head|Chief|Lead', case=False, na=False)
            mid_mask = (df['level'] == 'mid') & dept_mask & \
                    df['position'].str.contains('Lead|Senior|Manager', case=False, na=False)
            
            dept_size = len(dept_indices)
            if dept_size == 0:
                continue
                
            # Ensure at least one manager per department
            dept_senior_managers = max(1, int(dept_size * self.senior_ratio))
            dept_mid_managers = max(2, int(dept_size * self.mid_ratio))
            
            # Get valid candidates
            senior_candidates = df[senior_mask].sort_values('performance_score', ascending=False)
            mid_candidates = df[mid_mask].sort_values('performance_score', ascending=False)
            
            # Handle case where there aren't enough candidates
            max_senior = min(dept_senior_managers, 
                            len(senior_candidates), 
                            int(self.max_senior_managers / len(self.departments)))
            max_mid = min(dept_mid_managers, 
                        len(mid_candidates), 
                        int(self.max_mid_managers / len(self.departments)))
            
            if max_senior == 0 or max_mid == 0:
                continue
                
            senior_manager_indices = senior_candidates.head(max_senior).index
            mid_manager_indices = mid_candidates.head(max_mid).index
            
            # Assign managers
            df.loc[senior_manager_indices, 'is_manager'] = True
            df.loc[senior_manager_indices, 'management_level'] = 'senior'
            df.loc[mid_manager_indices, 'is_manager'] = True
            df.loc[mid_manager_indices, 'management_level'] = 'mid'
            
            # Assign reports to managers
            mid_level_mask = (df['level'] == 'mid') & dept_mask & ~df.index.isin(mid_manager_indices)
            mid_level_indices = df[mid_level_mask].index
            
            for idx in mid_level_indices:
                valid_managers = df.loc[senior_manager_indices]
                if not valid_managers.empty:
                    valid_managers = valid_managers[valid_managers['direct_reports'] < self.max_direct_reports_senior]
                    if not valid_managers.empty:
                        manager = valid_managers.sample(n=1)
                        df.loc[idx, 'manager_id'] = manager.iloc[0]['employee_id']
                        df.loc[manager.index[0], 'direct_reports'] += 1
            
            entry_mask = (df['level'] == 'entry') & dept_mask
            entry_indices = df[entry_mask].index
            
            for idx in entry_indices:
                # Try to assign to mid-level managers first
                valid_mid_managers = df.loc[mid_manager_indices]
                valid_mid_managers = valid_mid_managers[valid_mid_managers['direct_reports'] < self.max_direct_reports_mid]
                
                if not valid_mid_managers.empty:
                    manager = valid_mid_managers.sample(n=1)
                    df.loc[idx, 'manager_id'] = manager.iloc[0]['employee_id']
                    df.loc[manager.index[0], 'direct_reports'] += 1
                else:
                    # Fall back to senior managers if needed
                    valid_senior_managers = df.loc[senior_manager_indices]
                    valid_senior_managers = valid_senior_managers[valid_senior_managers['direct_reports'] < self.max_direct_reports_senior]
                    
                    if not valid_senior_managers.empty:
                        manager = valid_senior_managers.sample(n=1)
                        df.loc[idx, 'manager_id'] = manager.iloc[0]['employee_id']
                        df.loc[manager.index[0], 'direct_reports'] += 1
        
        # Calculate span of control
        df['span_of_control'] = df['direct_reports'].apply(
            lambda x: 'None' if x == 0 else 
                    'Small (1-3)' if x <= 3 else 
                    'Medium (4-6)' if x <= 6 else 
                    'Large (7+)'
        )
        
        return df

    def _calculate_management_compensation_adjustment(self, df):
        if df.empty:
            return df
            
        df['management_premium'] = 0.0
        df['span_premium'] = 0.0
        
        # Calculate premium for senior managers
        senior_mask = df['is_manager'] & (df['management_level'] == 'senior')
        if any(senior_mask):
            df.loc[senior_mask, 'management_premium'] = (df.loc[senior_mask, 'base_salary'] * 
                np.random.uniform(0.25, 0.35, size=senior_mask.sum())).round(2)
        
        # Calculate premium for mid-level managers
        mid_mask = df['is_manager'] & (df['management_level'] == 'mid')
        if any(mid_mask):
            df.loc[mid_mask, 'management_premium'] = (df.loc[mid_mask, 'base_salary'] * 
                np.random.uniform(0.15, 0.25, size=mid_mask.sum())).round(2)
        
        # Calculate span premium based on number of direct reports
        df['span_premium'] = (df['direct_reports'].apply(lambda x: 
            0.0 if x == 0 
            else 0.05 if x <= 3 
            else 0.10 if x <= 6 
            else 0.15) * df['base_salary']).round(2)
        
        # Calculate total compensation with rounded values
        df['total_comp'] = (df['base_salary'] + df['management_premium'] + df['span_premium']).round(2)
        
        return df

    def _update_team_metrics(self, df):
        """
        Updates team metrics in a more robust way
        """
        # Update direct reports count
        manager_counts = df['manager_id'].value_counts()
        manager_mask = df['employee_id'].isin(manager_counts.index)
        df.loc[manager_mask, 'direct_reports'] = df.loc[manager_mask, 'employee_id'].map(manager_counts)
        
        # Update team lead projects
        senior_mask = df['level'] == 'senior'
        mid_mask = df['level'] == 'mid'
        
        # For senior employees
        df.loc[senior_mask, 'team_lead_projects'] = df.loc[senior_mask, 'active_projects'].apply(
            lambda x: random.randint(1, max(1, x)) if pd.notnull(x) else 0
        )
        
        # For mid-level employees
        df.loc[mid_mask, 'team_lead_projects'] = df.loc[mid_mask, 'active_projects'].apply(
            lambda x: random.randint(0, max(0, x-1)) if pd.notnull(x) else 0
        )
        
        return df
    
    def _assign_outliers(self):
        # Calculate number of outliers (2% of total)
        total_outliers = max(len(self.departments), int(self.num_records * 0.02))
        
        # Initialize outlier tracking dictionary
        self.outlier_assignments = {
            dept: {level: 0 for level in self.levels}
            for dept in self.departments
        }
        
        # Ensure at least one outlier per department
        for dept in self.departments:
            level = random.choice(self.levels)
            self.outlier_assignments[dept][level] += 1
            total_outliers -= 1
            
        # Distribute remaining outliers randomly but evenly across levels and departments
        while total_outliers > 0:
            dept = random.choice(self.departments)
            level = random.choice(self.levels)
            self.outlier_assignments[dept][level] += 1
            total_outliers -= 1
    
    def _get_expected_count(self, department, level):
        """Calculate expected number of employees for each department/level"""
        return int(self.num_records * self.department_weights[department] * self.level_weights[department][level])

    def _calculate_total_compensation(self, df):
        """Calculate total compensation including bonus and benefits"""
        df['total_comp'] = df.apply(lambda row: 
            row['base_salary'] * (1 + 
                                (0.2 if row['level'] == 'senior' else
                                 0.15 if row['level'] == 'mid' else 0.1) +
                                (0.05 if row['performance_score'] >= 4.5 else 0)), 
            axis=1)
        
    def generate_dataset(self):
        """
        Generates the complete dataset with error handling
        """
        try:
            # Generate initial employee records
            employees = [self.employee_generator.generate_employee() for _ in range(self.num_records)]
            
            # Convert to DataFrame
            df = pd.DataFrame(employees)
            
            # Ensure required columns exist
            required_columns = ['employee_id', 'department', 'level', 'position', 'base_salary']
            missing_columns = [col for col in required_columns if col not in df.columns]
            if missing_columns:
                raise ValueError(f"Missing required columns: {missing_columns}")
            
            # Assign managers and update metrics
            df = self._assign_managers(df)
            df = self._calculate_management_compensation_adjustment(df)
            df = self._update_team_metrics(df)
            
            return df
            
        except Exception as e:
            print(f"Error generating dataset: {str(e)}")
            raise

    def _prepare_data_for_export(self):
        """Prepare and clean data for export"""
        df = self.generate_dataset()
        
        df['hire_date'] = df['hire_date'].dt.strftime('%Y-%m-%d')
        
        for column in df.select_dtypes(include=[np.number]).columns:
            df[column] = df[column].astype(float)
        
        list_columns = ['industry_expertise', 'certifications']
        for col in list_columns:
            df[col] = df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        
        return df

    def _save_to_excel(self, df, filepath):
        """Save data to Excel with formatting"""
        with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Employee Data', index=False)
            
            summary_stats = pd.DataFrame({
                'Department Distribution': df['department'].value_counts(),
                'Average Salary by Level': df.groupby('level')['base_salary'].mean(),
                'Average Utilization': df.groupby('department')['actual_utilization'].mean()
            })
            summary_stats.to_excel(writer, sheet_name='Summary Stats')

    def _save_to_json(self, df, filepath):
        """Save data to JSON with proper formatting"""
        records = df.to_dict(orient='records')
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(records, f, indent=2, ensure_ascii=False)

    def generate_and_save_data(self, base_filename='consultancy_data'):
        """Generate and save data in both Excel and JSON formats"""
        df = self._prepare_data_for_export()
        
        os.makedirs('output', exist_ok=True)
        
        excel_path = os.path.join('output', f'{base_filename}.xlsx')
        json_path = os.path.join('output', f'{base_filename}.json')
        
        self._save_to_excel(df, excel_path)
        print(f"Excel file created successfully: {excel_path}")
        
        self._save_to_json(df, json_path)
        print(f"JSON file created successfully: {json_path}")
        
        print(f"Number of records: {len(df)}")

def main():
    generator = ConsultancyDataGenerator(num_records=1450)
    
    try:
        generator.generate_and_save_data()
        print("Data generation completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Excel file created successfully: output\consultancy_data.xlsx
JSON file created successfully: output\consultancy_data.json
Number of records: 1450
Data generation completed successfully!


In [ ]:
class ConsultancyEmployee:
 def __init__(self,faker,reference_date=datetime(2024,1,1)):
    self.fake=faker
    self.reference_date=reference_date
    self._init_department_config()
    self._init_location_config()
    self._init_specialization_config()
    self._init_certification_config()
    self.assigned_unique_positions=set()
 def _init_department_config(self):
    self.department_weights={'Product Architecture':0.2,'Design & UX':0.2,'Engineering Delivery':0.3,'Product Strategy':0.15,'Client Services':0.15}
    self.departments={'Product Architecture':{'entry':['Junior Solution Architect','System Analyst I','Technical Documentation Specialist'],'mid':['Solution Architect','Enterprise Architect','Integration Specialist'],'senior':['Principal Architect','Lead Solution Architect','Senior Enterprise Architect']},'Design & UX':{'entry':['UI Designer I','UX Researcher I','Design System Specialist'],'mid':['Senior UX Designer','Design Lead','Design Strategist'],'senior':['Principal Designer','Lead Design Strategist','Senior Design Lead']},'Engineering Delivery':{'entry':['Associate Developer','Junior DevOps','QA Engineer I'],'mid':['Tech Lead','Senior Developer','Cloud Architect'],'senior':['Principal Engineer','Engineering Lead','Development Director']},'Product Strategy':{'entry':['Product Analyst','Business Analyst I','Market Researcher'],'mid':['Product Strategist','Senior Product Manager','Domain Expert'],'senior':['Principal Product Manager','Lead Product Strategist','Senior Strategy Lead']},'Client Services':{'entry':['Project Coordinator','Client Success Associate','Engagement Analyst'],'mid':['Engagement Manager','Client Partner','Program Manager'],'senior':['Senior Program Director','Principal Client Partner','Senior Delivery Lead']}}
    self.executive_positions={'Product Architecture':['Chief Architect','Head of Architecture'],'Design & UX':['Head of Design','Chief Experience Officer'],'Engineering Delivery':['CTO','VP of Engineering'],'Product Strategy':['Head of Product','Chief Product Officer'],'Client Services':['Head of Client Services','Chief Customer Officer']}
    self.level_weights={'Product Architecture':{'entry':0.3,'mid':0.5,'senior':0.2},'Design & UX':{'entry':0.4,'mid':0.4,'senior':0.2},'Engineering Delivery':{'entry':0.5,'mid':0.3,'senior':0.2},'Product Strategy':{'entry':0.3,'mid':0.4,'senior':0.3},'Client Services':{'entry':0.2,'mid':0.5,'senior':0.3}}
 def _select_position(self,department,level):
    if level=='senior' and len(self.assigned_unique_positions)<len([pos for positions in self.executive_positions.values() for pos in positions]):
        available_exec_positions=set(self.executive_positions[department])-self.assigned_unique_positions
        if available_exec_positions and random.random()<0.2:
            position=random.choice(list(available_exec_positions))
            self.assigned_unique_positions.add(position)
            return position
    return random.choice(self.departments[department][level])
 def generate_employee(self,department=None,level=None,is_outlier=False):
    self.current_department=department or self._select_department()
    self.current_level=level or self._select_level(self.current_department)
    self.hire_date=self._generate_hire_date(self.current_level,is_outlier)
    self.current_location=self._select_location(self.current_department)
    return {'employee_id':self.fake.unique.random_number(digits=6),'full_name':self.fake.name(),'department':self.current_department,'position':self._select_position(self.current_department,self.current_level),'level':self.current_level,'hire_date':self.hire_date,**self._generate_location_data(),**self._generate_financial_metrics(),**self._generate_expertise_data(),**self._generate_performance_metrics(is_outlier),**self._generate_project_metrics(),**self._generate_development_metrics(),**self._generate_risk_metrics(),'manager_id':None,'total_comp':None,'team_lead_projects':0,'direct_reports':0}
 def _init_location_config(self):
    self.location_data={'London':{'country':'UK','region':'EMEA','cost_multiplier':1.2,'timezone':'GMT','market_demand':0.9},'New York':{'country':'USA','region':'Americas','cost_multiplier':1.3,'timezone':'EST','market_demand':1.0},'San Francisco':{'country':'USA','region':'Americas','cost_multiplier':1.35,'timezone':'PST','market_demand':1.1},'Singapore':{'country':'Singapore','region':'APAC','cost_multiplier':1.15,'timezone':'SGT','market_demand':0.95},'Berlin':{'country':'Germany','region':'EMEA','cost_multiplier':1.1,'timezone':'CET','market_demand':0.85}}
    self.location_weights={'Product Architecture':{'London':0.3,'New York':0.2,'San Francisco':0.2,'Singapore':0.15,'Berlin':0.15},'Design & UX':{'London':0.25,'New York':0.3,'San Francisco':0.2,'Singapore':0.15,'Berlin':0.1},'Engineering Delivery':{'London':0.2,'New York':0.25,'San Francisco':0.25,'Singapore':0.15,'Berlin':0.15},'Product Strategy':{'London':0.3,'New York':0.25,'San Francisco':0.2,'Singapore':0.15,'Berlin':0.1},'Client Services':{'London':0.3,'New York':0.2,'San Francisco':0.2,'Singapore':0.2,'Berlin':0.1}}
 def _init_specialization_config(self):
    self.specializations={'Product Architecture':['Cloud Infrastructure','Microservices','System Integration','API Design','Data Architecture','Security Architecture'],'Design & UX':['Mobile Design','Web Applications','Design Systems','User Research','Service Design','Design Thinking'],'Engineering Delivery':['Full Stack','Frontend','Backend','DevOps','Cloud Native','Mobile Development'],'Product Strategy':['Digital Transformation','Innovation Strategy','Go-to-Market','Product Discovery','Market Analysis'],'Client Services':['Agile Delivery','Digital Strategy','Change Management','Portfolio Management','Strategic Planning']}
 def _init_certification_config(self):
    self.certification_types={'Product Architecture':['AWS Solutions Architect','Azure Architect','TOGAF','Google Cloud Architect'],'Design & UX':['Google UX Design','Interaction Design Foundation','Human Factors International'],'Engineering Delivery':['AWS Developer','Kubernetes','Azure Developer','Scrum Master'],'Product Strategy':['Product Management (PMP)','Agile Product Management','PSPO'],'Client Services':['Prince2','PMP','Agile Project Management','Scrum Master']}
 def _select_department(self):
    return random.choices(list(self.department_weights.keys()),weights=list(self.department_weights.values()))[0]
 def _select_level(self,department):
    weights=self.level_weights[department]
    return random.choices(list(weights.keys()),weights=list(weights.values()))[0]
 def _select_location(self,department):
    weights=self.location_weights[department]
    return random.choices(list(weights.keys()),weights=list(weights.values()))[0]
 def _initialize_core_attributes(self):
    is_outlier=random.random()<0.05
    self.current_department=self._select_department()
    self.current_level=self._select_level(is_outlier)
    self.hire_date=self._generate_hire_date(self.current_level,is_outlier)
    self.current_location=self._select_location(self.current_department)
 def _generate_hire_date(self,level,is_outlier):
    experience_ranges={'entry':(0,2),'mid':(2,6),'senior':(5,12)}
    if is_outlier: experience_ranges={'entry':(2,4),'mid':(5,8),'senior':(10,15)}
    min_years,max_years=experience_ranges[level]
    days_ago=random.randint(min_years*365,max_years*365)
    return self.reference_date-timedelta(days=days_ago)
 def _get_position(self):
    return random.choice(self.departments[self.current_department][self.current_level])
 def _generate_location_data(self):
    location_info=self.location_data[self.current_location]
    return {'city':self.current_location,'country':location_info['country'],'region':location_info['region'],'remote_work_ratio':self._calculate_remote_ratio(),'travel_percentage':self._calculate_travel_percentage()}
 def _generate_financial_metrics(self):
    base_salary=self._calculate_base_salary()
    return {'base_salary':base_salary,'billing_rate':self._calculate_billing_rate(base_salary),'utilization_target':self._calculate_utilization_target(),'actual_utilization':self._calculate_actual_utilization()}
 def _calculate_base_salary(self):
    base_ranges={'entry':(65000,85000),'mid':(95000,145000),'senior':(150000,250000)}
    dept_multipliers={'Product Architecture':1.25,'Design & UX':1.15,'Engineering Delivery':1.20,'Product Strategy':1.18,'Client Services':1.10}
    min_salary,max_salary=base_ranges[self.current_level]
    base=random.uniform(min_salary,max_salary)
    location_multiplier=self.location_data[self.current_location]['cost_multiplier']
    dept_multiplier=dept_multipliers[self.current_department]
    experience_multiplier=self._calculate_experience_multiplier()
    final_salary=base*location_multiplier*dept_multiplier*experience_multiplier
    return round(final_salary,-3)
 def _calculate_experience_multiplier(self):
    years_experience=(self.reference_date-self.hire_date).days/365
    return 1+(min(years_experience,15)*0.025)
 def _calculate_billing_rate(self,base_salary):
    annual_cost=base_salary*2.5
    utilization_target=self._calculate_utilization_target()/100
    annual_billable_hours=2080*utilization_target
    base_rate=annual_cost/annual_billable_hours
    market_multipliers={'Product Architecture':1.4,'Design & UX':1.3,'Engineering Delivery':1.35,'Product Strategy':1.45,'Client Services':1.25}
    location_demand=self.location_data[self.current_location]['market_demand']
    final_rate=base_rate*market_multipliers[self.current_department]*location_demand
    return round(final_rate/50)*50
 def _calculate_utilization_target(self):
    base_targets={'Product Architecture':70,'Design & UX':75,'Engineering Delivery':85,'Product Strategy':65,'Client Services':80}
    level_adjustments={'entry':10,'mid':0,'senior':-15}
    target=base_targets[self.current_department]+level_adjustments[self.current_level]
    target+=random.uniform(-3,3)
    return round(min(max(target,50),90))
 def _calculate_actual_utilization(self):
    target=self._calculate_utilization_target()
    variation=random.uniform(-15,5)
    return round(min(max(target+variation,50),95),1)
 def _generate_expertise_data(self):
    return {'primary_specialization':self._select_specialization(),'secondary_specialization':self._select_specialization(),'industry_expertise':self._select_industry_expertise(),'certifications':self._assign_certifications()}
 def _select_specialization(self):
    specializations={'Product Architecture':['Cloud Infrastructure','Microservices','System Integration','API Design','Data Architecture','Security Architecture'],'Design & UX':['Mobile Design','Web Applications','Design Systems','User Research','Service Design','Design Thinking'],'Engineering Delivery':['Full Stack','Frontend','Backend','DevOps','Cloud Native','Mobile Development'],'Product Strategy':['Digital Transformation','Innovation Strategy','Go-to-Market','Product Discovery','Market Analysis'],'Client Services':['Agile Delivery','Digital Strategy','Change Management','Portfolio Management','Strategic Planning']}
    return random.choice(specializations[self.current_department])
 def _select_industry_expertise(self):
    industries=['FinTech','HealthTech','E-commerce','Enterprise Software','Social Impact','Consumer Tech','EdTech','Industrial Tech']
    num_industries=random.randint(2,4)
    return random.sample(industries,k=num_industries)
 def _assign_certifications(self):
    available_certs=self.certification_types.get(self.current_department,[])
    cert_counts={'entry':(0,2),'mid':(1,3),'senior':(2,4)}
    min_certs,max_certs=cert_counts[self.current_level]
    num_certs=random.randint(min_certs,min(max_certs,len(available_certs)))
    return random.sample(available_certs,k=num_certs)
 def _calculate_project_complexity(self,level,is_outlier):
    complexity_weights={'entry':[0.5,0.3,0.15,0.05,0],'mid':[0.1,0.3,0.4,0.15,0.05],'senior':[0,0.1,0.3,0.4,0.2]}
    if is_outlier:complexity_weights={'entry':[0.2,0.3,0.3,0.15,0.05],'mid':[0,0.15,0.25,0.4,0.2],'senior':[0,0,0.2,0.4,0.4]}
    return np.random.choice(range(1,6),p=complexity_weights[level])
 def _calculate_flight_risk(self,is_outlier):
    base_risk=20;years_in_role=(self.reference_date-self.hire_date).days/365
    engagement_score=self._generate_engagement_score(is_outlier);performance_score=self._generate_performance_score(self.current_level,is_outlier);utilization=self._calculate_actual_utilization()
    if years_in_role>4 and self.current_level!='senior':base_risk+=np.random.normal(15,3)
    if engagement_score<6:base_risk+=np.random.normal(20,4)
    elif engagement_score<7:base_risk+=np.random.normal(10,2)
    if performance_score>4.5:base_risk+=np.random.normal(12,3)
    elif performance_score<3.0:base_risk+=np.random.normal(15,3)
    market_demand=self.location_data[self.current_location]['market_demand']
    if market_demand>0.9:base_risk+=np.random.normal(8,2)
    high_demand_positions={'Solution Architect':12,'Cloud Architect':15,'Tech Lead':10,'Senior Developer':8,'Senior UX Designer':7,'Product Strategist':6}
    position_risk=high_demand_positions.get(self._get_position(),0)
    base_risk+=np.random.normal(position_risk,2)if position_risk>0 else 0;base_risk+=np.random.normal(0,3)
    return round(min(max(base_risk,15),95))
 def _calculate_promotion_readiness(self,level,hire_date,is_outlier):
    if level=='senior':return round(random.uniform(0,30),1)
    years_in_role=(self.reference_date-hire_date).days/365;performance_score=self._generate_performance_score(level,is_outlier)
    base_score=20;experience_score=min(years_in_role*8,24)
    performance_factor=25 if performance_score>=4.5 else 15 if performance_score>=4.0 else 8 if performance_score>=3.5 else 0
    cert_count=len(self._assign_certifications());skill_factor=min(cert_count*3,12);project_complexity=self._calculate_project_complexity(level,is_outlier);complexity_factor=project_complexity*3
    total_score=base_score+experience_score+performance_factor+skill_factor+complexity_factor
    total_score+=np.random.normal(0,2)
    if total_score>85:total_score=min(total_score,92)
    return round(min(max(total_score,15),92),1)
 def _calculate_innovation_score(self,level,department):
    base_innovation={'Product Architecture':7.5,'Design & UX':8.0,'Engineering Delivery':7.0,'Product Strategy':8.5,'Client Services':6.5}
    level_factors={'entry':0.7,'mid':1.0,'senior':1.3}
    performance_score=self._generate_performance_score(level,False);performance_factor=(performance_score-3)*0.2
    project_complexity=self._calculate_project_complexity(level,False);complexity_factor=(project_complexity-3)*0.15
    cert_count=len(self._assign_certifications());certification_factor=cert_count*0.1
    innovation=(base_innovation[department]*level_factors[level]*(1+performance_factor+complexity_factor+certification_factor))
    return round(min(max(innovation+random.uniform(-0.5,0.5),1),10),1)
 def _assign_office(self,department):
    weights={'Product Architecture':[0.3,0.2,0.2,0.15,0.15],'Design & UX':[0.25,0.3,0.2,0.15,0.1],'Engineering Delivery':[0.2,0.25,0.25,0.15,0.15],'Product Strategy':[0.3,0.25,0.2,0.15,0.1],'Client Services':[0.3,0.2,0.2,0.2,0.1]}
    office_key=np.random.choice(list(self.office_locations.keys()),p=weights[department])
    return self.office_locations[office_key]
 def _calculate_remote_ratio(self):
    base_remote={'entry':20,'mid':30,'senior':40}[self.current_level]
    return round(min(max(base_remote+random.uniform(-10,10),0),100),1)
 def _calculate_travel_percentage(self):
    base_travel={'entry':10,'mid':20,'senior':30}[self.current_level]
    dept_modifier={'Product Architecture':0,'Design & UX':-5,'Engineering Delivery':-10,'Product Strategy':15,'Client Services':20}[self.current_department]
    return round(min(max(base_travel+dept_modifier+random.uniform(-5,5),0),100),1)
 def _assign_industry_expertise(self):
    num_industries=random.randint(2,min(4,len(self.industry_expertise)))
    return random.sample(self.industry_expertise,k=num_industries)
 def _generate_performance_score(self,level,is_outlier):
    base_scores={'entry':np.random.normal(3.5,0.4),'mid':np.random.normal(3.8,0.3),'senior':np.random.normal(4.1,0.2)}
    score=base_scores[level]
    if is_outlier:score=random.choice([2.0,4.8])
    return round(min(max(score,1),5),1)
 def _generate_project_metrics(self):
    is_outlier=random.random()<0.05;base_projects={'entry':(1,2),'mid':(2,3),'senior':(2,4)}
    dept_modifiers={'Product Architecture':0,'Design & UX':-1,'Engineering Delivery':1,'Product Strategy':0,'Client Services':1}
    min_projects,max_projects=base_projects[self.current_level];modifier=dept_modifiers[self.current_department]
    min_projects=max(1,min_projects+modifier);max_projects=max(min_projects,max_projects+modifier)
    active_projects=random.randint(min_projects,max_projects)
    projects=[]
    for _ in range(active_projects):
        project={'complexity':self._calculate_project_complexity(self.current_level,is_outlier),'status':random.choice(['Planning','In Progress','Final Stage']),'duration_months':random.randint(3,12),'team_size':random.randint(3,10)}
        projects.append(project)
    avg_complexity=round(sum(p['complexity']for p in projects)/len(projects),1)
    avg_duration=round(sum(p['duration_months']for p in projects)/len(projects),1)
    avg_team_size=round(sum(p['team_size']for p in projects)/len(projects),1)
    return{'active_projects':active_projects,'avg_project_complexity':avg_complexity,'avg_project_duration':avg_duration,'avg_team_size':avg_team_size,'projects_on_time':self._calculate_projects_on_time(active_projects,is_outlier),'project_satisfaction':self._generate_client_satisfaction(self.current_level,self.current_department,projects,self._generate_performance_score(self.current_level,is_outlier))}
 def _calculate_projects_on_time(self,active_projects,is_outlier):
    if is_outlier:return random.choice([100,60])
    base_on_time={'entry':80,'mid':85,'senior':90}
    dept_modifiers={'Product Architecture':-5,'Design & UX':0,'Engineering Delivery':-3,'Product Strategy':2,'Client Services':5}
    on_time_pct=base_on_time[self.current_level]+dept_modifiers[self.current_department]
    on_time_pct+=random.uniform(-5,5)
    return round(min(max(on_time_pct,60),100),1)
 def _generate_development_metrics(self):
    is_outlier=random.random()<0.05
    return{'training_hours':self._calculate_training_hours(self.current_level,is_outlier),'mentorship_hours':self._calculate_mentorship_hours(self.current_level),'knowledge_sharing_score':self._calculate_knowledge_sharing(self.current_level,is_outlier),'promotion_readiness':self._calculate_promotion_readiness(self.current_level,self.hire_date,is_outlier)}
 def _generate_risk_metrics(self):
    is_outlier=random.random()<0.05;engagement_score=self._generate_engagement_score(is_outlier);flight_risk=self._calculate_flight_risk(is_outlier)
    return{'engagement_score':engagement_score,'flight_risk':flight_risk,'retention_risk':'High'if flight_risk>70 else'Medium'if flight_risk>40 else'Low'}
 def _generate_engagement_score(self,is_outlier):
    if is_outlier:return random.choice([4.0,9.5])
    base_engagement={'entry':np.random.normal(7.5,0.8),'mid':np.random.normal(7.8,0.7),'senior':np.random.normal(8.2,0.6)}
    score=base_engagement[self.current_level];dept_adjustments={'Product Architecture':0.2,'Design & UX':0.3,'Engineering Delivery':0.1,'Product Strategy':0.4,'Client Services':0.2}
    score+=dept_adjustments[self.current_department]
    return round(min(max(score,1),10),1)
 def _generate_performance_metrics(self,is_outlier):
    if is_outlier:
        performance_score=round(random.uniform(4.5,5.0),2);innovation_score=round(random.uniform(90,100),1);delivery_quality=round(random.uniform(95,100),1)
    else:
        performance_score=round(random.normalvariate(3.7,0.4),2);performance_score=max(min(performance_score,4.5),2.5)
        innovation_score=round(random.normalvariate(75,8),1);innovation_score=max(min(innovation_score,90),50)
        delivery_quality=round(random.normalvariate(82,7),1);delivery_quality=max(min(delivery_quality,95),60)
    return{'performance_score':performance_score,'innovation_score':innovation_score,'delivery_quality':delivery_quality}
 def _generate_project_data(self):
    statuses=['Planning','In Progress','Final Stage']
    return{'complexity':random.randint(1,5),'status':random.choice(statuses)}
 def _generate_client_satisfaction(self,level,department,projects,performance_score):
    if not projects:return None
    dept_base_satisfaction={'Product Architecture':4.2,'Design & UX':4.0,'Engineering Delivery':4.1,'Product Strategy':4.0,'Client Services':4.3}
    total_weight=0;weighted_satisfaction=0
    for project in projects:
        project_weight=1.0;base=dept_base_satisfaction[department]
        complexity=project['complexity'];complexity_factor={1:0.1,2:0.05,3:0,4:-0.05,5:-0.1}
        status_factor={'Planning':0.05,'In Progress':0,'Final Stage':-0.05}
        project_satisfaction=base;project_satisfaction+=complexity_factor[complexity]+status_factor[project['status']]
        performance_impact=(performance_score-3.5)*0.2;project_satisfaction+=performance_impact
        level_factor={'entry':-0.1,'mid':0,'senior':0.1};project_satisfaction+=level_factor[level]
        project_satisfaction+=random.uniform(-0.1,0.1);weighted_satisfaction+=project_satisfaction*project_weight;total_weight+=project_weight
    final_satisfaction=round(weighted_satisfaction/total_weight,1)
    return min(max(final_satisfaction,1.0),5.0)
 def _generate_internal_contribution(self,level):
    base_contribution={'entry':(1,3),'mid':(3,5),'senior':(4,7)}
    min_contrib,max_contrib=base_contribution[level]
    return random.randint(min_contrib,max_contrib)
 def _generate_delivery_quality(self,level,is_outlier):
    base_quality={'entry':np.random.normal(85,5),'mid':np.random.normal(90,4),'senior':np.random.normal(93,3)}
    quality=base_quality[level]
    if is_outlier:quality=random.choice([75,98])
    return round(min(max(quality,70),100),1)
 def _calculate_training_hours(self,level,is_outlier):
    base_hours={'entry':(40,60),'mid':(30,50),'senior':(20,40)}
    if is_outlier:return random.choice([10,100])
    min_hours,max_hours=base_hours[level]
    return random.randint(min_hours,max_hours)
 def _calculate_mentorship_hours(self,level):
    mentorship_ranges={'entry':(-30,-10),'mid':(-10,20),'senior':(20,40)}
    min_hours,max_hours=mentorship_ranges[level]
    return random.randint(min_hours,max_hours)
 def _calculate_knowledge_sharing(self,level,is_outlier):
    base_sharing={'entry':(3,6),'mid':(5,8),'senior':(7,9)}
    if is_outlier:
        if random.random()<0.8:return round(random.uniform(8.5,9.5),1)
        return round(random.uniform(2.5,3.5),1)
    min_share,max_share=base_sharing[level]
    if level=='entry':score=np.random.beta(4,3)*(max_share-min_share)+min_share
    elif level=='mid':score=np.random.beta(5,3)*(max_share-min_share)+min_share
    else:score=np.random.beta(6,2)*(max_share-min_share)+min_share
    return round(score,1)

class ConsultancyDataGenerator: 
 def __init__(self, num_records, seed=42): 
  self.fake, self.num_records = Faker(), num_records; Faker.seed(seed); np.random.seed(seed); random.seed(seed) 
  self.employee_generator = ConsultancyEmployee(self.fake); self.departments = list(self.employee_generator.departments.keys()) 
  self.levels = ['entry', 'mid', 'senior']; self.department_weights, self.level_weights = self.employee_generator.department_weights, self.employee_generator.level_weights 
  self.senior_ratio = 0.03 if num_records >= 1000 else 0.05; self.mid_ratio = 0.10 if num_records >= 1000 else 0.12 if num_records >= 100 else 0.15 
  self.max_senior_managers = max(1, int(num_records * self.senior_ratio)); self.max_mid_managers = max(2, int(num_records * self.mid_ratio)) 
  self.max_direct_reports_senior = 10 if num_records >= 1000 else 8 if num_records >= 100 else 6; self.max_direct_reports_mid = 8 if num_records >= 1000 else 6 if num_records >= 100 else 4 
 def _select_department(self):
    weights = list(self.department_weights.values())
    if not all(w > 0 for w in weights) or abs(sum(weights) - 1) > 1e-9:
        total = sum(weights)
        weights = [w/total if w > 0 else 0.01 for w in weights]
        weights = [w/sum(weights) for w in weights]
    return random.choices(list(self.department_weights.keys()), weights=weights)[0]
 def _select_level(self, department):
    weights = list(self.level_weights[department].values())
    if not all(w > 0 for w in weights) or abs(sum(weights) - 1) > 1e-9:
        total = sum(weights)
        weights = [w/total if w > 0 else 0.01 for w in weights]
        weights = [w/sum(weights) for w in weights]
    return random.choices(list(self.level_weights[department].keys()), weights=weights)[0]
 def _assign_managers(self, df):
    if df.empty: return df
    df['is_manager'], df['management_level'], df['direct_reports'] = False, None, 0
    for dept in self.departments:
        dept_mask = df['department'] == dept
        if not any(dept_mask): continue
        dept_indices = df[dept_mask].index
        senior_mask = (df['level'] == 'senior') & dept_mask & df['position'].str.contains('Director|Head|Chief|Lead', case=False, na=False)
        mid_mask = (df['level'] == 'mid') & dept_mask & df['position'].str.contains('Lead|Senior|Manager', case=False, na=False)
        dept_size = len(dept_indices)
        if dept_size == 0: continue
        dept_senior_managers, dept_mid_managers = max(1, int(dept_size * self.senior_ratio)), max(2, int(dept_size * self.mid_ratio))
        senior_candidates, mid_candidates = df[senior_mask].sort_values('performance_score', ascending=False), df[mid_mask].sort_values('performance_score', ascending=False)
        max_senior, max_mid = min(dept_senior_managers, len(senior_candidates), int(self.max_senior_managers / len(self.departments))), min(dept_mid_managers, len(mid_candidates), int(self.max_mid_managers / len(self.departments)))
        if max_senior == 0 or max_mid == 0: continue
        senior_manager_indices, mid_manager_indices = senior_candidates.head(max_senior).index, mid_candidates.head(max_mid).index
        df.loc[senior_manager_indices, 'is_manager'], df.loc[senior_manager_indices, 'management_level'] = True, 'senior'
        df.loc[mid_manager_indices, 'is_manager'], df.loc[mid_manager_indices, 'management_level'] = True, 'mid'
        mid_level_mask = (df['level'] == 'mid') & dept_mask & ~df.index.isin(mid_manager_indices)
        for idx in df[mid_level_mask].index:
            valid_managers = df.loc[senior_manager_indices]
            if not valid_managers.empty:
                valid_managers = valid_managers[valid_managers['direct_reports'] < self.max_direct_reports_senior]
                if not valid_managers.empty:
                    manager = valid_managers.sample(n=1)
                    df.loc[idx, 'manager_id'], df.loc[manager.index[0], 'direct_reports'] = manager.iloc[0]['employee_id'], df.loc[manager.index[0], 'direct_reports'] + 1
        for idx in df[(df['level'] == 'entry') & dept_mask].index:
            valid_mid_managers = df.loc[mid_manager_indices]
            valid_mid_managers = valid_mid_managers[valid_mid_managers['direct_reports'] < self.max_direct_reports_mid]
            if not valid_mid_managers.empty:
                manager = valid_mid_managers.sample(n=1)
                df.loc[idx, 'manager_id'], df.loc[manager.index[0], 'direct_reports'] = manager.iloc[0]['employee_id'], df.loc[manager.index[0], 'direct_reports'] + 1
            else:
                valid_senior_managers = df.loc[senior_manager_indices]
                valid_senior_managers = valid_senior_managers[valid_senior_managers['direct_reports'] < self.max_direct_reports_senior]
                if not valid_senior_managers.empty:
                    manager = valid_senior_managers.sample(n=1)
                    df.loc[idx, 'manager_id'], df.loc[manager.index[0], 'direct_reports'] = manager.iloc[0]['employee_id'], df.loc[manager.index[0], 'direct_reports'] + 1
    df['span_of_control'] = df['direct_reports'].apply(lambda x: 'None' if x == 0 else 'Small (1-3)' if x <= 3 else 'Medium (4-6)' if x <= 6 else 'Large (7+)')
    return df
 def _calculate_management_compensation_adjustment(self, df):
    if df.empty: return df
    df['management_premium'], df['span_premium'] = 0.0, 0.0
    senior_mask = df['is_manager'] & (df['management_level'] == 'senior')
    if any(senior_mask): df.loc[senior_mask, 'management_premium'] = df.loc[senior_mask, 'base_salary'] * np.random.uniform(0.25, 0.35, size=senior_mask.sum())
    mid_mask = df['is_manager'] & (df['management_level'] == 'mid')
    if any(mid_mask): df.loc[mid_mask, 'management_premium'] = df.loc[mid_mask, 'base_salary'] * np.random.uniform(0.15, 0.25, size=mid_mask.sum())
    df['span_premium'] = df['direct_reports'].apply(lambda x: 0.0 if x == 0 else 0.05 if x <= 3 else 0.10 if x <= 6 else 0.15) * df['base_salary']
    df['total_comp'] = df['base_salary'] + df['management_premium'] + df['span_premium']
    return df
 def _update_team_metrics(self, df): 
  manager_counts = df['manager_id'].value_counts(); manager_mask = df['employee_id'].isin(manager_counts.index) 
  df.loc[manager_mask, 'direct_reports'] = df.loc[manager_mask, 'employee_id'].map(manager_counts) 
  senior_mask, mid_mask = df['level'] == 'senior', df['level'] == 'mid' 
  df.loc[senior_mask, 'team_lead_projects'] = df.loc[senior_mask, 'active_projects'].apply(lambda x: random.randint(1, max(1, x)) if pd.notnull(x) else 0) 
  df.loc[mid_mask, 'team_lead_projects'] = df.loc[mid_mask, 'active_projects'].apply(lambda x: random.randint(0, max(0, x-1)) if pd.notnull(x) else 0) 
  return df 
 def _assign_outliers(self):
    total_outliers=max(len(self.departments),int(self.num_records*0.02))
    self.outlier_assignments={dept:{level:0 for level in self.levels}for dept in self.departments}
    for dept in self.departments:
        level=random.choice(self.levels);self.outlier_assignments[dept][level]+=1;total_outliers-=1
    while total_outliers>0:
        dept=random.choice(self.departments);level=random.choice(self.levels)
        self.outlier_assignments[dept][level]+=1;total_outliers-=1
 def _get_expected_count(self,department,level):
    return int(self.num_records*self.department_weights[department]*self.level_weights[department][level])
 def _calculate_total_compensation(self,df):
    df['total_comp']=df.apply(lambda row:row['base_salary']*(1+(0.2 if row['level']=='senior' else 0.15 if row['level']=='mid' else 0.1)+(0.05 if row['performance_score']>=4.5 else 0)),axis=1)
 def generate_dataset(self):
    try:
        employees=[self.employee_generator.generate_employee() for _ in range(self.num_records)]
        df=pd.DataFrame(employees)
        required_columns=['employee_id','department','level','position','base_salary']
        missing_columns=[col for col in required_columns if col not in df.columns]
        if missing_columns:raise ValueError(f"Missing required columns: {missing_columns}")
        df=self._assign_managers(df)
        df=self._calculate_management_compensation_adjustment(df)
        df=self._update_team_metrics(df)
        return df
    except Exception as e:print(f"Error generating dataset: {str(e)}");raise
 def _prepare_data_for_export(self):
    df=self.generate_dataset()
    df['hire_date']=df['hire_date'].dt.strftime('%Y-%m-%d')
    for column in df.select_dtypes(include=[np.number]).columns: df[column]=df[column].astype(float)
    for col in ['industry_expertise', 'certifications']: df[col]=df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
    return df

#### Import_hr_data code

In [ ]:
import pandas as pd, psycopg2
from psycopg2.extras import execute_values
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class HRDatabaseImporter:
    def __init__(self, dbname="hr_resource_db", user="postgres", password="1234", host="localhost", port="5432"):
        self.conn_params = {"dbname": dbname, "user": user, "password": password, "host": host, "port": port}
        self.conn, self.cur, self.column_mappings = None, None, {
            'employee_id': ['employee_id', 'employeeid', 'id'], 'full_name': ['full_name', 'fullname', 'name'],
            'department': ['department', 'dept'], 'position': ['position', 'job_title', 'role'],
            'level': ['position_level', 'level', 'job_level'], 'hire_date': ['hire_date', 'hiredate', 'start_date'],
            'city': ['city', 'location'], 'country': ['country', 'nation'], 'region': ['region', 'area'],
            'performance_score': ['performance_score', 'perf_score'], 'innovation_score': ['innovation_score', 'innovation'],
            'delivery_quality': ['delivery_quality', 'quality'], 'engagement_score': ['engagement_score', 'engagement'],
            'knowledge_sharing_score': ['knowledge_sharing_score', 'knowledge_sharing'], 'active_projects': ['active_projects', 'current_projects'],
            'avg_project_complexity': ['avg_project_complexity', 'project_complexity'], 'avg_project_duration': ['avg_project_duration', 'project_duration'],
            'avg_team_size': ['avg_team_size', 'team_size'], 'projects_on_time': ['projects_on_time', 'ontime_projects'],
            'project_satisfaction': ['project_satisfaction', 'proj_satisfaction'], 'team_lead_projects': ['team_lead_projects', 'lead_projects'],
            'flight_risk': ['flight_risk', 'attrition_risk'], 'retention_risk': ['retention_risk', 'retention'], 'promotion_readiness': ['promotion_readiness', 'readiness'],
            'is_manager': ['is_manager', 'manager'], 'management_level': ['management_level', 'manager_level'],
            'span_of_control': ['span_of_control', 'direct_reports'], 'management_premium': ['management_premium', 'mgmt_premium'],
            'span_premium': ['span_premium'], 'total_comp': ['total_comp', 'total_compensation']
        }

    def find_column_name(self, df, possible_names):
        for name in possible_names:
            if name in df.columns: return name
        return None

    def validate_required_columns(self, df):
        missing_columns, column_mapping_result = [], {}
        for required_col, possible_names in self.column_mappings.items():
            found_name = self.find_column_name(df, possible_names)
            if found_name is None: missing_columns.append(required_col)
            else: column_mapping_result[required_col] = found_name
        if missing_columns: raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
        return column_mapping_result

    def connect(self):
        try:
            self.conn = psycopg2.connect(**self.conn_params)
            self.cur = self.conn.cursor()
            logging.info("Successfully connected to database")
        except Exception as e:
            logging.error(f"Database connection error: {e}")
            raise

    def create_tables(self):
        try:
            tables_sql = [
                """CREATE TABLE IF NOT EXISTS employees (employee_id INTEGER PRIMARY KEY, full_name VARCHAR(100), department VARCHAR(50), position VARCHAR(100), position_level VARCHAR(20), hire_date DATE, city VARCHAR(50), country VARCHAR(50), region VARCHAR(20))""",
                """CREATE TABLE IF NOT EXISTS performance_metrics (metric_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), performance_score DECIMAL(4,2), innovation_score DECIMAL(5,2), delivery_quality DECIMAL(5,2), engagement_score DECIMAL(4,2), knowledge_sharing_score DECIMAL(4,2), UNIQUE(employee_id))""",
                """CREATE TABLE IF NOT EXISTS project_metrics (project_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), active_projects INTEGER, avg_project_complexity DECIMAL(4,2), avg_project_duration DECIMAL(4,2), avg_team_size DECIMAL(4,2), projects_on_time DECIMAL(5,2), project_satisfaction DECIMAL(4,2), team_lead_projects INTEGER, UNIQUE(employee_id))""",
                """CREATE TABLE IF NOT EXISTS risk_assessment (risk_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), flight_risk INTEGER, retention_risk VARCHAR(20), promotion_readiness DECIMAL(5,2), risk_factors TEXT[], UNIQUE(employee_id))""",
                """CREATE TABLE IF NOT EXISTS management_info (management_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), is_manager BOOLEAN, management_level VARCHAR(20), span_of_control INTEGER, management_premium NUMERIC(10,2), span_premium NUMERIC(10,2), total_comp NUMERIC(10,2), UNIQUE(employee_id))"""
            ]
            for table_sql in tables_sql:
                self.cur.execute(table_sql)
            self.conn.commit()
            logging.info("Successfully created all tables")
        except Exception as e:
            self.conn.rollback()
            logging.error(f"Error creating tables: {e}")
            raise

    def safe_numeric(self, value):
        try: return float(value) if pd.notna(value) else None
        except: return None

    def safe_date(self, value):
        if pd.isna(value): return None
        try: return pd.to_datetime(value).date()
        except: return None

    def calculate_risk_factors(self, row, cols):
        risk_factors = []
        if self.safe_numeric(row.get(cols['engagement_score'])) < 7.0: risk_factors.append('low_engagement')
        if self.safe_numeric(row.get(cols['performance_score'])) < 3.5: risk_factors.append('low_performance')
        if self.safe_numeric(row.get('actual_utilization', 0)) < 70: risk_factors.append('low_utilization')
        if self.safe_numeric(row.get(cols['promotion_readiness'])) < 40 and str(row.get(cols['level'], '')).lower() != 'senior': risk_factors.append('promotion_due')
        if self.safe_numeric(row.get(cols['project_satisfaction'])) < 4.0: risk_factors.append('low_satisfaction')
        return risk_factors

    def import_data(self, df):
        try:
            cols = self.validate_required_columns(df)
            self.cur.execute("BEGIN")
            employee_data = [(int(row[cols['employee_id']]), str(row[cols['full_name']]), str(row[cols['department']]), str(row[cols['position']]), str(row[cols['level']]), self.safe_date(row[cols['hire_date']]), str(row[cols['city']]), str(row[cols['country']]), str(row[cols['region']])) for _, row in df.iterrows()]
            execute_values(self.cur, """INSERT INTO employees (employee_id, full_name, department, position, position_level, hire_date, city, country, region) VALUES %s ON CONFLICT (employee_id) DO UPDATE SET full_name = EXCLUDED.full_name, department = EXCLUDED.department, position = EXCLUDED.position, position_level = EXCLUDED.position_level, hire_date = EXCLUDED.hire_date, city = EXCLUDED.city, country = EXCLUDED.country, region = EXCLUDED.region""", employee_data)
            performance_data = [(int(row[cols['employee_id']]), self.safe_numeric(row[cols['performance_score']]), self.safe_numeric(row[cols['innovation_score']]), self.safe_numeric(row[cols['delivery_quality']]), self.safe_numeric(row[cols['engagement_score']]), self.safe_numeric(row[cols['knowledge_sharing_score']])) for _, row in df.iterrows()]
            execute_values(self.cur, """INSERT INTO performance_metrics (employee_id, performance_score, innovation_score, delivery_quality, engagement_score, knowledge_sharing_score) VALUES %s ON CONFLICT (employee_id) DO UPDATE SET performance_score = EXCLUDED.performance_score, innovation_score = EXCLUDED.innovation_score, delivery_quality = EXCLUDED.delivery_quality, engagement_score = EXCLUDED.engagement_score, knowledge_sharing_score = EXCLUDED.knowledge_sharing_score""", performance_data)
            risk_data = [(int(row['employee_id']), int(row['flight_risk']), row['retention_risk'], self.safe_numeric(row['promotion_readiness']), self.calculate_risk_factors(row, cols)) for _, row in df.iterrows()]
            execute_values(self.cur, """INSERT INTO risk_assessment (employee_id, flight_risk, retention_risk, promotion_readiness, risk_factors) VALUES %s ON CONFLICT (employee_id) DO UPDATE SET flight_risk = EXCLUDED.flight_risk, retention_risk = EXCLUDED.retention_risk, promotion_readiness = EXCLUDED.promotion_readiness, risk_factors = EXCLUDED.risk_factors""", risk_data)
            management_data = [(int(row['employee_id']), row['is_manager'], row['management_level'], int(row['span_of_control']), self.safe_numeric(row['management_premium']), self.safe_numeric(row['span_premium']), self.safe_numeric(row['total_comp'])) for _, row in df.iterrows() if row.get('is_manager')]
            execute_values(self.cur, """INSERT INTO management_info (employee_id, is_manager, management_level, span_of_control, management_premium, span_premium, total_comp) VALUES %s ON CONFLICT (employee_id) DO UPDATE SET is_manager = EXCLUDED.is_manager, management_level = EXCLUDED.management_level, span_of_control = EXCLUDED.span_of_control, management_premium = EXCLUDED.management_premium, span_premium = EXCLUDED.span_premium, total_comp = EXCLUDED.total_comp""", management_data)
            self.conn.commit()
            logging.info("Data imported successfully")
        except Exception as e:
            self.conn.rollback()
            logging.error(f"Data import error: {e}")
            raise
        finally:
            self.conn.close()

if __name__ == "__main__":
    df = pd.read_csv('employee_data.csv')  # Example path, replace with your data source
    importer = HRDatabaseImporter()
    try:
        importer.connect()
        importer.create_tables()
        importer.import_data(df)
    except Exception as e:
        logging.error(f"Error during import process: {e}")

#### Continued

In [ ]:
import pandas as pd, psycopg2, os, sys
from datetime import datetime

def create_hr_database():
    try:
        conn=psycopg2.connect(dbname="hr_resource_db",user="postgres",password="1234",host="localhost",port="5432")
        print("Successfully connected to database")
        cur=conn.cursor()
        cur.execute("""CREATE TABLE IF NOT EXISTS employees (employee_id INTEGER PRIMARY KEY, full_name VARCHAR(100), department VARCHAR(50), position VARCHAR(100), position_level VARCHAR(20), hire_date DATE, city VARCHAR(50), country VARCHAR(50), region VARCHAR(20), remote_work_ratio DECIMAL(5,2), travel_percentage DECIMAL(5,2), base_salary NUMERIC(10,2), billing_rate NUMERIC(10,2), utilization_target DECIMAL(5,2), actual_utilization DECIMAL(5,2), primary_specialization VARCHAR(50), secondary_specialization VARCHAR(50), industry_expertise TEXT, is_manager BOOLEAN, management_level VARCHAR(20), span_of_control INTEGER, management_premium NUMERIC(10,2), span_premium NUMERIC(10,2), total_comp NUMERIC(10,2))""")
        cur.execute("""CREATE TABLE IF NOT EXISTS performance_metrics (metric_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), performance_score DECIMAL(4,2), innovation_score DECIMAL(5,2), delivery_quality DECIMAL(5,2), engagement_score DECIMAL(4,2), flight_risk INTEGER, retention_risk VARCHAR(20), promotion_readiness DECIMAL(5,2), knowledge_sharing_score DECIMAL(4,2))""")
        cur.execute("""CREATE TABLE IF NOT EXISTS project_metrics (project_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), active_projects INTEGER, avg_project_complexity DECIMAL(3,1), avg_project_duration DECIMAL(4,1), avg_team_size DECIMAL(4,1), projects_on_time DECIMAL(5,2), project_satisfaction DECIMAL(3,1), team_lead_projects INTEGER)""")
        cur.execute("""CREATE TABLE IF NOT EXISTS development_metrics (dev_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), training_hours INTEGER, mentorship_hours INTEGER, direct_reports INTEGER)""")
        cur.execute("""CREATE TABLE IF NOT EXISTS employee_certifications (cert_id SERIAL PRIMARY KEY, employee_id INTEGER REFERENCES employees(employee_id), certification_name VARCHAR(200), UNIQUE(employee_id,certification_name))""")
        return conn,cur
    except psycopg2.Error as e:print(f"Database connection error: {e}");raise

def safe_numeric(value): 
    try:return float(value)if pd.notna(value)else None 
    except: return None

def safe_date(value): 
    if pd.isna(value):return None
    try: return pd.to_datetime(value).date() 
    except:return None

def insert_employee_data(df,conn,cur):
    for _,row in df.iterrows():
        try:
            industry_expertise=', '.join(str(row['industry_expertise']).split(', '))if pd.notna(row['industry_expertise'])else None
            cur.execute("""INSERT INTO employees (employee_id, full_name, department, position, position_level, hire_date, city, country, region, remote_work_ratio, travel_percentage, base_salary, billing_rate, utilization_target, actual_utilization, primary_specialization, secondary_specialization, industry_expertise, is_manager, management_level, span_of_control, management_premium, span_premium, total_comp) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (employee_id) DO UPDATE SET full_name=EXCLUDED.full_name, department=EXCLUDED.department, position=EXCLUDED.position""",(int(row['employee_id']),row['full_name'],row['department'],row['position'],row['position_level'],safe_date(row['hire_date']),row['city'],row['country'],row['region'],safe_numeric(row['remote_work_ratio']),safe_numeric(row['travel_percentage']),safe_numeric(row['base_salary']),safe_numeric(row['billing_rate']),safe_numeric(row['utilization_target']),safe_numeric(row['actual_utilization']),row['primary_specialization'],row['secondary_specialization'],industry_expertise,row['is_manager'],row['management_level'],safe_numeric(row['span_of_control']),safe_numeric(row['management_premium']),safe_numeric(row['span_premium']),safe_numeric(row['total_comp'])))
        except Exception as e: print(f"Error inserting employee {row['employee_id']}: {e}")

def insert_performance_metrics(df,conn,cur):
    for _,row in df.iterrows():
        try: cur.execute("""INSERT INTO performance_metrics (employee_id, performance_score, innovation_score, delivery_quality, engagement_score, flight_risk, retention_risk, promotion_readiness, knowledge_sharing_score) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)""",(int(row['employee_id']),safe_numeric(row['performance_score']),safe_numeric(row['innovation_score']),safe_numeric(row['delivery_quality']),safe_numeric(row['engagement_score']),safe_numeric(row['flight_risk']),row['retention_risk'],safe_numeric(row['promotion_readiness']),safe_numeric(row['knowledge_sharing_score'])))
        except Exception as e: print(f"Error inserting performance metrics for employee {row['employee_id']}: {e}")

def insert_project_metrics(df,conn,cur):
    for _,row in df.iterrows():
        try: cur.execute("""INSERT INTO project_metrics (employee_id, active_projects, avg_project_complexity, avg_project_duration, avg_team_size, projects_on_time, project_satisfaction, team_lead_projects) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""",(int(row['employee_id']),safe_numeric(row['active_projects']),safe_numeric(row['avg_project_complexity']),safe_numeric(row['avg_project_duration']),safe_numeric(row['avg_team_size']),safe_numeric(row['projects_on_time']),safe_numeric(row['project_satisfaction']),safe_numeric(row['team_lead_projects'])))
        except Exception as e: print(f"Error inserting project metrics for employee {row['employee_id']}: {e}")

def insert_development_metrics(df,conn,cur):
    for _,row in df.iterrows():
        try: cur.execute("""INSERT INTO development_metrics (employee_id, training_hours, mentorship_hours, direct_reports) VALUES (%s, %s, %s, %s)""",(int(row['employee_id']),safe_numeric(row['training_hours']),safe_numeric(row['mentorship_hours']),safe_numeric(row['direct_reports'])))
        except Exception as e: print(f"Error inserting development metrics for employee {row['employee_id']}: {e}")

def insert_certifications(df,conn,cur):
    for _,row in df.iterrows():
        if pd.notna(row['certifications']):
            certs=str(row['certifications']).split(',')
            for cert in certs:
                try: cur.execute("""INSERT INTO employee_certifications (employee_id, certification_name) VALUES (%s, %s) ON CONFLICT (employee_id, certification_name) DO NOTHING""",(int(row['employee_id']),cert.strip()))
                except Exception as e: print(f"Error inserting certification for employee {row['employee_id']}: {e}")

def main():
    try:
        print("Starting enhanced HR data import process...")
        conn,cur=create_hr_database()
        excel_file="path/to/your/hr_data.xlsx"
        if not os.path.exists(excel_file):print(f"Error: Excel file not found at {excel_file}");sys.exit(1)
        print(f"Reading Excel file from: {excel_file}")
        df=pd.read_excel(excel_file)
        print(f"\nDataframe shape: {df.shape}")
        print(f"Columns found: {', '.join(df.columns)}")
        if len(df)==0:print("Warning: Excel file contains no data!");sys.exit(1)
        print("\nInserting data into tables...")
        insert_employee_data(df,conn,cur)
        insert_performance_metrics(df,conn,cur)
        insert_project_metrics(df,conn,cur)
        insert_development_metrics(df,conn,cur)
        insert_certifications(df,conn,cur)
        conn.commit()
        tables=['employees','performance_metrics','project_metrics','development_metrics','employee_certifications']
        print("\nVerifying data insertion:")
        for table in tables:
            cur.execute(f"SELECT COUNT(*) FROM {table}")
            count=cur.fetchone()[0]
            print(f"{table}: {count} rows")
        print("\nHR data import process completed successfully!")
    except Exception as e:
        print(f"Error during import process: {str(e)}")
        print(f"Error type: {type(e).__name__}")
        sys.exit(1)
    finally:
        if 'conn' in locals():conn.close()

if __name__=="__main__":main()


In [ ]:
class ConsultancyEmployee:
    def __init__(self, faker, reference_date=datetime(2024, 1, 1)):
        self.fake=faker;self.reference_date=reference_date
        self._init_department_config();self._init_location_config();self._init_specialization_config();self._init_certification_config()

    def _init_department_config(self):
        self.department_weights={'Product Architecture':0.2,'Design & UX':0.2,'Engineering Delivery':0.3,'Product Strategy':0.15,'Client Services':0.15}
        self.departments={'Product Architecture':{'entry':['Junior Solution Architect','System Analyst I','Technical Documentation Specialist'],'mid':['Solution Architect','Enterprise Architect','Integration Specialist'],'senior':['Chief Architect','Head of Architecture','Technical Director']},
                          'Design & UX':{'entry':['UI Designer I','UX Researcher I','Design System Specialist'],'mid':['Senior UX Designer','Design Lead','Design Strategist'],'senior':['Design Director','Head of Design','Chief Experience Officer']},
                          'Engineering Delivery':{'entry':['Associate Developer','Junior DevOps','QA Engineer I'],'mid':['Tech Lead','Senior Developer','Cloud Architect'],'senior':['Delivery Director','Engineering Manager','CTO']},
                          'Product Strategy':{'entry':['Product Analyst','Business Analyst I','Market Researcher'],'mid':['Product Strategist','Senior Product Manager','Domain Expert'],'senior':['Head of Product','Strategy Director','Innovation Lead']},
                          'Client Services':{'entry':['Project Coordinator','Client Success Associate','Engagement Analyst'],'mid':['Engagement Manager','Client Partner','Program Manager'],'senior':['Account Director','Head of Client Services','Partnership Director']}}
        self.level_weights={'Product Architecture':[0.3,0.5,0.2],'Design & UX':[0.4,0.4,0.2],'Engineering Delivery':[0.5,0.3,0.2],'Product Strategy':[0.3,0.4,0.3],'Client Services':[0.2,0.5,0.3]}
        self.dept_salary_multipliers={'Product Architecture':1.25,'Design & UX':1.15,'Engineering Delivery':1.2,'Product Strategy':1.18,'Client Services':1.1}

    def _init_location_config(self):
        self.location_data={'London':{'country':'UK','region':'EMEA','cost_multiplier':1.2,'timezone':'GMT','market_demand':0.9},
                            'New York':{'country':'USA','region':'Americas','cost_multiplier':1.3,'timezone':'EST','market_demand':1.0},
                            'San Francisco':{'country':'USA','region':'Americas','cost_multiplier':1.35,'timezone':'PST','market_demand':1.1},
                            'Singapore':{'country':'Singapore','region':'APAC','cost_multiplier':1.15,'timezone':'SGT','market_demand':0.95},
                            'Berlin':{'country':'Germany','region':'EMEA','cost_multiplier':1.1,'timezone':'CET','market_demand':0.85}}
        self.location_weights={'Product Architecture':{'London':0.3,'New York':0.2,'San Francisco':0.2,'Singapore':0.15,'Berlin':0.15},
                               'Design & UX':{'London':0.25,'New York':0.3,'San Francisco':0.2,'Singapore':0.15,'Berlin':0.1},
                               'Engineering Delivery':{'London':0.2,'New York':0.25,'San Francisco':0.25,'Singapore':0.15,'Berlin':0.15},
                               'Product Strategy':{'London':0.3,'New York':0.25,'San Francisco':0.2,'Singapore':0.15,'Berlin':0.1},
                               'Client Services':{'London':0.3,'New York':0.2,'San Francisco':0.2,'Singapore':0.2,'Berlin':0.1}}

    def _init_specialization_config(self):
        self.specializations={'Product Architecture':['Cloud Infrastructure','Microservices','System Integration','API Design','Data Architecture','Security Architecture'],
                              'Design & UX':['Mobile Design','Web Applications','Design Systems','User Research','Service Design','Design Thinking'],
                              'Engineering Delivery':['Full Stack','Frontend','Backend','DevOps','Cloud Native','Mobile Development'],
                              'Product Strategy':['Digital Transformation','Innovation Strategy','Go-to-Market','Product Discovery','Market Analysis'],
                              'Client Services':['Agile Delivery','Digital Strategy','Change Management','Portfolio Management','Strategic Planning']}

    def _init_certification_config(self):
        self.certification_types={'Product Architecture':['AWS Solutions Architect','Azure Architect','TOGAF','Google Cloud Architect'],
                                  'Design & UX':['Google UX Design','Interaction Design Foundation','Human Factors International'],
                                  'Engineering Delivery':['AWS Developer','Kubernetes','Azure Developer','Scrum Master'],
                                  'Product Strategy':['Product Management (PMP)','Agile Product Management','PSPO'],
                                  'Client Services':['Prince2','PMP','Agile Project Management','Scrum Master']}

    def generate_employee(self):
        self._initialize_core_attributes()
        return {'employee_id':self.fake.unique.random_number(digits=6),'full_name':self.fake.name(),'department':self.current_department,'position':self._get_position(),'level':self.current_level,'hire_date':self.hire_date,**self._generate_location_data(),**self._generate_financial_metrics(),**self._generate_expertise_data(),**self._generate_performance_metrics(),**self._generate_project_metrics(),**self._generate_development_metrics(),**self._generate_risk_metrics(),'manager_id':None,'total_comp':None,'team_lead_projects':0,'direct_reports':0}

    def _initialize_core_attributes(self):
        is_outlier=random.random()<0.05;self.current_department=self._select_department()
        self.current_level=self._select_level(is_outlier);self.hire_date=self._generate_hire_date(self.current_level,is_outlier)
        self.current_location=self._select_location(self.current_department)

    def _generate_hire_date(self, level, is_outlier):
        experience_ranges={'entry':(0,2),'mid':(2,6),'senior':(5,12)} if not is_outlier else {'entry':(2,4),'mid':(5,8),'senior':(10,15)}
        min_years,max_years=experience_ranges[level]
        days_ago=random.randint(min_years*365,max_years*365)
        return self.reference_date-timedelta(days=days_ago)

    def _generate_location_data(self):
        location_info=self.location_data[self.current_location]
        return {'city':self.current_location,'country':location_info['country'],'region':location_info['region'],'remote_work_ratio':self._calculate_remote_ratio(),'travel_percentage':self._calculate_travel_percentage()}

    def _generate_financial_metrics(self):
        base_salary=self._calculate_base_salary()
        return {'base_salary':base_salary,'billing_rate':self._calculate_billing_rate(base_salary),'utilization_target':self._calculate_utilization_target(),'actual_utilization':self._calculate_actual_utilization()}

    def _calculate_base_salary(self):
        base_ranges={'entry':(65000,85000),'mid':(95000,145000),'senior':(150000,250000)}
        dept_multipliers={'Product Architecture':1.25,'Design & UX':1.15,'Engineering Delivery':1.2,'Product Strategy':1.18,'Client Services':1.1}
        min_salary,max_salary=base_ranges[self.current_level];base=random.uniform(min_salary,max_salary)
        location_multiplier=self.location_data[self.current_location]['cost_multiplier'];dept_multiplier=dept_multipliers[self.current_department]
        experience_multiplier=self._calculate_experience_multiplier()
        return round(base*location_multiplier*dept_multiplier*experience_multiplier,-3)
    
        def _calculate_experience_multiplier(self):
        years_experience=(self.reference_date-self.hire_date).days/365
        return 1+(min(years_experience,15)*0.025)

    def _calculate_billing_rate(self, base_salary):
        annual_cost=base_salary*2.5;utilization_target=self._calculate_utilization_target()/100
        annual_billable_hours=2080*utilization_target;base_rate=annual_cost/annual_billable_hours
        market_multipliers={'Product Architecture':1.4,'Design & UX':1.3,'Engineering Delivery':1.35,'Product Strategy':1.45,'Client Services':1.25}
        location_demand=self.location_data[self.current_location]['market_demand']
        return round(base_rate*market_multipliers[self.current_department]*location_demand/50)*50

    def _calculate_utilization_target(self):
        base_targets={'Product Architecture':70,'Design & UX':75,'Engineering Delivery':85,'Product Strategy':65,'Client Services':80}
        level_adjustments={'entry':10,'mid':0,'senior':-15}
        return round(min(max(base_targets[self.current_department]+level_adjustments[self.current_level]+random.uniform(-3,3),50),90))

    def _calculate_actual_utilization(self):
        target=self._calculate_utilization_target();variation=random.uniform(-15,5)
        return round(min(max(target+variation,50),95),1)

    def _generate_expertise_data(self):
        return {'primary_specialization':self._select_specialization(),'secondary_specialization':self._select_specialization(),
                'industry_expertise':self._select_industry_expertise(),'certifications':self._assign_certifications()}

    def _select_specialization(self):
        return random.choice(self.specializations[self.current_department])

    def _select_industry_expertise(self):
        industries=['FinTech','HealthTech','E-commerce','Enterprise Software','Social Impact','Consumer Tech','EdTech','Industrial Tech']
        return random.sample(industries,k=random.randint(2,4))

    def _assign_certifications(self):
        certs=self.certification_types.get(self.current_department,[])
        cert_counts={'entry':(0,2),'mid':(1,3),'senior':(2,4)}
        min_certs,max_certs=cert_counts[self.current_level]
        return random.sample(certs,k=random.randint(min_certs,min(max_certs,len(certs))))

    def _generate_project_metrics(self):
        is_outlier=random.random()<0.05;base_projects={'entry':(1,2),'mid':(2,3),'senior':(2,4)}
        dept_modifiers={'Product Architecture':0,'Design & UX':-1,'Engineering Delivery':1,'Product Strategy':0,'Client Services':1}
        min_projects,max_projects=base_projects[self.current_level]
        modifier=dept_modifiers[self.current_department]
        active_projects=random.randint(max(1,min_projects+modifier),max(min_projects,max_projects+modifier))
        projects=[{'complexity':self._calculate_project_complexity(self.current_level,is_outlier),'status':random.choice(['Planning','In Progress','Final Stage']),
                   'duration_months':random.randint(3,12),'team_size':random.randint(3,10)} for _ in range(active_projects)]
        avg_complexity=round(sum(p['complexity'] for p in projects)/len(projects),1)
        avg_duration=round(sum(p['duration_months'] for p in projects)/len(projects),1)
        avg_team_size=round(sum(p['team_size'] for p in projects)/len(projects),1)
        return {'active_projects':active_projects,'avg_project_complexity':avg_complexity,'avg_project_duration':avg_duration,
                'avg_team_size':avg_team_size,'projects_on_time':self._calculate_projects_on_time(active_projects,is_outlier),
                'project_satisfaction':self._generate_client_satisfaction(self.current_level,self.current_department,projects,self._generate_performance_score(self.current_level,is_outlier))}

    def _calculate_projects_on_time(self, active_projects, is_outlier):
        if is_outlier:return random.choice([100,60])
        base_on_time={'entry':80,'mid':85,'senior':90};dept_modifiers={'Product Architecture':-5,'Design & UX':0,'Engineering Delivery':-3,'Product Strategy':2,'Client Services':5}
        on_time_pct=base_on_time[self.current_level]+dept_modifiers[self.current_department]+random.uniform(-5,5)
        return round(min(max(on_time_pct,60),100),1)

    def _generate_development_metrics(self):
        is_outlier=random.random()<0.05
        return {'training_hours':self._calculate_training_hours(self.current_level,is_outlier),
                'mentorship_hours':self._calculate_mentorship_hours(self.current_level),
                'knowledge_sharing_score':self._calculate_knowledge_sharing(self.current_level,is_outlier),
                'promotion_readiness':self._calculate_promotion_readiness(self.current_level,self.hire_date,is_outlier)}

    def _generate_risk_metrics(self):
        is_outlier=random.random()<0.05
        engagement_score=self._generate_engagement_score(is_outlier)
        return {'engagement_score':engagement_score,'flight_risk':self._calculate_flight_risk(is_outlier),
                'retention_risk':'High' if engagement_score<5 else 'Medium' if engagement_score<7 else 'Low'}

        def _generate_engagement_score(self, is_outlier):
        if is_outlier:return random.choice([4.0,9.5])
        base_engagement={'entry':np.random.normal(7.5,0.8),'mid':np.random.normal(7.8,0.7),'senior':np.random.normal(8.2,0.6)}
        score=base_engagement[self.current_level]+{'Product Architecture':0.2,'Design & UX':0.3,'Engineering Delivery':0.1,'Product Strategy':0.4,'Client Services':0.2}[self.current_department]
        return round(min(max(score,1),10),1)

    def _generate_performance_metrics(self, is_outlier):
        if is_outlier:return {'performance_score':round(random.uniform(4.5,5.0),2),'innovation_score':round(random.uniform(90,100),1),'delivery_quality':round(random.uniform(95,100),1)}
        return {'performance_score':round(max(min(random.normalvariate(3.7,0.4),4.5),2.5),2),'innovation_score':round(max(min(random.normalvariate(75,8),90),50),1),'delivery_quality':round(max(min(random.normalvariate(82,7),95),60),1)}

    def _generate_client_satisfaction(self, level, department, projects, performance_score):
        if not projects:return None
        dept_base_satisfaction={'Product Architecture':4.2,'Design & UX':4.0,'Engineering Delivery':4.1,'Product Strategy':4.0,'Client Services':4.3}
        total_weight=0;weighted_satisfaction=0
        for project in projects:
            project_satisfaction=dept_base_satisfaction[department]+{'entry':-0.1,'mid':0,'senior':0.1}[level]+(performance_score-3.5)*0.2+{'Planning':0.05,'In Progress':0,'Final Stage':-0.05}[project['status']]
            project_satisfaction+=random.uniform(-0.1,0.1)
            weighted_satisfaction+=project_satisfaction;total_weight+=1
        return round(weighted_satisfaction/total_weight,1)

    def _generate_internal_contribution(self, level):
        return random.randint({'entry':(1,3),'mid':(3,5),'senior':(4,7)}[level][0],{'entry':(1,3),'mid':(3,5),'senior':(4,7)}[level][1])

    def _generate_delivery_quality(self, level, is_outlier):
        if is_outlier:return random.choice([75,98])
        return round(min(max({'entry':np.random.normal(85,5),'mid':np.random.normal(90,4),'senior':np.random.normal(93,3)}[level],70),100),1)

    def _calculate_training_hours(self, level, is_outlier):
        if is_outlier:return random.choice([10,100])
        return random.randint(*{'entry':(40,60),'mid':(30,50),'senior':(20,40)}[level])

    def _calculate_mentorship_hours(self, level):
        return random.randint(*{'entry':(-30,-10),'mid':(-10,20),'senior':(20,40)}[level])

    def _calculate_knowledge_sharing(self, level, is_outlier):
        if is_outlier:return random.choice([1,10])
        return round(random.uniform(*{'entry':(2,4),'mid':(4,7),'senior':(7,9)}[level]),1)

    def _calculate_projects_on_time(self, active_projects, is_outlier):
        if is_outlier:return random.choice([100,60])
        on_time_pct={'entry':80,'mid':85,'senior':90}[self.current_level]+{'Product Architecture':-5,'Design & UX':0,'Engineering Delivery':-3,'Product Strategy':2,'Client Services':5}[self.current_department]+random.uniform(-5,5)
        return round(min(max(on_time_pct,60),100),1)

class ConsultancyDataGenerator:
    def __init__(self, num_records, seed=42):
        self.fake=Faker();Faker.seed(seed);np.random.seed(seed);random.seed(seed)
        self.num_records=num_records;self.employee_generator=ConsultancyEmployee(self.fake)
        self.departments=list(self.employee_generator.departments.keys());self.levels=['entry','mid','senior']
        self._set_company_ratios()
        self.senior_ratio,self.mid_ratio=(0.05,0.15) if num_records<100 else (0.05,0.12) if num_records<1000 else (0.03,0.10)
        self.max_senior_managers=max(1,int(num_records*self.senior_ratio));self.max_mid_managers=max(2,int(num_records*self.mid_ratio))
        self.max_direct_reports_senior,self.max_direct_reports_mid=(6,4) if num_records<100 else (8,6) if num_records<1000 else (10,8)

    def _assign_managers(self, df):
        df['is_manager'],df['management_level']=False,None
        for dept in self.departments:
            dept_df=df[df['department']==dept].copy();dept_size=len(dept_df)
            dept_senior_managers=max(1,int(dept_size*self.senior_ratio));dept_mid_managers=max(2,int(dept_size*self.mid_ratio))
            senior_candidates=dept_df[(dept_df['level']=='senior')&(dept_df['position'].str.contains('Director|Head|Chief|Lead'))].sort_values('performance_score',ascending=False)
            senior_managers=senior_candidates.head(min(dept_senior_managers,int(self.max_senior_managers/len(self.departments))))
            df.loc[senior_managers.index,'is_manager']=True;df.loc[senior_managers.index,'management_level']='senior'
            mid_candidates=dept_df[(dept_df['level']=='mid')&(dept_df['position'].str.contains('Lead|Senior|Manager'))].sort_values('performance_score',ascending=False)
            mid_managers=mid_candidates.head(min(dept_mid_managers,int(self.max_mid_managers/len(self.departments))))
            df.loc[mid_managers.index,'is_manager']=True;df.loc[mid_managers.index,'management_level']='mid'
        for dept in self.departments:
            dept_df=df[df['department']==dept];senior_managers=dept_df[df['management_level']=='senior']
            mid_managers=dept_df[df['management_level']=='mid']
            mid_level_staff=dept_df[(dept_df['level']=='mid')&(df['management_level'].isna())]
            for idx,employee in mid_level_staff.iterrows():
                valid_managers=senior_managers[df['direct_reports']<self.max_direct_reports_senior]
                if not valid_managers.empty:
                    manager=valid_managers.sample(1).iloc[0];df.loc[idx,'manager_id']=manager['employee_id']
                    df.loc[manager.name,'direct_reports']+=1
            entry_level_staff=dept_df[dept_df['level']=='entry']
            for idx,employee in entry_level_staff.iterrows():
                valid_managers=mid_managers[df['direct_reports']<self.max_direct_reports_mid]
                if valid_managers.empty:
                    valid_managers=senior_managers[df['direct_reports']<self.max_direct_reports_senior]
                if not valid_managers.empty:
                    manager=valid_managers.sample(1).iloc[0];df.loc[idx,'manager_id']=manager['employee_id']
                    df.loc[manager.name,'direct_reports']+=1
        df['span_of_control']=df['direct_reports'].apply(lambda x:'None' if x==0 else 'Small (1-3)' if x<=3 else 'Medium (4-6)' if x<=6 else 'Large (7+)')
        return df

    def _calculate_management_compensation_adjustment(self, df):
        df['management_premium']=0
        senior_premium=df['is_manager']&(df['management_level']=='senior')
        df.loc[senior_premium,'management_premium']=df.loc[senior_premium,'base_salary']*random.uniform(0.25,0.35)
        mid_premium=df['is_manager']&(df['management_level']=='mid')
        df.loc[mid_premium,'management_premium']=df.loc[mid_premium,'base_salary']*random.uniform(0.15,0.25)
        df['span_premium']=df['direct_reports'].apply(lambda x:0 if x==0 else 0.05 if x<=3 else 0.10 if x<=6 else 0.15)*df['base_salary']
        df['total_comp']=df['base_salary']+df['management_premium']+df['span_premium']
        return df
    
    def _select_department(self):
        department_weights={'Product Architecture':0.2,'Design & UX':0.2,'Engineering Delivery':0.3,'Product Strategy':0.15,'Client Services':0.15}
        return random.choices(list(department_weights.keys()),weights=list(department_weights.values()))[0]
    
    def _select_level(self, department):
        weights=self.level_weights[department]
        return random.choices(list(weights.keys()),weights=list(weights.values()))[0]
        
    def _assign_outliers(self):
        total_outliers=max(len(self.departments),int(self.num_records*0.02))
        self.outlier_assignments={dept:{level:0 for level in self.levels} for dept in self.departments}
        for dept in self.departments:
            level=random.choice(self.levels);self.outlier_assignments[dept][level]+=1;total_outliers-=1
        while total_outliers>0:
            dept=random.choice(self.departments);level=random.choice(self.levels)
            self.outlier_assignments[dept][level]+=1;total_outliers-=1

    def generate_dataset(self):
        self._assign_outliers();employees=[]
        generated_counts={dept:{level:0 for level in self.levels} for dept in self.departments}
        for _ in range(self.num_records):
            department=self._select_department();level=self._select_level(department)
            is_outlier=self.outlier_assignments[department][level]>0 and generated_counts[department][level]<self._get_expected_count(department,level)
            if is_outlier:self.outlier_assignments[department][level]-=1
            employee=self.employee_generator.generate_employee(department,level,is_outlier)
            employees.append(employee);generated_counts[department][level]+=1
        df=pd.DataFrame(employees);self._assign_managers(df);self._calculate_total_compensation(df);self._update_team_metrics(df)
        return df
    
    def _get_expected_count(self, department, level):
        return int(self.num_records*self.department_weights[department]*self.level_weights[department][level])

    def _calculate_total_compensation(self, df):
        df['total_comp']=df.apply(lambda row:row['base_salary']*(1+(0.2 if row['level']=='senior' else 0.15 if row['level']=='mid' else 0.1)+(0.05 if row['performance_score']>=4.5 else 0)),axis=1)

    def _update_team_metrics(self, df):
        manager_counts=df['manager_id'].value_counts()
        for emp_id in manager_counts.index:
            if pd.notna(emp_id):df.loc[df['employee_id']==emp_id,'direct_reports']=manager_counts[emp_id]
        df.loc[df['level']=='senior','team_lead_projects']=df.loc[df['level']=='senior','active_projects'].apply(lambda x:random.randint(1,x))
        df.loc[df['level']=='mid','team_lead_projects']=df.loc[df['level']=='mid','active_projects'].apply(lambda x:random.randint(0,max(x-1,0)))

    def _prepare_data_for_export(self):
        df=self.generate_dataset();df['hire_date']=df['hire_date'].dt.strftime('%Y-%m-%d')
        for column in df.select_dtypes(include=[np.number]).columns:df[column]=df[column].astype(float)
        for col in ['industry_expertise','certifications']:df[col]=df[col].apply(lambda x:', '.join(x) if isinstance(x,list) else x)
        return df

In [3]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker

class ConsultancyEmployee:
    def __init__(self, faker, reference_date=datetime(2024, 1, 1)):
        self.fake = faker
        self.reference_date = reference_date
        # Initialize other attributes that will be set during employee generation
        self.current_department = None
        self.current_level = None
        self.hire_date = None
        self.departments = {
            'Product Architecture': {
                'entry': ['Junior Solution Architect', 'System Analyst I', 'Technical Documentation Specialist'],
                'mid': ['Solution Architect', 'Enterprise Architect', 'Integration Specialist'],
                'senior': ['Chief Architect', 'Head of Architecture', 'Technical Director']
            },
            'Design & UX': {
                'entry': ['UI Designer I', 'UX Researcher I', 'Design System Specialist'],
                'mid': ['Senior UX Designer', 'Design Lead', 'Design Strategist'],
                'senior': ['Design Director', 'Head of Design', 'Chief Experience Officer']
            },
            'Engineering Delivery': {
                'entry': ['Associate Developer', 'Junior DevOps', 'QA Engineer I'],
                'mid': ['Tech Lead', 'Senior Developer', 'Cloud Architect'],
                'senior': ['Delivery Director', 'Engineering Manager', 'CTO']
            },
            'Product Strategy': {
                'entry': ['Product Analyst', 'Business Analyst I', 'Market Researcher'],
                'mid': ['Product Strategist', 'Senior Product Manager', 'Domain Expert'],
                'senior': ['Head of Product', 'Strategy Director', 'Innovation Lead']
            },
            'Client Services': {
                'entry': ['Project Coordinator', 'Client Success Associate', 'Engagement Analyst'],
                'mid': ['Engagement Manager', 'Client Partner', 'Program Manager'],
                'senior': ['Account Director', 'Head of Client Services', 'Partnership Director']
            }
        }
        
        self.location_data = {
            'London': {
                'country': 'UK',
                'region': 'EMEA',
                'cost_multiplier': 1.2,
                'timezone': 'GMT',
                'market_demand': 0.9
            },
            'New York': {
                'country': 'USA',
                'region': 'Americas',
                'cost_multiplier': 1.3,
                'timezone': 'EST',
                'market_demand': 1.0
            },
            'San Francisco': {
                'country': 'USA',
                'region': 'Americas',
                'cost_multiplier': 1.35,
                'timezone': 'PST',
                'market_demand': 1.1
            },
            'Singapore': {
                'country': 'Singapore',
                'region': 'APAC',
                'cost_multiplier': 1.15,
                'timezone': 'SGT',
                'market_demand': 0.95
            },
            'Berlin': {
                'country': 'Germany',
                'region': 'EMEA',
                'cost_multiplier': 1.1,
                'timezone': 'CET',
                'market_demand': 0.85
            }
        }
        
        self.specializations = {
            'Product Architecture': ['Cloud Infrastructure', 'Microservices', 'System Integration', 'API Design', 
                                   'Data Architecture', 'Security Architecture'],
            'Design & UX': ['Mobile Design', 'Web Applications', 'Design Systems', 'User Research', 
                           'Service Design', 'Design Thinking'],
            'Engineering Delivery': ['Full Stack', 'Frontend', 'Backend', 'DevOps', 'Cloud Native', 
                                   'Mobile Development'],
            'Product Strategy': ['Digital Transformation', 'Innovation Strategy', 'Go-to-Market', 
                               'Product Discovery', 'Market Analysis'],
            'Client Services': ['Agile Delivery', 'Digital Strategy', 'Change Management', 
                              'Portfolio Management', 'Strategic Planning']
        }
        
        self.industry_expertise = ['FinTech', 'HealthTech', 'E-commerce', 'Enterprise Software', 
                                 'Social Impact', 'Consumer Tech', 'EdTech', 'Industrial Tech']
        
        self.office_locations = {
            'HQ': {'city': 'London', 'country': 'UK', 'region': 'EMEA'},
            'NYC': {'city': 'New York', 'country': 'USA', 'region': 'Americas'},
            'SF': {'city': 'San Francisco', 'country': 'USA', 'region': 'Americas'},
            'SG': {'city': 'Singapore', 'country': 'Singapore', 'region': 'APAC'},
            'BER': {'city': 'Berlin', 'country': 'Germany', 'region': 'EMEA'}
        }

        self.certification_types = {
            'Product Architecture': ['AWS Solutions Architect', 'Azure Architect', 'TOGAF', 'Google Cloud Architect'],
            'Design & UX': ['Google UX Design', 'Interaction Design Foundation', 'Human Factors International'],
            'Engineering Delivery': ['AWS Developer', 'Kubernetes', 'Azure Developer', 'Scrum Master'],
            'Product Strategy': ['Product Management (PMP)', 'Agile Product Management', 'PSPO'],
            'Client Services': ['Prince2', 'PMP', 'Agile Project Management', 'Scrum Master']
        }

    def generate_employee(self):
        """Generate a complete employee record with realistic, interconnected data."""
        # Initialize key attributes that influence other calculations
        self._initialize_core_attributes()
        
        # Generate employee data
        employee_data = {
            'employee_id': self.fake.unique.random_number(digits=6),
            'full_name': self.fake.name(),
            'department': self.current_department,
            'position': self._get_position(),
            'level': self.current_level,
            'hire_date': self.hire_date,
            
            # Location and work arrangement data
            **self._generate_location_data(),
            
            # Financial metrics
            **self._generate_financial_metrics(),
            
            # Skills and expertise
            **self._generate_expertise_data(),
            
            # Performance metrics
            **self._generate_performance_metrics(),
            
            # Project and delivery metrics
            **self._generate_project_metrics(),
            
            # Development and growth metrics
            **self._generate_development_metrics(),
            
            # Risk and engagement metrics
            **self._generate_risk_metrics(),
            
            # Organizational metrics (will be populated later)
            'manager_id': None,
            'total_comp': None,
            'team_lead_projects': 0,
            'direct_reports': 0
        }
        
        return employee_data

    def _initialize_core_attributes(self):
        """Initialize core employee attributes that influence other calculations."""
        is_outlier = random.random() < 0.05
        self.current_department = self._select_department()
        self.current_level = self._select_level(is_outlier)
        self.hire_date = self._generate_hire_date(self.current_level, is_outlier)
        self.current_location = self._select_location(self.current_department)

    def _select_department(self):
        """Select department based on company needs and market demand."""
        department_weights = {
            'Product Architecture': 0.2,
            'Design & UX': 0.2,
            'Engineering Delivery': 0.3,
            'Product Strategy': 0.15,
            'Client Services': 0.15
        }
        return random.choices(list(department_weights.keys()), 
                            weights=list(department_weights.values()))[0]

    def _select_level(self, is_outlier):
        """Select employee level considering department distribution and outlier status."""
        base_weights = {
            'Product Architecture': [0.3, 0.5, 0.2],
            'Design & UX': [0.4, 0.4, 0.2],
            'Engineering Delivery': [0.5, 0.3, 0.2],
            'Product Strategy': [0.3, 0.4, 0.3],
            'Client Services': [0.2, 0.5, 0.3]
        }
        
        weights = [0.2, 0.3, 0.5] if is_outlier else base_weights[self.current_department]
        return random.choices(['entry', 'mid', 'senior'], weights=weights)[0]

    def _generate_hire_date(self, level, is_outlier):
        """Generate realistic hire date based on level and experience requirements."""
        experience_ranges = {
            'entry': (0, 2),
            'mid': (2, 6),
            'senior': (5, 12)
        }
        
        if is_outlier:
            experience_ranges = {
                'entry': (2, 4),
                'mid': (5, 8),
                'senior': (10, 15)
            }
        
        min_years, max_years = experience_ranges[level]
        days_ago = random.randint(min_years * 365, max_years * 365)
        return self.reference_date - timedelta(days=days_ago)

    def _select_location(self, department):
        """Select office location based on department needs and regional strategy."""
        location_weights = {
            'Product Architecture': {
                'London': 0.3, 'New York': 0.2, 'San Francisco': 0.2, 
                'Singapore': 0.15, 'Berlin': 0.15
            },
            'Design & UX': {
                'London': 0.25, 'New York': 0.3, 'San Francisco': 0.2, 
                'Singapore': 0.15, 'Berlin': 0.1
            },
            'Engineering Delivery': {
                'London': 0.2, 'New York': 0.25, 'San Francisco': 0.25, 
                'Singapore': 0.15, 'Berlin': 0.15
            },
            'Product Strategy': {
                'London': 0.3, 'New York': 0.25, 'San Francisco': 0.2, 
                'Singapore': 0.15, 'Berlin': 0.1
            },
            'Client Services': {
                'London': 0.3, 'New York': 0.2, 'San Francisco': 0.2, 
                'Singapore': 0.2, 'Berlin': 0.1
            }
        }
        
        weights = location_weights[department]
        return random.choices(list(weights.keys()), 
                            weights=list(weights.values()))[0]

    def _get_position(self):
        """Get position title based on department and level."""
        return random.choice(self.departments[self.current_department][self.current_level])

    def _generate_location_data(self):
        """Generate location-related data including remote work arrangements."""
        location_info = self.location_data[self.current_location]
        
        return {
            'city': self.current_location,
            'country': location_info['country'],
            'region': location_info['region'],
            'remote_work_ratio': self._calculate_remote_ratio(),
            'travel_percentage': self._calculate_travel_percentage()
        }

    def _generate_financial_metrics(self):
        """Generate financial metrics including salary and billing rates."""
        base_salary = self._calculate_base_salary()
        
        return {
            'base_salary': base_salary,
            'billing_rate': self._calculate_billing_rate(base_salary),
            'utilization_target': self._calculate_utilization_target(),
            'actual_utilization': self._calculate_actual_utilization()
        }

    def _calculate_base_salary(self):
        """Calculate base salary considering multiple factors."""
        # Base salary ranges by level
        base_ranges = {
            'entry': (65000, 85000),
            'mid': (95000, 145000),
            'senior': (150000, 250000)
        }
        
        # Department multipliers
        dept_multipliers = {
            'Product Architecture': 1.25,
            'Design & UX': 1.15,
            'Engineering Delivery': 1.20,
            'Product Strategy': 1.18,
            'Client Services': 1.10
        }
        
        # Calculate base salary
        min_salary, max_salary = base_ranges[self.current_level]
        base = random.uniform(min_salary, max_salary)
        
        # Apply multipliers
        location_multiplier = self.location_data[self.current_location]['cost_multiplier']
        dept_multiplier = dept_multipliers[self.current_department]
        experience_multiplier = self._calculate_experience_multiplier()
        
        final_salary = base * location_multiplier * dept_multiplier * experience_multiplier
        
        # Round to nearest thousand
        return round(final_salary, -3)

    def _calculate_experience_multiplier(self):
        """Calculate experience multiplier based on years of experience."""
        years_experience = (self.reference_date - self.hire_date).days / 365
        return 1 + (min(years_experience, 15) * 0.025)

    def _calculate_billing_rate(self, base_salary):
        """Calculate billing rate based on cost and market factors."""
        # Basic cost calculation
        annual_cost = base_salary * 2.5  # Including overhead
        utilization_target = self._calculate_utilization_target() / 100
        annual_billable_hours = 2080 * utilization_target
        
        base_rate = annual_cost / annual_billable_hours
        
        # Apply market adjustments
        market_multipliers = {
            'Product Architecture': 1.4,
            'Design & UX': 1.3,
            'Engineering Delivery': 1.35,
            'Product Strategy': 1.45,
            'Client Services': 1.25
        }
        
        location_demand = self.location_data[self.current_location]['market_demand']
        
        final_rate = base_rate * market_multipliers[self.current_department] * location_demand
        
        # Round to nearest 50
        return round(final_rate / 50) * 50

    def _calculate_utilization_target(self):
        """Calculate utilization target considering role and responsibilities."""
        base_targets = {
            'Product Architecture': 70,
            'Design & UX': 75,
            'Engineering Delivery': 85,
            'Product Strategy': 65,
            'Client Services': 80
        }
        
        level_adjustments = {
            'entry': 10,
            'mid': 0,
            'senior': -15
        }
        
        target = base_targets[self.current_department] + level_adjustments[self.current_level]
        
        # Add realistic variation
        target += random.uniform(-3, 3)
        
        return round(min(max(target, 50), 90))

    def _calculate_actual_utilization(self):
        """Calculate actual utilization with realistic variation from target."""
        target = self._calculate_utilization_target()
        variation = random.uniform(-15, 5)  # Tendency to under-utilize rather than over-utilize
        
        return round(min(max(target + variation, 50), 95), 1)

    def _generate_expertise_data(self):
        """Generate expertise-related data including specializations and certifications."""
        return {
            'primary_specialization': self._select_specialization(),
            'secondary_specialization': self._select_specialization(),
            'industry_expertise': self._select_industry_expertise(),
            'certifications': self._assign_certifications()
        }

    def _select_specialization(self):
        """Select specialization based on department."""
        specializations = {
            'Product Architecture': [
                'Cloud Infrastructure', 'Microservices', 'System Integration',
                'API Design', 'Data Architecture', 'Security Architecture'
            ],
            'Design & UX': [
                'Mobile Design', 'Web Applications', 'Design Systems',
                'User Research', 'Service Design', 'Design Thinking'
            ],
            'Engineering Delivery': [
                'Full Stack', 'Frontend', 'Backend', 'DevOps',
                'Cloud Native', 'Mobile Development'
            ],
            'Product Strategy': [
                'Digital Transformation', 'Innovation Strategy', 'Go-to-Market',
                'Product Discovery', 'Market Analysis'
            ],
            'Client Services': [
                'Agile Delivery', 'Digital Strategy', 'Change Management',
                'Portfolio Management', 'Strategic Planning'
            ]
        }
        
        return random.choice(specializations[self.current_department])

    def _select_industry_expertise(self):
        """Select industry expertise areas."""
        industries = [
            'FinTech', 'HealthTech', 'E-commerce', 'Enterprise Software',
            'Social Impact', 'Consumer Tech', 'EdTech', 'Industrial Tech'
        ]
        
        num_industries = random.randint(2, 4)
        return random.sample(industries, k=num_industries)

    def _assign_certifications(self):
        """Assign relevant certifications based on role and level."""
        available_certs = self.certification_types.get(self.current_department, [])
        
        # Number of certifications based on level
        cert_counts = {
            'entry': (0, 2),
            'mid': (1, 3),
            'senior': (2, 4)
        }
        
        min_certs, max_certs = cert_counts[self.current_level]
        num_certs = random.randint(min_certs, min(max_certs, len(available_certs)))
        
        return random.sample(available_certs, k=num_certs)

    def _calculate_project_complexity(self, level, is_outlier):
        complexity_weights = {
            'entry': [0.5, 0.3, 0.15, 0.05, 0],     # Mostly simple projects
            'mid': [0.1, 0.3, 0.4, 0.15, 0.05],     # Mix of complexities
            'senior': [0, 0.1, 0.3, 0.4, 0.2]       # More complex projects
        }
        
        if is_outlier:
            complexity_weights = {
                'entry': [0.2, 0.3, 0.3, 0.15, 0.05],
                'mid': [0, 0.15, 0.25, 0.4, 0.2],
                'senior': [0, 0, 0.2, 0.4, 0.4]
            }
        
        return np.random.choice(range(1, 6), p=complexity_weights[level])
    def _calculate_promotion_readiness(self, level, hire_date, is_outlier):
        if level == 'senior':
            return round(random.uniform(0, 30), 1)
        
        years_in_role = (self.reference_date - hire_date).days / 365
        performance_score = self._generate_performance_score(level, is_outlier)
        project_complexity = self._calculate_project_complexity(level, is_outlier)
        
        base_score = 40
        
        # Experience factor
        experience_score = min(years_in_role * 10, 30)
        
        # Performance factor
        performance_factor = (performance_score - 3) * 20
        
        # Complexity factor
        complexity_factor = project_complexity * 5
        
        # Certifications factor - Fixed to use self._assign_certifications() correctly
        cert_count = len(self._assign_certifications())
        cert_factor = cert_count * 5
        
        total_score = base_score + experience_score + performance_factor + complexity_factor + cert_factor
        
        if is_outlier:
            total_score *= random.uniform(1.2, 1.3)
        
        return round(min(max(total_score, 0), 100), 1)


    def _calculate_flight_risk(self, is_outlier):
        base_risk = 30
        
        # Factors that increase risk
        if self._calculate_promotion_readiness(self.current_level, self.hire_date, is_outlier) > 80:
            base_risk += 20
        
        if self._generate_engagement_score(is_outlier) < 7:
            base_risk += 15
        
        if self._calculate_actual_utilization() < 90:  # Removed is_outlier parameter
            base_risk += 10
        
        years_in_role = (self.reference_date - self.hire_date).days / 365
        if years_in_role > 3:
            base_risk += 15
        
        if self._generate_performance_score(self.current_level, is_outlier) < 3.5:
            base_risk += 25
        
        # Market demand factor
        if self.current_department in ['Engineering Delivery', 'Product Architecture']:
            base_risk += 10
        
        if is_outlier:
            return random.choice([85, 15])
        
        return round(min(max(base_risk + random.uniform(-10, 10), 0), 100))

    def _calculate_innovation_score(self, level, department):
        base_innovation = {
            'Product Architecture': 7.5,    # High technical innovation
            'Design & UX': 8.0,            # Creative solutions
            'Engineering Delivery': 7.0,    # Technical implementation
            'Product Strategy': 8.5,        # Business innovation
            'Client Services': 6.5          # Process innovation
        }
        
        level_factors = {
            'entry': 0.7,    # Learning phase
            'mid': 1.0,      # Established
            'senior': 1.3    # Leading innovation
        }
        
        performance_score = self._generate_performance_score(level, False)
        performance_factor = (performance_score - 3) * 0.2
        
        project_complexity = self._calculate_project_complexity(level, False)
        complexity_factor = (project_complexity - 3) * 0.15
        
        # Fixed certification count calculation
        cert_count = len(self._assign_certifications())
        certification_factor = cert_count * 0.1
        
        innovation = (base_innovation[department] * level_factors[level] * 
                    (1 + performance_factor + complexity_factor + certification_factor))
        
        return round(min(max(innovation + random.uniform(-0.5, 0.5), 1), 10), 1)

    def _assign_office(self, department):
        weights = {
            'Product Architecture': [0.3, 0.2, 0.2, 0.15, 0.15],
            'Design & UX': [0.25, 0.3, 0.2, 0.15, 0.1],
            'Engineering Delivery': [0.2, 0.25, 0.25, 0.15, 0.15],
            'Product Strategy': [0.3, 0.25, 0.2, 0.15, 0.1],
            'Client Services': [0.3, 0.2, 0.2, 0.2, 0.1]
        }
        
        office_key = np.random.choice(
            list(self.office_locations.keys()),
            p=weights[department]
        )
        return self.office_locations[office_key]

    def _calculate_remote_ratio(self):
        """Calculate remote work ratio based on department and level."""
        base_ratio = {
            'Product Architecture': 0.7,
            'Design & UX': 0.6,
            'Engineering Delivery': 0.8,
            'Product Strategy': 0.5,
            'Client Services': 0.4
        }
        
        level_adjustments = {
            'entry': -0.1,
            'mid': 0,
            'senior': 0.1
        }
        
        ratio = base_ratio[self.current_department] + level_adjustments[self.current_level]
        ratio = min(max(ratio + random.uniform(-0.1, 0.1), 0.2), 0.9)
        return round(ratio, 2)

    def _calculate_travel_percentage(self):
        """Calculate travel percentage based on department and level."""
        base_travel = {
            'Product Architecture': 30,
            'Design & UX': 20,
            'Engineering Delivery': 15,
            'Product Strategy': 40,
            'Client Services': 50
        }
        
        level_multipliers = {
            'entry': 0.5,
            'mid': 1.0,
            'senior': 1.2
        }
        
        travel = base_travel[self.current_department] * level_multipliers[self.current_level]
        return round(min(travel + random.uniform(-5, 5), 80))

    def _assign_industry_expertise(self):
        # Ensure we don't try to sample more than available
        num_industries = random.randint(2, min(4, len(self.industry_expertise)))
        return random.sample(self.industry_expertise, k=num_industries)

    def _generate_performance_score(self, level, is_outlier):
        base_scores = {
            'entry': np.random.normal(3.5, 0.4),
            'mid': np.random.normal(3.8, 0.3),
            'senior': np.random.normal(4.1, 0.2)
        }
        
        score = base_scores[level]
        if is_outlier:
            score = random.choice([2.0, 4.8])
        
        return round(min(max(score, 1), 5), 1)
    
    def _generate_project_metrics(self):
        """Generate project-related metrics and statistics."""
        is_outlier = random.random() < 0.05
        
        # Calculate number of active projects based on level and department
        base_projects = {
            'entry': (1, 2),
            'mid': (2, 3),
            'senior': (2, 4)
        }
        
        # Department modifiers for project count
        dept_modifiers = {
            'Product Architecture': 0,
            'Design & UX': -1,
            'Engineering Delivery': 1,
            'Product Strategy': 0,
            'Client Services': 1
        }
        
        min_projects, max_projects = base_projects[self.current_level]
        modifier = dept_modifiers[self.current_department]
        
        # Adjust project range based on department modifier
        min_projects = max(1, min_projects + modifier)
        max_projects = max(min_projects, max_projects + modifier)
        
        active_projects = random.randint(min_projects, max_projects)
        
        # Generate individual project data
        projects = []
        for _ in range(active_projects):
            project = {
                'complexity': self._calculate_project_complexity(self.current_level, is_outlier),
                'status': random.choice(['Planning', 'In Progress', 'Final Stage']),
                'duration_months': random.randint(3, 12),
                'team_size': random.randint(3, 10)
            }
            projects.append(project)
        
        # Calculate average project metrics
        avg_complexity = round(sum(p['complexity'] for p in projects) / len(projects), 1)
        avg_duration = round(sum(p['duration_months'] for p in projects) / len(projects), 1)
        avg_team_size = round(sum(p['team_size'] for p in projects) / len(projects), 1)
        
        return {
            'active_projects': active_projects,
            'avg_project_complexity': avg_complexity,
            'avg_project_duration': avg_duration,
            'avg_team_size': avg_team_size,
            'projects_on_time': self._calculate_projects_on_time(active_projects, is_outlier),
            'project_satisfaction': self._generate_client_satisfaction(
                self.current_level, 
                self.current_department, 
                projects,
                self._generate_performance_score(self.current_level, is_outlier)
            )
        }

    def _calculate_projects_on_time(self, active_projects, is_outlier):
        """Calculate percentage of projects delivered on time."""
        if is_outlier:
            return random.choice([100, 60])
            
        base_on_time = {
            'entry': 80,
            'mid': 85,
            'senior': 90
        }
        
        dept_modifiers = {
            'Product Architecture': -5,
            'Design & UX': 0,
            'Engineering Delivery': -3,
            'Product Strategy': 2,
            'Client Services': 5
        }
        
        on_time_pct = base_on_time[self.current_level] + dept_modifiers[self.current_department]
        # Add some random variation
        on_time_pct += random.uniform(-5, 5)
        
        return round(min(max(on_time_pct, 60), 100), 1)

    def _generate_development_metrics(self):
        """Generate development and growth-related metrics."""
        is_outlier = random.random() < 0.05
        
        return {
            'training_hours': self._calculate_training_hours(self.current_level, is_outlier),
            'mentorship_hours': self._calculate_mentorship_hours(self.current_level),
            'knowledge_sharing_score': self._calculate_knowledge_sharing(self.current_level, is_outlier),
            'promotion_readiness': self._calculate_promotion_readiness(
                self.current_level, 
                self.hire_date, 
                is_outlier
            )
        }

    def _generate_risk_metrics(self):
        """Generate risk and retention-related metrics."""
        is_outlier = random.random() < 0.05
        
        engagement_score = self._generate_engagement_score(is_outlier)
        flight_risk = self._calculate_flight_risk(is_outlier)
        
        return {
            'engagement_score': engagement_score,
            'flight_risk': flight_risk,
            'retention_risk': 'High' if flight_risk > 70 else 'Medium' if flight_risk > 40 else 'Low'
        }

    def _generate_engagement_score(self, is_outlier):
        """Generate employee engagement score."""
        if is_outlier:
            return random.choice([4.0, 9.5])
        
        base_engagement = {
            'entry': np.random.normal(7.5, 0.8),
            'mid': np.random.normal(7.8, 0.7),
            'senior': np.random.normal(8.2, 0.6)
        }
        
        score = base_engagement[self.current_level]
        
        # Adjust based on department
        dept_adjustments = {
            'Product Architecture': 0.2,
            'Design & UX': 0.3,
            'Engineering Delivery': 0.1,
            'Product Strategy': 0.4,
            'Client Services': 0.2
        }
        
        score += dept_adjustments[self.current_department]
        
        return round(min(max(score, 1), 10), 1)

    def _generate_performance_metrics(self):
        """Generate performance-related metrics."""
        is_outlier = random.random() < 0.05
        performance_score = self._generate_performance_score(self.current_level, is_outlier)
        
        active_projects = random.randint(1, 5)
        projects = [self._generate_project_data() for _ in range(active_projects)]
        
        return {
            'performance_score': performance_score,
            'delivery_quality': self._generate_delivery_quality(self.current_level, is_outlier),
            'client_satisfaction': self._generate_client_satisfaction(self.current_level, 
                                                                self.current_department, 
                                                                projects, 
                                                                performance_score),
            'internal_contribution': self._generate_internal_contribution(self.current_level),
            'innovation_score': self._calculate_innovation_score(self.current_level, 
                                                            self.current_department)
        }

    def _generate_project_data(self):
        """Generate random project data."""
        statuses = ['Planning', 'In Progress', 'Final Stage']
        return {
            'complexity': random.randint(1, 5),
            'status': random.choice(statuses)
        }

    def _generate_client_satisfaction(self, level, department, projects, performance_score):
        """Generate client satisfaction score based on multiple factors"""
        if not projects:
            return None
        
        # Base satisfaction weighted by department expertise
        dept_base_satisfaction = {
            'Product Architecture': 4.2,
            'Design & UX': 4.0,
            'Engineering Delivery': 4.1,
            'Product Strategy': 4.0,
            'Client Services': 4.3
        }
        
        # Calculate weighted satisfaction based on multiple factors
        total_weight = 0
        weighted_satisfaction = 0
        
        for project in projects:
            project_weight = 1.0
            base = dept_base_satisfaction[department]
            
            # Project complexity factor
            complexity = project['complexity']
            complexity_factor = {
                1: 0.1,    # Simple projects easier to satisfy
                2: 0.05,
                3: 0,
                4: -0.05,
                5: -0.1    # Complex projects harder to satisfy
            }
            
            # Project status factor
            status_factor = {
                'Planning': 0.05,      # Early stage optimism
                'In Progress': 0,      # Neutral
                'Final Stage': -0.05   # Delivery pressure
            }
            
            # Calculate project-specific satisfaction
            project_satisfaction = base
            project_satisfaction += complexity_factor[complexity]
            project_satisfaction += status_factor[project['status']]
            
            # Performance impact
            performance_impact = (performance_score - 3.5) * 0.2
            project_satisfaction += performance_impact
            
            # Level-based adjustments
            level_factor = {
                'entry': -0.1,    # Less experience
                'mid': 0,         # Baseline
                'senior': 0.1     # More experience
            }
            project_satisfaction += level_factor[level]
            
            # Add slight randomization for realism
            project_satisfaction += random.uniform(-0.1, 0.1)
            
            # Accumulate weighted satisfaction
            weighted_satisfaction += project_satisfaction * project_weight
            total_weight += project_weight
        
        final_satisfaction = round(weighted_satisfaction / total_weight, 1)
        return min(max(final_satisfaction, 1.0), 5.0)

    def _generate_internal_contribution(self, level):
        base_contribution = {
            'entry': (1, 3),
            'mid': (3, 5),
            'senior': (4, 7)
        }
        min_contrib, max_contrib = base_contribution[level]
        return random.randint(min_contrib, max_contrib)

    def _generate_delivery_quality(self, level, is_outlier):
        base_quality = {
            'entry': np.random.normal(85, 5),
            'mid': np.random.normal(90, 4),
            'senior': np.random.normal(93, 3)
        }
        
        quality = base_quality[level]
        if is_outlier:
            quality = random.choice([75, 98])
            
        return round(min(max(quality, 70), 100), 1)

    def _calculate_training_hours(self, level, is_outlier):
        base_hours = {
            'entry': (40, 60),
            'mid': (30, 50),
            'senior': (20, 40)
        }
        
        if is_outlier:
            return random.choice([10, 100])
            
        min_hours, max_hours = base_hours[level]
        return random.randint(min_hours, max_hours)

    def _calculate_mentorship_hours(self, level):
        # Negative hours indicate receiving mentorship, positive hours indicate giving mentorship
        mentorship_ranges = {
            'entry': (-30, -10),
            'mid': (-10, 20),
            'senior': (20, 40)
        }
        min_hours, max_hours = mentorship_ranges[level]
        return random.randint(min_hours, max_hours)


    def _calculate_knowledge_sharing(self, level, is_outlier):
        base_sharing = {
            'entry': (2, 4),
            'mid': (4, 7),
            'senior': (7, 9)
        }
        
        if is_outlier:
            return random.choice([1, 10])
            
        min_share, max_share = base_sharing[level]
        return round(random.uniform(min_share, max_share), 1)
    

class ConsultancyDataGenerator:
    def __init__(self, num_records, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        
        self.employee_generator = ConsultancyEmployee(self.fake)
        self.num_records = num_records

    def generate_dataset(self):
        """Generate the complete dataset"""
        employees = []
        for _ in range(self.num_records):
            employee = self.employee_generator.generate_employee()
            employees.append(employee)
            
        # Convert to DataFrame
        df = pd.DataFrame(employees)
        
        # Post-process the data
        self._assign_managers(df)
        self._calculate_total_compensation(df)
        self._update_team_metrics(df)
        
        return df
    
    def _assign_managers(self, df):
        """Assign managers to employees based on level and department"""
        for dept in df['department'].unique():
            dept_df = df[df['department'] == dept]
            
            # Senior employees don't have managers
            seniors = dept_df[dept_df['level'] == 'senior']
            potential_managers = seniors['employee_id'].tolist()
            
            if potential_managers:
                # Assign managers to mid and entry level employees
                for level in ['mid', 'entry']:
                    level_employees = df[(df['department'] == dept) & (df['level'] == level)]
                    for idx in level_employees.index:
                        if potential_managers:
                            df.at[idx, 'manager_id'] = random.choice(potential_managers)

    def _calculate_total_compensation(self, df):
        """Calculate total compensation including bonus and benefits"""
        df['total_comp'] = df.apply(lambda row: 
            row['base_salary'] * (1 + 
                                (0.2 if row['level'] == 'senior' else
                                 0.15 if row['level'] == 'mid' else 0.1) +
                                (0.05 if row['performance_score'] >= 4.5 else 0)), 
            axis=1)

    def _update_team_metrics(self, df):
        """Update team lead projects and direct reports counts"""
        # Count direct reports
        manager_counts = df['manager_id'].value_counts()
        for emp_id in manager_counts.index:
            if pd.notna(emp_id):
                mask = df['employee_id'] == emp_id
                df.loc[mask, 'direct_reports'] = manager_counts[emp_id]
                
        # Assign team lead projects for senior and mid-level employees
        df.loc[df['level'] == 'senior', 'team_lead_projects'] = \
            df.loc[df['level'] == 'senior', 'active_projects'].apply(lambda x: random.randint(1, x))
        df.loc[df['level'] == 'mid', 'team_lead_projects'] = \
            df.loc[df['level'] == 'mid', 'active_projects'].apply(lambda x: random.randint(0, max(x-1, 0)))

    def _prepare_data_for_export(self):
        """Prepare and clean data for export"""
        df = self.generate_dataset()
        
        df['hire_date'] = df['hire_date'].dt.strftime('%Y-%m-%d')
        
        for column in df.select_dtypes(include=[np.number]).columns:
            df[column] = df[column].astype(float)
        
        list_columns = ['industry_expertise', 'certifications']
        for col in list_columns:
            df[col] = df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        
        return df

    def _save_to_excel(self, df, filepath):
        """Save data to Excel with formatting"""
        with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Employee Data', index=False)
            
            summary_stats = pd.DataFrame({
                'Department Distribution': df['department'].value_counts(),
                'Average Salary by Level': df.groupby('level')['base_salary'].mean(),
                'Average Utilization': df.groupby('department')['actual_utilization'].mean()
            })
            summary_stats.to_excel(writer, sheet_name='Summary Stats')

    def _save_to_json(self, df, filepath):
        """Save data to JSON with proper formatting"""
        records = df.to_dict(orient='records')
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(records, f, indent=2, ensure_ascii=False)

    def generate_and_save_data(self, base_filename='consultancy_data'):
        """Generate and save data in both Excel and JSON formats"""
        df = self._prepare_data_for_export()
        
        os.makedirs('output', exist_ok=True)
        
        excel_path = os.path.join('output', f'{base_filename}.xlsx')
        json_path = os.path.join('output', f'{base_filename}.json')
        
        self._save_to_excel(df, excel_path)
        print(f"Excel file created successfully: {excel_path}")
        
        self._save_to_json(df, json_path)
        print(f"JSON file created successfully: {json_path}")
        
        print(f"Number of records: {len(df)}")

def main():
    generator = ConsultancyDataGenerator(num_records=50)
    
    try:
        generator.generate_and_save_data()
        print("Data generation completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

Excel file created successfully: output\consultancy_data.xlsx
JSON file created successfully: output\consultancy_data.json
Number of records: 50
Data generation completed successfully!


In [ ]:
import json
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from faker import Faker

class ConsultancyEmployee:
    def __init__(self, faker, reference_date=datetime(2024, 1, 1)):
        self.fake = faker
        self.reference_date = reference_date
        self.departments = {
            'Product Architecture': {
                'entry': ['Junior Solution Architect', 'System Analyst I', 'Technical Documentation Specialist'],
                'mid': ['Solution Architect', 'Enterprise Architect', 'Integration Specialist'],
                'senior': ['Chief Architect', 'Head of Architecture', 'Technical Director']
            },
            'Design & UX': {
                'entry': ['UI Designer I', 'UX Researcher I', 'Design System Specialist'],
                'mid': ['Senior UX Designer', 'Design Lead', 'Design Strategist'],
                'senior': ['Design Director', 'Head of Design', 'Chief Experience Officer']
            },
            'Engineering Delivery': {
                'entry': ['Associate Developer', 'Junior DevOps', 'QA Engineer I'],
                'mid': ['Tech Lead', 'Senior Developer', 'Cloud Architect'],
                'senior': ['Delivery Director', 'Engineering Manager', 'CTO']
            },
            'Product Strategy': {
                'entry': ['Product Analyst', 'Business Analyst I', 'Market Researcher'],
                'mid': ['Product Strategist', 'Senior Product Manager', 'Domain Expert'],
                'senior': ['Head of Product', 'Strategy Director', 'Innovation Lead']
            },
            'Client Services': {
                'entry': ['Project Coordinator', 'Client Success Associate', 'Engagement Analyst'],
                'mid': ['Engagement Manager', 'Client Partner', 'Program Manager'],
                'senior': ['Account Director', 'Head of Client Services', 'Partnership Director']
            }
        }
        
        self.specializations = {
            'Product Architecture': ['Cloud Infrastructure', 'Microservices', 'System Integration', 'API Design', 
                                   'Data Architecture', 'Security Architecture'],
            'Design & UX': ['Mobile Design', 'Web Applications', 'Design Systems', 'User Research', 
                           'Service Design', 'Design Thinking'],
            'Engineering Delivery': ['Full Stack', 'Frontend', 'Backend', 'DevOps', 'Cloud Native', 
                                   'Mobile Development'],
            'Product Strategy': ['Digital Transformation', 'Innovation Strategy', 'Go-to-Market', 
                               'Product Discovery', 'Market Analysis'],
            'Client Services': ['Agile Delivery', 'Digital Strategy', 'Change Management', 
                              'Portfolio Management', 'Strategic Planning']
        }
        
        self.industry_expertise = ['FinTech', 'HealthTech', 'E-commerce', 'Enterprise Software', 
                                 'Social Impact', 'Consumer Tech', 'EdTech', 'Industrial Tech']
        
        self.office_locations = {
            'HQ': {'city': 'London', 'country': 'UK', 'region': 'EMEA'},
            'NYC': {'city': 'New York', 'country': 'USA', 'region': 'Americas'},
            'SF': {'city': 'San Francisco', 'country': 'USA', 'region': 'Americas'},
            'SG': {'city': 'Singapore', 'country': 'Singapore', 'region': 'APAC'},
            'BER': {'city': 'Berlin', 'country': 'Germany', 'region': 'EMEA'}
        }

        self.certification_types = {
            'Product Architecture': ['AWS Solutions Architect', 'Azure Architect', 'TOGAF', 'Google Cloud Architect'],
            'Design & UX': ['Google UX Design', 'Interaction Design Foundation', 'Human Factors International'],
            'Engineering Delivery': ['AWS Developer', 'Kubernetes', 'Azure Developer', 'Scrum Master'],
            'Product Strategy': ['Product Management (PMP)', 'Agile Product Management', 'PSPO'],
            'Client Services': ['Prince2', 'PMP', 'Agile Project Management', 'Scrum Master']
        }

    def generate_employee(self):
        is_outlier = random.random() < 0.05  # 5% chance of outlier
        department = random.choice(list(self.departments.keys()))
        
        level_weights = self._get_level_weights(department, is_outlier)
        level = np.random.choice(['entry', 'mid', 'senior'], p=level_weights)
        
        position = random.choice(self.departments[department][level])
        hire_date = self._generate_hire_date(level, is_outlier)
        
        location_data = self._assign_office(department)
        
        employee_data = {
            'employee_id': self.fake.unique.random_number(digits=6),
            'full_name': self.fake.name(),
            'department': department,
            'position': position,
            'level': level,
            'hire_date': hire_date,
            
            'city': location_data['city'],
            'country': location_data['country'],
            'region': location_data['region'],
            'remote_work_ratio': self._calculate_remote_ratio(department, level),
            'travel_percentage': self._calculate_travel_percentage(department, level),
            
            'base_salary': self._calculate_salary(level, department, hire_date, is_outlier),
            'billing_rate': self._calculate_billing_rate(level, department, is_outlier),
            'utilization_target': self._calculate_utilization_target(department, level),
            'actual_utilization': self._calculate_actual_utilization(is_outlier),
            
            'primary_specialization': random.choice(self.specializations[department]),
            'secondary_specialization': random.choice(self.specializations[department]),
            'industry_expertise': random.sample(self.industry_expertise, k=random.randint(2, 4)),
            'certifications': self._assign_certifications(department, level),
            
            'performance_score': self._generate_performance_score(level, is_outlier),
            'client_satisfaction': self._generate_client_satisfaction(is_outlier),
            'internal_project_contribution': self._generate_internal_contribution(level),
            
            'active_projects': self._calculate_active_projects(level, department),
            'project_complexity_score': self._calculate_project_complexity(level, is_outlier),
            'delivery_quality_score': self._generate_delivery_quality(level, is_outlier),
            'innovation_contribution': self._calculate_innovation_score(level, department),
            
            'training_hours': self._calculate_training_hours(level, is_outlier),
            'mentorship_hours': self._calculate_mentorship_hours(level),
            'promotion_readiness': self._calculate_promotion_readiness(level, hire_date, is_outlier),
            
            'flight_risk_score': self._calculate_flight_risk(is_outlier),
            'engagement_score': self._generate_engagement_score(is_outlier),
            'knowledge_sharing_index': self._calculate_knowledge_sharing(level, is_outlier),
            
            'manager_id': None,
            'total_comp': None,
            'team_lead_projects': 0,
            'direct_reports': 0
        }
        
        return employee_data


    def _get_level_weights(self, department, is_outlier):
        base_weights = {
            'Product Architecture': [0.3, 0.5, 0.2],
            'Design & UX': [0.4, 0.4, 0.2],
            'Engineering Delivery': [0.5, 0.3, 0.2],
            'Product Strategy': [0.3, 0.4, 0.3],
            'Client Services': [0.2, 0.5, 0.3]
        }
        
        if is_outlier:
            return [0.2, 0.3, 0.5]  # More senior roles for outliers
        return base_weights[department]

    def _generate_hire_date(self, level, is_outlier):
        experience_ranges = {
            'entry': (0, 2),
            'mid': (2, 6),
            'senior': (5, 12)
        }
        
        if is_outlier:
            experience_ranges = {
                'entry': (2, 4),
                'mid': (5, 8),
                'senior': (10, 15)
            }
        
        min_years, max_years = experience_ranges[level]
        days_ago = random.randint(min_years * 365, max_years * 365)
        return self.reference_date - timedelta(days=days_ago)

    def _calculate_salary(self, level, department, hire_date, is_outlier):
        base_rates = {
            'entry': {'min': 65000, 'max': 85000},
            'mid': {'min': 95000, 'max': 145000},
            'senior': {'min': 150000, 'max': 250000}
        }
        
        dept_multipliers = {
            'Product Architecture': 1.25,  # High demand for architects
            'Design & UX': 1.15,  # Growing field
            'Engineering Delivery': 1.20,  # Technical expertise premium
            'Product Strategy': 1.18,  # Strategic value
            'Client Services': 1.10  # Client-facing premium
        }
        
        location_multipliers = {
            'London': 1.2,
            'New York': 1.3,
            'San Francisco': 1.35,
            'Singapore': 1.15,
            'Berlin': 1.1
        }
        
        years_experience = (self.reference_date - hire_date).days / 365
        experience_multiplier = 1 + (min(years_experience, 15) * 0.025)  # Diminishing returns after 15 years
        
        range_min = base_rates[level]['min']
        range_max = base_rates[level]['max']
        
        if is_outlier:
            experience_multiplier *= random.uniform(1.2, 1.4)
            
        base = random.uniform(range_min, range_max)
        market_adjusted = base * dept_multipliers[department] * experience_multiplier
        
        return round(market_adjusted * location_multipliers[self._get_employee_location()], -3)

    def _calculate_billing_rate(self, level, department, is_outlier):
        salary_cost = self._calculate_salary(level, department, self.reference_date, is_outlier)
        overhead_multiplier = 2.5  # Standard consulting overhead
        utilization_target = self._calculate_utilization_target(department, level) / 100
        annual_billable_hours = 2080 * utilization_target  # 2080 = 52 weeks * 40 hours
        
        base_rate = (salary_cost * overhead_multiplier) / annual_billable_hours
        
        # Premium factors
        expertise_premium = {
            'Product Architecture': 1.4,
            'Design & UX': 1.3,
            'Engineering Delivery': 1.35,
            'Product Strategy': 1.45,
            'Client Services': 1.25
        }
        
        market_rate = base_rate * expertise_premium[department]
        
        if is_outlier:
            market_rate *= random.uniform(1.15, 1.25)
        
        # Round to nearest 50
        return round(market_rate / 50) * 50

    def _calculate_utilization_target(self, department, level):
        base_target = {
            'Product Architecture': 70,  # More time for design and planning
            'Design & UX': 75,  # Balance between client work and research
            'Engineering Delivery': 85,  # High billable focus
            'Product Strategy': 65,  # Strategic work requires non-billable time
            'Client Services': 80   # Client-facing with administrative overhead
        }
        
        level_adjustments = {
            'entry': 10,    # Higher utilization expected
            'mid': 0,       # Baseline
            'senior': -15   # More time for leadership and strategic activities
        }
        
        position_overhead = {
            'Chief Architect': -10,
            'Design Director': -15,
            'Delivery Director': -20,
            'Head of Product': -15,
            'Account Director': -10
        }
        
        target = base_target[department] + level_adjustments[level]
        
        # Apply position-specific adjustments
        if self.departments[department][level] in position_overhead:
            target += position_overhead[self.departments[department][level]]
        
        # Add slight variation
        target += random.uniform(-3, 3)
        
        return round(min(max(target, 50), 90))

    def _calculate_active_projects(self, level, department):
        max_capacity = {
            'Product Architecture': 3,    # Complex, long-term engagements
            'Design & UX': 4,            # Multiple parallel design projects
            'Engineering Delivery': 2,    # Focused development work
            'Product Strategy': 4,        # Multiple client strategies
            'Client Services': 6         # Multiple client relationships
        }
        
        level_multipliers = {
            'entry': 0.6,   # Limited capacity
            'mid': 0.8,     # Growing capacity
            'senior': 1.0   # Full capacity
        }
        
        utilization = self._calculate_actual_utilization(False) / 100
        base_capacity = max_capacity[department] * level_multipliers[level] * utilization
        
        # Round to nearest whole number and ensure minimum of 1
        return max(1, round(base_capacity))

    def _calculate_project_complexity(self, level, is_outlier):
        complexity_weights = {
            'entry': [0.5, 0.3, 0.15, 0.05, 0],     # Mostly simple projects
            'mid': [0.1, 0.3, 0.4, 0.15, 0.05],     # Mix of complexities
            'senior': [0, 0.1, 0.3, 0.4, 0.2]       # More complex projects
        }
        
        if is_outlier:
            complexity_weights = {
                'entry': [0.2, 0.3, 0.3, 0.15, 0.05],
                'mid': [0, 0.15, 0.25, 0.4, 0.2],
                'senior': [0, 0, 0.2, 0.4, 0.4]
            }
        
        return np.random.choice(range(1, 6), p=complexity_weights[level])

    def _calculate_promotion_readiness(self, level, hire_date, is_outlier):
        if level == 'senior':
            return round(random.uniform(0, 30), 1)
        
        years_in_role = (self.reference_date - hire_date).days / 365
        performance_score = self._generate_performance_score(level, is_outlier)
        project_complexity = self._calculate_project_complexity(level, is_outlier)
        
        base_score = 40
        
        # Experience factor
        experience_score = min(years_in_role * 10, 30)
        
        # Performance factor
        performance_factor = (performance_score - 3) * 20
        
        # Complexity factor
        complexity_factor = project_complexity * 5
        
        # Certifications factor
        cert_count = len(self._assign_certifications(self.current_department, level))
        cert_factor = cert_count * 5
        
        total_score = base_score + experience_score + performance_factor + complexity_factor + cert_factor
        
        if is_outlier:
            total_score *= random.uniform(1.2, 1.3)
        
        return round(min(max(total_score, 0), 100), 1)

    def _calculate_flight_risk(self, is_outlier):
        base_risk = 30
        
        # Factors that increase risk
        if self._calculate_promotion_readiness(self.current_level, self.hire_date, is_outlier) > 80:
            base_risk += 20
        
        if self._generate_engagement_score(is_outlier) < 7:
            base_risk += 15
        
        if self._calculate_actual_utilization(is_outlier) > 90:
            base_risk += 10
        
        years_in_role = (self.reference_date - self.hire_date).days / 365
        if years_in_role > 3:
            base_risk += 15
        
        if self._generate_performance_score(self.current_level, is_outlier) < 3.5:
            base_risk += 25
        
        # Market demand factor
        if self.current_department in ['Engineering Delivery', 'Product Architecture']:
            base_risk += 10
        
        if is_outlier:
            return random.choice([85, 15])
        
        return round(min(max(base_risk + random.uniform(-10, 10), 0), 100))

    def _calculate_innovation_score(self, level, department):
        base_innovation = {
            'Product Architecture': 7.5,    # High technical innovation
            'Design & UX': 8.0,            # Creative solutions
            'Engineering Delivery': 7.0,    # Technical implementation
            'Product Strategy': 8.5,        # Business innovation
            'Client Services': 6.5          # Process innovation
        }
        
        level_factors = {
            'entry': 0.7,    # Learning phase
            'mid': 1.0,      # Established
            'senior': 1.3    # Leading innovation
        }
        
        performance_score = self._generate_performance_score(level, False)
        performance_factor = (performance_score - 3) * 0.2
        
        project_complexity = self._calculate_project_complexity(level, False)
        complexity_factor = (project_complexity - 3) * 0.15
        
        cert_count = len(self._assign_certifications(department, level))
        certification_factor = cert_count * 0.1
        
        innovation = (base_innovation[department] * level_factors[level] * 
                    (1 + performance_factor + complexity_factor + certification_factor))
        
        return round(min(max(innovation + random.uniform(-0.5, 0.5), 1), 10), 1)

    def _assign_office(self, department):
        weights = {
            'Product Architecture': [0.3, 0.2, 0.2, 0.15, 0.15],
            'Design & UX': [0.25, 0.3, 0.2, 0.15, 0.1],
            'Engineering Delivery': [0.2, 0.25, 0.25, 0.15, 0.15],
            'Product Strategy': [0.3, 0.25, 0.2, 0.15, 0.1],
            'Client Services': [0.3, 0.2, 0.2, 0.2, 0.1]
        }
        
        office_key = np.random.choice(
            list(self.office_locations.keys()),
            p=weights[department]
        )
        return self.office_locations[office_key]

    def _calculate_remote_ratio(self):
        """Calculate remote work ratio based on department and level."""
        base_ratio = {
            'Product Architecture': 0.7,
            'Design & UX': 0.6,
            'Engineering Delivery': 0.8,
            'Product Strategy': 0.5,
            'Client Services': 0.4
        }
        
        level_adjustments = {
            'entry': -0.1,
            'mid': 0,
            'senior': 0.1
        }
        
        ratio = base_ratio[self.current_department] + level_adjustments[self.current_level]
        ratio = min(max(ratio + random.uniform(-0.1, 0.1), 0.2), 0.9)
        return round(ratio, 2)

    def _calculate_travel_percentage(self):
        """Calculate travel percentage based on department and level."""
        base_travel = {
            'Product Architecture': 30,
            'Design & UX': 20,
            'Engineering Delivery': 15,
            'Product Strategy': 40,
            'Client Services': 50
        }
        
        level_multipliers = {
            'entry': 0.5,
            'mid': 1.0,
            'senior': 1.2
        }
        
        travel = base_travel[self.current_department] * level_multipliers[self.current_level]
        return round(min(travel + random.uniform(-5, 5), 80))

    def _calculate_actual_utilization(self, is_outlier):
        if is_outlier:
            return random.choice([45, 95])
        return round(random.uniform(65, 85), 1)

    def _assign_certifications(self):
        """Assign relevant certifications based on role and level."""
        available_certs = self.certification_types.get(self.current_department, [])
        
        # Number of certifications based on level
        cert_counts = {
            'entry': (0, 2),
            'mid': (1, 3),
            'senior': (2, 4)
        }
        
        min_certs, max_certs = cert_counts[self.current_level]
        num_certs = random.randint(min_certs, min(max_certs, len(available_certs)))
        
        return random.sample(available_certs, k=num_certs)

    def _assign_industry_expertise(self):
        # Ensure we don't try to sample more than available
        num_industries = random.randint(2, min(4, len(self.industry_expertise)))
        return random.sample(self.industry_expertise, k=num_industries)

    def _generate_performance_score(self, level, is_outlier):
        base_scores = {
            'entry': np.random.normal(3.5, 0.4),
            'mid': np.random.normal(3.8, 0.3),
            'senior': np.random.normal(4.1, 0.2)
        }
        
        score = base_scores[level]
        if is_outlier:
            score = random.choice([2.0, 4.8])
        
        return round(min(max(score, 1), 5), 1)

    def _generate_client_satisfaction(self, level, department, projects, performance_score):
        """Generate client satisfaction score based on multiple factors"""
        if not projects:
            return None
        
        # Base satisfaction weighted by department expertise
        dept_base_satisfaction = {
            'Product Architecture': 4.2,
            'Design & UX': 4.0,
            'Engineering Delivery': 4.1,
            'Product Strategy': 4.0,
            'Client Services': 4.3
        }
        
        # Calculate weighted satisfaction based on multiple factors
        total_weight = 0
        weighted_satisfaction = 0
        
        for project in projects:
            project_weight = 1.0
            base = dept_base_satisfaction[department]
            
            # Project complexity factor
            complexity = project['complexity']
            complexity_factor = {
                1: 0.1,    # Simple projects easier to satisfy
                2: 0.05,
                3: 0,
                4: -0.05,
                5: -0.1    # Complex projects harder to satisfy
            }
            
            # Project status factor
            status_factor = {
                'Planning': 0.05,      # Early stage optimism
                'In Progress': 0,      # Neutral
                'Final Stage': -0.05   # Delivery pressure
            }
            
            # Calculate project-specific satisfaction
            project_satisfaction = base
            project_satisfaction += complexity_factor[complexity]
            project_satisfaction += status_factor[project['status']]
            
            # Performance impact
            performance_impact = (performance_score - 3.5) * 0.2
            project_satisfaction += performance_impact
            
            # Level-based adjustments
            level_factor = {
                'entry': -0.1,    # Less experience
                'mid': 0,         # Baseline
                'senior': 0.1     # More experience
            }
            project_satisfaction += level_factor[level]
            
            # Add slight randomization for realism
            project_satisfaction += random.uniform(-0.1, 0.1)
            
            # Accumulate weighted satisfaction
            weighted_satisfaction += project_satisfaction * project_weight
            total_weight += project_weight
        
        final_satisfaction = round(weighted_satisfaction / total_weight, 1)
        return min(max(final_satisfaction, 1.0), 5.0)

    def _generate_internal_contribution(self, level):
        base_contribution = {
            'entry': (1, 3),
            'mid': (3, 5),
            'senior': (4, 7)
        }
        min_contrib, max_contrib = base_contribution[level]
        return random.randint(min_contrib, max_contrib)

    def _generate_delivery_quality(self, level, is_outlier):
        base_quality = {
            'entry': np.random.normal(85, 5),
            'mid': np.random.normal(90, 4),
            'senior': np.random.normal(93, 3)
        }
        
        quality = base_quality[level]
        if is_outlier:
            quality = random.choice([75, 98])
            
        return round(min(max(quality, 70), 100), 1)

    def _calculate_training_hours(self, level, is_outlier):
        base_hours = {
            'entry': (40, 60),
            'mid': (30, 50),
            'senior': (20, 40)
        }
        
        if is_outlier:
            return random.choice([10, 100])
            
        min_hours, max_hours = base_hours[level]
        return random.randint(min_hours, max_hours)

    def _calculate_mentorship_hours(self, level):
        # Negative hours indicate receiving mentorship, positive hours indicate giving mentorship
        mentorship_ranges = {
            'entry': (-30, -10),
            'mid': (-10, 20),
            'senior': (20, 40)
        }
        min_hours, max_hours = mentorship_ranges[level]
        return random.randint(min_hours, max_hours)


    def _calculate_knowledge_sharing(self, level, is_outlier):
        base_sharing = {
            'entry': (2, 4),
            'mid': (4, 7),
            'senior': (7, 9)
        }
        
        if is_outlier:
            return random.choice([1, 10])
            
        min_share, max_share = base_sharing[level]
        return round(random.uniform(min_share, max_share), 1)
    

class ConsultancyDataGenerator:
    def __init__(self, num_records, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        
        self.employee_generator = ConsultancyEmployee(self.fake)
        self.num_records = num_records

    def generate_dataset(self):
        """Generate the complete dataset"""
        employees = []
        for _ in range(self.num_records):
            employee = self.employee_generator.generate_employee()
            employees.append(employee)
            
        # Convert to DataFrame
        df = pd.DataFrame(employees)
        
        # Post-process the data
        self._assign_managers(df)
        self._calculate_total_compensation(df)
        self._update_team_metrics(df)
        
        return df
    
    def _assign_managers(self, df):
        """Assign managers to employees based on level and department"""
        for dept in df['department'].unique():
            dept_df = df[df['department'] == dept]
            
            # Senior employees don't have managers
            seniors = dept_df[dept_df['level'] == 'senior']
            potential_managers = seniors['employee_id'].tolist()
            
            if potential_managers:
                # Assign managers to mid and entry level employees
                for level in ['mid', 'entry']:
                    level_employees = df[(df['department'] == dept) & (df['level'] == level)]
                    for idx in level_employees.index:
                        if potential_managers:
                            df.at[idx, 'manager_id'] = random.choice(potential_managers)

    def _calculate_total_compensation(self, df):
        """Calculate total compensation including bonus and benefits"""
        df['total_comp'] = df.apply(lambda row: 
            row['base_salary'] * (1 + 
                                (0.2 if row['level'] == 'senior' else
                                 0.15 if row['level'] == 'mid' else 0.1) +
                                (0.05 if row['performance_score'] >= 4.5 else 0)), 
            axis=1)

    def _update_team_metrics(self, df):
        """Update team lead projects and direct reports counts"""
        # Count direct reports
        manager_counts = df['manager_id'].value_counts()
        for emp_id in manager_counts.index:
            if pd.notna(emp_id):
                mask = df['employee_id'] == emp_id
                df.loc[mask, 'direct_reports'] = manager_counts[emp_id]
                
        # Assign team lead projects for senior and mid-level employees
        df.loc[df['level'] == 'senior', 'team_lead_projects'] = \
            df.loc[df['level'] == 'senior', 'active_projects'].apply(lambda x: random.randint(1, x))
        df.loc[df['level'] == 'mid', 'team_lead_projects'] = \
            df.loc[df['level'] == 'mid', 'active_projects'].apply(lambda x: random.randint(0, max(x-1, 0)))

    def _prepare_data_for_export(self):
        """Prepare and clean data for export"""
        df = self.generate_dataset()
        
        df['hire_date'] = df['hire_date'].dt.strftime('%Y-%m-%d')
        
        for column in df.select_dtypes(include=[np.number]).columns:
            df[column] = df[column].astype(float)
        
        list_columns = ['industry_expertise', 'certifications']
        for col in list_columns:
            df[col] = df[col].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
        
        return df

    def _save_to_excel(self, df, filepath):
        """Save data to Excel with formatting"""
        with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name='Employee Data', index=False)
            
            summary_stats = pd.DataFrame({
                'Department Distribution': df['department'].value_counts(),
                'Average Salary by Level': df.groupby('level')['base_salary'].mean(),
                'Average Utilization': df.groupby('department')['actual_utilization'].mean()
            })
            summary_stats.to_excel(writer, sheet_name='Summary Stats')

    def _save_to_json(self, df, filepath):
        """Save data to JSON with proper formatting"""
        records = df.to_dict(orient='records')
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(records, f, indent=2, ensure_ascii=False)

    def generate_and_save_data(self, base_filename='consultancy_data'):
        """Generate and save data in both Excel and JSON formats"""
        df = self._prepare_data_for_export()
        
        os.makedirs('output', exist_ok=True)
        
        excel_path = os.path.join('output', f'{base_filename}.xlsx')
        json_path = os.path.join('output', f'{base_filename}.json')
        
        self._save_to_excel(df, excel_path)
        print(f"Excel file created successfully: {excel_path}")
        
        self._save_to_json(df, json_path)
        print(f"JSON file created successfully: {json_path}")
        
        print(f"Number of records: {len(df)}")

def main():
    generator = ConsultancyDataGenerator(num_records=50)
    
    try:
        generator.generate_and_save_data()
        print("Data generation completed successfully!")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

An error occurred: 'ConsultancyEmployee' object has no attribute '_get_employee_location'


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import timedelta, date
import os
from openpyxl import load_workbook


class HRDataGenerator:
    def __init__(self, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        
        # Updated constants
        self.MIN_AGE = 18  # Changed to include interns
        self.MAX_AGE = 65
        self.MIN_YEARS_HISTORY = 10
        self.TODAY = date.today()
        self.BASE_YEAR = self.TODAY.year
        
        self._initialize_lookup_data()
    
    def _initialize_lookup_data(self):
        """Initialize all lookup data with realistic values"""
        self.departments = {
            'Engineering': {
                'roles': ['Intern', 'Junior Software Engineer', 'Software Engineer', 'Senior Software Engineer', 'Tech Lead', 'Engineering Manager'],
                'salary_ranges': {
                    'Intern': (45000, 65000),
                    'Junior Software Engineer': (65000, 85000),
                    'Software Engineer': (85000, 130000),
                    'Senior Software Engineer': (120000, 180000),
                    'Tech Lead': (150000, 200000),
                    'Engineering Manager': (170000, 250000)
                },
                'experience_requirements': {
                    'Intern': (0, 1),
                    'Junior Software Engineer': (0, 2),
                    'Software Engineer': (2, 5),
                    'Senior Software Engineer': (5, 10),
                    'Tech Lead': (8, 15),
                    'Engineering Manager': (10, 20)
                },
                'intern_ratio': 0.15
            },
            'IT': {
                'roles': ['Intern', 'Junior System Administrator', 'System Administrator', 'Senior System Administrator', 'Network Engineer', 'Security Analyst', 'IT Manager'],
                'salary_ranges': {
                    'Intern': (40000, 55000),
                    'Junior System Administrator': (55000, 75000),
                    'System Administrator': (70000, 115000),
                    'Senior System Administrator': (90000, 140000),
                    'Network Engineer': (85000, 145000),
                    'Security Analyst': (95000, 160000),
                    'IT Manager': (130000, 190000)
                },
                'experience_requirements': {
                    'Intern': (0, 1),
                    'Junior System Administrator': (0, 2),
                    'System Administrator': (2, 5),
                    'Senior System Administrator': (5, 10),
                    'Network Engineer': (3, 8),
                    'Security Analyst': (3, 8),
                    'IT Manager': (8, 15)
                },
                'intern_ratio': 0.10
            },
            'Sales': {
                'roles': ['Sales Intern', 'Junior Sales Representative', 'Sales Representative', 'Senior Sales Representative', 'Account Executive', 'Sales Manager'],
                'salary_ranges': {
                    'Sales Intern': (35000, 45000),
                    'Junior Sales Representative': (40000, 60000),
                    'Sales Representative': (50000, 90000),
                    'Senior Sales Representative': (70000, 120000),
                    'Account Executive': (85000, 150000),
                    'Sales Manager': (110000, 180000)
                },
                'experience_requirements': {
                    'Sales Intern': (0, 1),
                    'Junior Sales Representative': (0, 2),
                    'Sales Representative': (1, 4),
                    'Senior Sales Representative': (4, 8),
                    'Account Executive': (5, 10),
                    'Sales Manager': (8, 15)
                },
                'intern_ratio': 0.08
            },
            'HR': {
                'roles': ['HR Intern', 'HR Assistant', 'HR Specialist', 'Senior HR Specialist', 'Recruiter', 'HR Manager'],
                'salary_ranges': {
                    'HR Intern': (35000, 45000),
                    'HR Assistant': (40000, 55000),
                    'HR Specialist': (55000, 85000),
                    'Senior HR Specialist': (75000, 110000),
                    'Recruiter': (60000, 100000),
                    'HR Manager': (95000, 150000)
                },
                'experience_requirements': {
                    'HR Intern': (0, 1),
                    'HR Assistant': (0, 2),
                    'HR Specialist': (2, 5),
                    'Senior HR Specialist': (5, 10),
                    'Recruiter': (2, 8),
                    'HR Manager': (8, 15)
                },
                'intern_ratio': 0.05
            }
        }
        
        # Add previous companies distribution
        self.prev_companies_probs = {
            '0': 0.15,  # First job
            '1-2': 0.35,  # Most common
            '3-4': 0.30,
            '5-6': 0.15,
            '7+': 0.05   # Rare but possible
        }
        
        # Add project completion base metrics
        self.project_completion_mean = 0.85  # 85% average completion rate
        self.project_completion_std = 0.12   # Standard deviation for normal distribution
        
        self.performance_ratings = {
            'Outstanding': 0.05,
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Needs Improvement': 0.20,
            'Performance Plan': 0.10
        }
        
        self.recruitment_sources = {
            'LinkedIn': 0.25,
            'Employee Referral': 0.20,
            'Company Website': 0.15,
            'Indeed': 0.12,
            'University Partnership': 0.10,
            'Recruitment Agency': 0.08,
            'Diversity Job Fair': 0.05,
            'Stack Overflow': 0.03,
            'GitHub': 0.02
        }
        
        self.termination_reasons = {
            'N/A-StillEmployed': 0.82,
            'Voluntary-Better Opportunity': 0.05,
            'Voluntary-Career Change': 0.02,
            'Voluntary-Relocation': 0.02,
            'Voluntary-Return to School': 0.01,
            'Voluntary-Retirement': 0.02,
            'Involuntary-Performance': 0.02,
            'Involuntary-Attendance': 0.01,
            'Involuntary-Policy Violation': 0.01,
            'Involuntary-Layoff': 0.02
        }
        
        self.education_levels = {
            'High School': 0.10,
            'Some College': 0.15,
            'Associates Degree': 0.10,
            'Bachelors Degree': 0.45,
            'Masters Degree': 0.18,
            'PhD': 0.02
        }
        
        self.remote_status = {
            'On-site': 0.40,
            'Hybrid-2 Days': 0.20,
            'Hybrid-3 Days': 0.25,
            'Full Remote': 0.15
        }
        
        self.ethnicities = {
            'White': 0.60,
            'Asian': 0.15,
            'Hispanic or Latino': 0.12,
            'Black or African American': 0.08,
            'Two or More Races': 0.03,
            'Native American': 0.01,
            'Pacific Islander': 0.01
        }

        self.engagement_modifiers = {
            'salary_satisfaction': (-0.5, 0.5),
            'years_in_role': (-0.2, -0.1),
            'performance': {
                'Outstanding': 0.8,
                'Exceeds Expectations': 0.5,
                'Meets Expectations': 0.0,
                'Needs Improvement': -0.5,
                'Performance Plan': -0.8
            }
        }

        self.base_performance_ratings = {
            'Outstanding': 0.05,
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Needs Improvement': 0.20,
            'Performance Plan': 0.10
        }
        
        self.dept_performance_biases = {
            'Engineering': {
                'Outstanding': 1.2,
                'Exceeds Expectations': 1.1
            },
            'Sales': {
                'Outstanding': 1.3,
                'Inconsistent': 1.2
            }
        }


    def _generate_date(self, start_year_offset=-10, end_year_offset=0):
        """
        Generate a random date within specified year offsets from today.
        
        Parameters:
        start_year_offset (int): Number of years before today to start range (negative number)
        end_year_offset (int): Number of years before/after today to end range
        
        Returns:
        date: A random date within the specified range
        """
        start_date = self.TODAY + timedelta(days=365 * start_year_offset)
        end_date = self.TODAY + timedelta(days=365 * end_year_offset)
        
        # Generate random date between start and end
        days_between = (end_date - start_date).days
        random_days = np.random.randint(0, max(1, days_between))
        return start_date + timedelta(days=random_days)

    def _calculate_age(self, birth_date):
        """Calculate age based on birth date"""
        today = self.TODAY
        age = today.year - birth_date.year
        # Adjust age if birthday hasn't occurred this year
        if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
            age -= 1
        return age

    def _calculate_years_experience(self, hire_date):
        """Calculate years of experience based on hire date"""
        days_employed = (self.TODAY - hire_date).days
        return round(days_employed / 365.25, 1)
        
    def _generate_work_satisfaction_metrics(self, years_experience, performance, role):
        """Generate realistic work satisfaction metrics based on various factors"""
        base_satisfaction = np.random.normal(7, 2)  # Base satisfaction out of 10
        
        # Adjust based on performance
        performance_adjustments = {
            'Outstanding': 2,
            'Exceeds Expectations': 1,
            'Meets Expectations': 0,
            'Needs Improvement': -1.5,
            'Performance Plan': -2.5
        }
        
        # Adjust based on role level (more senior roles tend to be more satisfied)
        is_senior = any(senior_term in role.lower() for senior_term in ['senior', 'lead', 'manager'])
        role_adjustment = 1 if is_senior else 0
        
        # Adjust based on years of experience (slight burnout factor)
        experience_adjustment = -0.5 if years_experience > 5 else 0
        
        # Calculate final satisfaction score
        satisfaction = base_satisfaction + performance_adjustments[performance] + role_adjustment + experience_adjustment
        satisfaction = max(1, min(10, satisfaction))  # Clamp between 1 and 10
        
        return round(satisfaction, 1)

    def _generate_productivity_metrics(self, performance, years_experience):
        """Generate realistic productivity metrics"""
        base_productivity = np.random.normal(80, 10)  # Base productivity percentage
        
        # Adjust based on performance
        performance_adjustments = {
            'Outstanding': 15,
            'Exceeds Expectations': 10,
            'Meets Expectations': 0,
            'Needs Improvement': -10,
            'Performance Plan': -15
        }
        
        # Experience adjustment (diminishing returns after 5 years)
        experience_adjustment = min(years_experience * 2, 10)
        
        productivity = base_productivity + performance_adjustments[performance] + experience_adjustment
        productivity = max(40, min(120, productivity))  # Clamp between 40% and 120%
        
        return round(productivity, 1)

    def _get_performance_distribution(self, department, years_experience, role):
        """Get performance distribution adjusted for department and experience"""
        base_dist = self.base_performance_ratings.copy()
        
        # Apply department-specific biases
        if department in self.dept_performance_biases:
            for rating, bias in self.dept_performance_biases[department].items():
                base_dist[rating] *= bias
        
        # Experience adjustments
        if years_experience < 1:
            base_dist['Exceeds Expectations'] *= 0.5
            base_dist['Meets+ Expectations'] *= 0.7
            base_dist['Meets Expectations'] *= 1.3
        elif years_experience > 5:
            base_dist['Exceeds Expectations'] *= 1.2
            base_dist['Needs Improvement'] *= 0.8
        
        # Role-specific adjustments
        if 'Senior' in role or 'Lead' in role:
            base_dist['Exceeds Expectations'] *= 1.3
            base_dist['Performance Plan'] *= 0.7
        elif 'Intern' in role:
            base_dist['Exceeds Expectations'] *= 0.6
            base_dist['Meets Expectations'] *= 1.2
        
        # Normalize probabilities
        total = sum(base_dist.values())
        return {k: v/total for k, v in base_dist.items()}

    def _generate_experience_based_role(self, department):
        """Generate role based on experience distribution with realistic ratios"""
        dept_roles = self.departments[department]['roles']
        experience_reqs = self.departments[department]['experience_requirements']
        
        # Weight roles to favor more junior positions and maintain realistic ratios
        weights = []
        for role in dept_roles:
            if 'Intern' in role:
                weights.append(0.08)  # 8% interns
            elif 'Junior' in role or 'Coordinator' in role:
                weights.append(0.25)  # 25% junior positions
            elif 'Senior' in role or 'Lead' in role:
                weights.append(0.17)  # 17% senior positions
            else:
                weights.append(0.50)  # 50% mid-level positions
        
        # Normalize weights
        weights = np.array(weights) / sum(weights)
        return np.random.choice(dept_roles, p=weights)

    def _generate_age(self, role, department, hire_date):
        """Generate age based on role and realistic distributions"""
        min_exp, _ = self.departments[department]['experience_requirements'][role]
        years_since_hire = (self.TODAY - hire_date).days / 365.25
        
        if 'Intern' in role:
            return np.random.randint(18, 25)
        elif 'Junior' in role or 'Coordinator' in role:
            base_age = np.random.normal(25, 3)
        elif min_exp <= 2:
            base_age = np.random.normal(28, 4)
        elif min_exp <= 5:
            base_age = np.random.normal(32, 5)
        else:
            base_age = np.random.normal(38, 7)
        
        # Add some outliers
        if np.random.random() < 0.05:  # 5% chance of being an outlier
            if np.random.random() < 0.5:
                base_age += np.random.normal(10, 3)  # Older outlier
            else:
                base_age -= np.random.normal(5, 2)   # Younger outlier
        
        # Ensure age makes sense with hire date and experience
        min_possible_age = 18 + years_since_hire
        max_possible_age = 65
        
        age = max(min_possible_age, min(max_possible_age, base_age))
        return int(round(age))

    def _calculate_salary(self, role, department, performance, years_experience):
        """Calculate salary with realistic variations and market factors"""
        base_range = self.departments[department]['salary_ranges'][role]
        min_exp, max_exp = self.departments[department]['experience_requirements'][role]
        
        # Calculate base salary within range with slight random variation
        base = np.random.normal(
            (base_range[0] + base_range[1]) / 2,
            (base_range[1] - base_range[0]) / 6
        )
        
        # Ensure within range
        base = max(base_range[0], min(base_range[1], base))
        
        # Updated performance multipliers
        performance_multipliers = {
            'Outstanding': np.random.uniform(1.15, 1.20),
            'Exceeds Expectations': np.random.uniform(1.08, 1.14),
            'Meets Expectations': np.random.uniform(0.98, 1.02),
            'Needs Improvement': np.random.uniform(0.92, 0.97),
            'Performance Plan': np.random.uniform(0.85, 0.90)
        }
        
        # [Rest of the method remains the same]
        exp_multiplier = 1 + (np.log1p(min(years_experience, max_exp)) * np.random.uniform(0.02, 0.04))
        
        market_adjustments = {
            'Engineering': np.random.uniform(1.05, 1.15),
            'IT': np.random.uniform(1.02, 1.10),
            'Sales': np.random.uniform(0.95, 1.05),
            'HR': np.random.uniform(0.98, 1.02)
        }
        
        role_adjustment = 1.0
        if 'Senior' in role:
            role_adjustment *= np.random.uniform(1.05, 1.15)
        elif 'Lead' in role:
            role_adjustment *= np.random.uniform(1.10, 1.20)
        elif 'Intern' in role:
            role_adjustment *= np.random.uniform(0.85, 0.95)
        
        final_salary = (base 
                    * performance_multipliers[performance]
                    * exp_multiplier
                    * market_adjustments[department]
                    * role_adjustment)
        
        noise = np.random.uniform(0.98, 1.02)
        final_salary *= noise
        
        return round(final_salary, 2)

    def _calculate_engagement_score(self, salary, years_experience, performance, role, termination_reason):
        """Calculate employee engagement score based on multiple factors"""
        base_score = np.random.normal(3.5, 0.5)
        
        # Salary satisfaction impact
        salary_mod = np.random.uniform(*self.engagement_modifiers['salary_satisfaction'])
        base_score += salary_mod
        
        # Years in role impact (stagnation effect)
        if years_experience > 2 and 'Senior' not in role and 'Lead' not in role:
            stagnation = np.random.uniform(*self.engagement_modifiers['years_in_role'])
            base_score += stagnation * (years_experience - 2)
        
        # Performance impact
        base_score += self.engagement_modifiers['performance'][performance]
        
        # Role-specific adjustments
        if 'Intern' in role:
            base_score += np.random.uniform(0.2, 0.5)  # Interns tend to be more engaged
        elif 'Senior' in role or 'Lead' in role:
            base_score += np.random.uniform(-0.2, 0.4)  # More variable engagement
        
        # Termination impact
        if termination_reason != 'N/A-StillEmployed':
            if 'Voluntary' in termination_reason:
                base_score -= np.random.uniform(0.5, 1.5)
            elif 'Involuntary' in termination_reason:
                base_score -= np.random.uniform(1.0, 2.0)
        
        # Ensure score is within realistic bounds (1-5)
        return round(max(1.0, min(5.0, base_score)), 2)

    def _generate_absence_days(self, performance, years_experience, role):
        """Generate realistic absence days based on multiple factors"""
        if 'Intern' in role:
            # Interns have fewer absence days due to shorter tenure
            base_days = np.random.negative_binomial(2, 0.5)
        else:
            # Base absence days following negative binomial distribution
            base_days = np.random.negative_binomial(5, 0.7)
        
        # Modify based on performance
        performance_multipliers = {
            'Exceeds Expectations': 0.7,
            'Meets+ Expectations': 0.85,
            'Meets Expectations': 1.0,
            'Needs Improvement': 1.3,
            'Performance Plan': 1.5
        }
        
        # Modify based on experience (slightly more absence with more experience)
        exp_modifier = 1 + (years_experience * 0.02)
        
        # Calculate final absence days
        absence_days = int(base_days * performance_multipliers[performance] * exp_modifier)
        
        # Add occasional outliers
        if np.random.random() < 0.05:  # 5% chance of unusual absence patterns
            absence_days += np.random.randint(10, 30)
        
        return absence_days

    def _generate_employee(self):
        """Generate a single employee record with enhanced realism"""
        # Select department and role with intern consideration
        department = np.random.choice(list(self.departments.keys()))
        
        # Determine if this will be an intern based on department's intern ratio
        is_intern = np.random.random() < self.departments[department]['intern_ratio']
        
        # Filter roles based on intern status
        available_roles = [role for role in self.departments[department]['roles'] 
                         if ('intern' in role.lower()) == is_intern]
        role = np.random.choice(available_roles)
        
        # Generate dates with consideration for role
        if 'intern' in role.lower():
            hire_date = self._generate_date(start_year_offset=-1)  # Interns typically within last year
            birth_date = self._generate_date(
                start_year_offset=-(self.MIN_AGE + 5),
                end_year_offset=-(self.MIN_AGE)
            )
        else:
            hire_date = self._generate_date()
            birth_date = self._generate_date(
                start_year_offset=-(self.MAX_AGE),
                end_year_offset=-(22)  # Most non-interns are at least 22
            )
        
        # Calculate age and experience
        age = self._calculate_age(birth_date)
        years_experience = self._calculate_years_experience(hire_date)
        
        # Generate performance rating (interns more likely to be "Meets Expectations")
        if is_intern:
            performance_probs = {'Meets Expectations': 0.7, 'Exceeds Expectations': 0.15, 
                               'Needs Improvement': 0.15}
            performance = np.random.choice(
                list(performance_probs.keys()),
                p=list(performance_probs.values())
            )
        else:
            performance = np.random.choice(
                list(self.performance_ratings.keys()),
                p=list(self.performance_ratings.values())
            )
        
        # Determine employment status and termination details
        term_reason = np.random.choice(
            list(self.termination_reasons.keys()),
            p=list(self.termination_reasons.values())
        )
        
        term_date = None
        if term_reason != 'N/A-StillEmployed':
            term_date = self._generate_date(
                start_year_offset=max(-3, int(-years_experience)),
                end_year_offset=0
            )
        
        # Generate last review date
        max_review_date = term_date if term_date else self.TODAY
        last_review_date = self._generate_date(
            start_year_offset=-1,
            end_year_offset=0
        )
        if last_review_date > max_review_date:
            last_review_date = max_review_date
        
        # Generate education level appropriate for role
        if 'intern' in role.lower():
            education_options = ['Some College', 'Bachelors Degree']
            education_probs = [0.3, 0.7]
        else:
            education_options = list(self.education_levels.keys())
            education_probs = list(self.education_levels.values())
        
        education = np.random.choice(education_options, p=education_probs)
        
        # Calculate work satisfaction and productivity
        satisfaction = self._generate_work_satisfaction_metrics(years_experience, performance, role)
        productivity = self._generate_productivity_metrics(performance, years_experience)
        
        # Generate base employee record
        employee = {
            "EmployeeID": self.fake.unique.random_int(min=10000, max=99999),
            "Name": self.fake.name(),
            "Department": department,
            "Position": role,
            "HireDate": hire_date.strftime('%Y-%m-%d'),
            "TerminationDate": term_date.strftime('%Y-%m-%d') if term_date else None,
            "TerminationReason": term_reason,
            "EmploymentStatus": "Terminated" if term_date else "Active",
            "DateOfBirth": birth_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Gender": np.random.choice(['M', 'F'], p=[0.51, 0.49]),
            "Ethnicity": np.random.choice(
                list(self.ethnicities.keys()),
                p=list(self.ethnicities.values())
            ),
            "Salary": self._calculate_salary(role, department, performance, years_experience),
            "PerformanceRating": performance,
            "RecruitmentSource": np.random.choice(
                list(self.recruitment_sources.keys()),
                p=list(self.recruitment_sources.values())
            ),
            "EducationLevel": education,
            "YearsExperience": round(years_experience, 1),
            "PreviousCompanies": np.random.choice(
                list(self.prev_companies_probs.keys()),
                p=list(self.prev_companies_probs.values())
            ),
            "RemoteWorkStatus": np.random.choice(
                list(self.remote_status.keys()),
                p=list(self.remote_status.values())
            ),
            "ProjectCompletionRate": min(100, max(0, np.random.normal(
                self.project_completion_mean * 100,
                self.project_completion_std * 100
            ))),
            "WorkSatisfaction": satisfaction,
            "Productivity": productivity,
            "EngagementScore": round(np.random.normal(3.5, 0.5), 2),
            "AbsenceDays": int(np.random.negative_binomial(5, 0.7)),
            "LastReviewDate": last_review_date.strftime('%Y-%m-%d'),
            "OvertimeHours": int(np.random.exponential(10)) if not is_intern else int(np.random.exponential(3)),
            "TrainingHoursCompleted": int(np.random.normal(40, 15)) if not is_intern else int(np.random.normal(60, 10)),
            "TeamSize": int(np.random.normal(8, 3)),
            "ProjectsCompleted": int(np.random.normal(12, 4) * years_experience) if years_experience > 0 else 0,
            "CertificationsCount": int(np.random.exponential(2)),
        }
        
        # Add department-specific metrics
        if department == 'Sales':
            employee.update({
                "QuotaAttainment": round(min(150, max(50, np.random.normal(100, 20))), 1),
                "LeadConversionRate": round(min(100, max(0, np.random.normal(30, 10))), 1),
                "ClientRetentionRate": round(min(100, max(60, np.random.normal(85, 8))), 1),
                "DealsClosed": int(np.random.normal(15, 5) * years_experience) if years_experience > 0 else 0,
                "AverageContractValue": round(np.random.normal(50000, 20000), 2),
                "ClientSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "SalesTargetPercent": round(min(200, max(0, np.random.normal(95, 25))), 1),
                "ProspectingHours": int(np.random.normal(15, 5)),
                "RepeatBusinessPercent": round(min(100, max(0, np.random.normal(65, 15))), 1)
            })

        elif department == 'Engineering':
            # Calculate bug metrics based on experience and performance
            base_bug_rate = np.random.normal(5, 2)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/5)))  # More experience = fewer bugs
            performance_factors = {
                'Outstanding': 0.6,
                'Exceeds Expectations': 0.8,
                'Meets Expectations': 1.0,
                'Inconsistent': 1.3,
                'Needs Improvement': 1.5,
                'Performance Plan': 1.8
            }
            
            # Calculate code metrics
            code_review_scores = []
            for _ in range(3):  # Last 3 code reviews
                base_score = np.random.normal(8, 1)
                exp_modifier = min(2, years_experience/3)  # Experience improves code quality
                score = min(10, max(1, base_score + exp_modifier))
                code_review_scores.append(round(score, 1))
            
            employee.update({
                "CodeQualityScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "BugsPerProject": round(max(0, base_bug_rate * experience_factor * performance_factors[performance]), 1),
                "CodeReviewScores": code_review_scores,
                "CommitsPerWeek": int(np.random.normal(12, 4)),
                "PullRequestsOpened": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "PullRequestsAccepted": int(np.random.normal(7, 3) * years_experience) if years_experience > 0 else 0,
                "TechnicalDebtScore": round(min(10, max(1, np.random.normal(4, 2))), 1),
                "DocumentationContribution": round(min(100, max(0, np.random.normal(70, 20))), 1),
                "SystemUptime": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "OnCallIncidents": int(np.random.exponential(3)),
                "DeploymentFrequency": int(np.random.normal(5, 2)),
                "CodeCoverage": round(min(100, max(60, np.random.normal(85, 8))), 1)
            })

        elif department == 'IT':
            # Calculate incident resolution metrics based on experience
            base_resolution_time = np.random.normal(4, 1)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/3)))
            
            employee.update({
                "TicketsResolved": int(np.random.normal(120, 30) * years_experience) if years_experience > 0 else 0,
                "AverageResolutionTime": round(max(0.5, base_resolution_time * experience_factor), 1),
                "FirstCallResolutionRate": round(min(100, max(50, np.random.normal(75, 10))), 1),
                "SystemUptimeManaged": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "SecurityIncidentsHandled": int(np.random.exponential(5)),
                "CustomerSatisfactionScore": round(min(10, max(1, np.random.normal(8, 1))), 1),
                "SystemsManaged": int(np.random.normal(15, 5)),
                "PreventiveMaintenance": round(min(100, max(0, np.random.normal(80, 15))), 1),
                "NetworkPerformanceScore": round(min(100, max(70, np.random.normal(92, 5))), 1),
                "IncidentResponseTime": round(max(0.1, np.random.exponential(0.5)), 1),
                "BackupSuccessRate": round(min(100, max(90, np.random.normal(98, 2))), 1),
                "ComplianceScore": round(min(100, max(80, np.random.normal(95, 3))), 1)
            })

        elif department == 'HR':
            # Calculate recruiting effectiveness based on experience
            base_time_to_fill = np.random.normal(30, 8)
            experience_factor = max(0.7, min(1.3, 1 / (1 + years_experience/4)))
            
            employee.update({
                "TimeToFill": round(max(10, base_time_to_fill * experience_factor), 1),
                "CandidatesHired": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "RetentionRate": round(min(100, max(70, np.random.normal(85, 8))), 1),
                "EmployeeSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "TrainingProgramsManaged": int(np.random.normal(6, 2)),
                "PolicyComplianceRate": round(min(100, max(90, np.random.normal(97, 2))), 1),
                "DisputesResolved": int(np.random.exponential(5)),
                "OnboardingEffectivenessScore": round(min(100, max(60, np.random.normal(85, 10))), 1),
                "ExitInterviewsCompleted": int(np.random.normal(5, 2)),
                "BenefitsAdministrationScore": round(min(100, max(70, np.random.normal(90, 5))), 1),
                "RecruitmentCostPerHire": round(np.random.normal(4000, 1000), 2),
                "EmployeeRelationsScore": round(min(10, max(1, np.random.normal(8, 1))), 1)
            })

        # Add learning and development metrics
        certifications = []
        if np.random.random() < 0.7:  # 70% chance of having certifications
            cert_count = employee['CertificationsCount']
            dept_certs = {
                'Engineering': ['AWS Certified Developer', 'CISSP', 'PMP', 'Azure Developer', 'Google Cloud Engineer'],
                'IT': ['CompTIA A+', 'CCNA', 'CISSP', 'AWS SysOps', 'Microsoft Certified'],
                'Sales': ['Certified Sales Professional', 'Salesforce Certified', 'HubSpot Sales', 'Customer Success'],
                'HR': ['PHR', 'SHRM-CP', 'HRCI', 'Talent Management', 'DEI Certification']
            }
            available_certs = dept_certs.get(department, [])
            if available_certs and cert_count > 0:
                certifications = np.random.choice(available_certs, size=min(cert_count, len(available_certs)), replace=False).tolist()
        
        employee["Certifications"] = certifications
        
        # Add collaboration metrics
        employee.update({
            "CrossTeamProjects": int(np.random.normal(3, 1) * years_experience) if years_experience > 0 else 0,
            "PeerReviewsCompleted": int(np.random.normal(10, 3) * years_experience) if years_experience > 0 else 0,
            "MentoringHours": int(np.random.exponential(20)) if years_experience > 2 else 0,
            "TeamCollaborationScore": round(min(10, max(1, np.random.normal(7.5, 1.5))), 1)
        })
        
        return employee

    def generate_dataset(self, num_records=1000):
        """Generate a complete HR dataset"""
        employees = [self._generate_employee() for _ in range(num_records)]
        return pd.DataFrame(employees)
    
    def save_dataset(self, num_records=1000, xlsx_path='hr_data.xlsx', json_path='hr_data.json'):
        """Generate and save dataset in both Excel and JSON formats with auto-adjusted column widths"""
        df = self.generate_dataset(num_records)
        
        # Save the DataFrame to Excel first
        df.to_excel(xlsx_path, index=False)
        
        # Adjust column widths
        workbook = load_workbook(xlsx_path)
        worksheet = workbook.active
        
        for column_cells in worksheet.columns:
            # Get the length of the longest cell in the column (including header)
            max_length = max(len(str(cell.value)) for cell in column_cells)
            column_letter = column_cells[0].column_letter  # Get column letter
            worksheet.column_dimensions[column_letter].width = max_length + 2  # Add padding
        
        # Save the adjusted workbook
        workbook.save(xlsx_path)
        
        # Also save JSON as before
        df.to_json(json_path, orient='records', date_format='iso')
            
        return df


# Create an instance of the HRDataGenerator class
generator = HRDataGenerator()

# Define paths for output files
current_dir = os.getcwd()
xlsx_path = os.path.join(current_dir, 'hr_data.xlsx')
json_path = os.path.join(current_dir, 'hr_data.json')

# Generate and save data
df = generator.save_dataset(num_records=900, xlsx_path=xlsx_path, json_path=json_path)

# Check if files are created and print a message
if os.path.exists(xlsx_path):
    print(f"Excel file created successfully at: {xlsx_path}")
else:
    print("Excel file was not created!")

if os.path.exists(json_path):
    print(f"JSON file created successfully at: {json_path}")
else:
    print("JSON file was not created!")

Excel file created successfully at: c:\Users\govar\OneDrive\Documents\HRM\python\data_processing\hr_data.xlsx
JSON file created successfully at: c:\Users\govar\OneDrive\Documents\HRM\python\data_processing\hr_data.json


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import date, timedelta
from openpyxl import load_workbook
import os

class HRDataGenerator:
    def __init__(self, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        
        # Updated constants
        self.MIN_AGE = 18  # Changed to include interns
        self.MAX_AGE = 65
        self.MIN_YEARS_HISTORY = 10
        self.TODAY = date.today()
        self.BASE_YEAR = self.TODAY.year
        
        self._initialize_lookup_data()
    
    def _initialize_lookup_data(self):
        """Initialize all lookup data with realistic values"""
        self.departments = {
            'Engineering': {
                'roles': ['Intern', 'Junior Software Engineer', 'Software Engineer', 'Senior Software Engineer', 'Tech Lead', 'Engineering Manager'],
                'salary_ranges': {
                    'Intern': (45000, 65000),
                    'Junior Software Engineer': (65000, 85000),
                    'Software Engineer': (85000, 130000),
                    'Senior Software Engineer': (120000, 180000),
                    'Tech Lead': (150000, 200000),
                    'Engineering Manager': (170000, 250000)
                },
                'experience_requirements': {
                    'Intern': (0, 1),
                    'Junior Software Engineer': (0, 2),
                    'Software Engineer': (2, 5),
                    'Senior Software Engineer': (5, 10),
                    'Tech Lead': (8, 15),
                    'Engineering Manager': (10, 20)
                },
                'intern_ratio': 0.15
            },
            'IT': {
                'roles': ['Intern', 'Junior System Administrator', 'System Administrator', 'Senior System Administrator', 'Network Engineer', 'Security Analyst', 'IT Manager'],
                'salary_ranges': {
                    'Intern': (40000, 55000),
                    'Junior System Administrator': (55000, 75000),
                    'System Administrator': (70000, 115000),
                    'Senior System Administrator': (90000, 140000),
                    'Network Engineer': (85000, 145000),
                    'Security Analyst': (95000, 160000),
                    'IT Manager': (130000, 190000)
                },
                'experience_requirements': {
                    'Intern': (0, 1),
                    'Junior System Administrator': (0, 2),
                    'System Administrator': (2, 5),
                    'Senior System Administrator': (5, 10),
                    'Network Engineer': (3, 8),
                    'Security Analyst': (3, 8),
                    'IT Manager': (8, 15)
                },
                'intern_ratio': 0.10
            },
            'Sales': {
                'roles': ['Sales Intern', 'Junior Sales Representative', 'Sales Representative', 'Senior Sales Representative', 'Account Executive', 'Sales Manager'],
                'salary_ranges': {
                    'Sales Intern': (35000, 45000),
                    'Junior Sales Representative': (40000, 60000),
                    'Sales Representative': (50000, 90000),
                    'Senior Sales Representative': (70000, 120000),
                    'Account Executive': (85000, 150000),
                    'Sales Manager': (110000, 180000)
                },
                'experience_requirements': {
                    'Sales Intern': (0, 1),
                    'Junior Sales Representative': (0, 2),
                    'Sales Representative': (1, 4),
                    'Senior Sales Representative': (4, 8),
                    'Account Executive': (5, 10),
                    'Sales Manager': (8, 15)
                },
                'intern_ratio': 0.08
            },
            'HR': {
                'roles': ['HR Intern', 'HR Assistant', 'HR Specialist', 'Senior HR Specialist', 'Recruiter', 'HR Manager'],
                'salary_ranges': {
                    'HR Intern': (35000, 45000),
                    'HR Assistant': (40000, 55000),
                    'HR Specialist': (55000, 85000),
                    'Senior HR Specialist': (75000, 110000),
                    'Recruiter': (60000, 100000),
                    'HR Manager': (95000, 150000)
                },
                'experience_requirements': {
                    'HR Intern': (0, 1),
                    'HR Assistant': (0, 2),
                    'HR Specialist': (2, 5),
                    'Senior HR Specialist': (5, 10),
                    'Recruiter': (2, 8),
                    'HR Manager': (8, 15)
                },
                'intern_ratio': 0.05
            }
        }

        self.promotion_patterns = {
            'Engineering': {
                'Intern': {'avg_time': 0.5, 'promotion_rate': 0.7, 'next_role': 'Junior Software Engineer'},
                'Junior Software Engineer': {'avg_time': 1.5, 'promotion_rate': 0.6, 'next_role': 'Software Engineer'},
                'Software Engineer': {'avg_time': 2.5, 'promotion_rate': 0.5, 'next_role': 'Senior Software Engineer'},
                'Senior Software Engineer': {'avg_time': 3, 'promotion_rate': 0.3, 'next_role': 'Tech Lead'},
                'Tech Lead': {'avg_time': 4, 'promotion_rate': 0.2, 'next_role': 'Engineering Manager'}
            },
            'Sales': {
                'Sales Intern': {'avg_time': 0.5, 'promotion_rate': 0.6, 'next_role': 'Junior Sales Representative'},
                'Junior Sales Representative': {'avg_time': 1, 'promotion_rate': 0.7, 'next_role': 'Sales Representative'},
                'Sales Representative': {'avg_time': 2, 'promotion_rate': 0.5, 'next_role': 'Senior Sales Representative'},
                'Senior Sales Representative': {'avg_time': 2.5, 'promotion_rate': 0.4, 'next_role': 'Account Executive'},
                'Account Executive': {'avg_time': 3, 'promotion_rate': 0.3, 'next_role': 'Sales Manager'}
            },
            'IT': {
                'Intern': {'avg_time': 0.5, 'promotion_rate': 0.65, 'next_role': 'Junior System Administrator'},
                'Junior System Administrator': {'avg_time': 1, 'promotion_rate': 0.6, 'next_role': 'System Administrator'},
                'System Administrator': {'avg_time': 2, 'promotion_rate': 0.5, 'next_role': 'Senior System Administrator'},
                'Senior System Administrator': {'avg_time': 3, 'promotion_rate': 0.35, 'next_role': 'Network Engineer'},
                'Network Engineer': {'avg_time': 3.5, 'promotion_rate': 0.3, 'next_role': 'Security Analyst'},
                'Security Analyst': {'avg_time': 4, 'promotion_rate': 0.25, 'next_role': 'IT Manager'}
            },
            'HR': {
                'HR Intern': {'avg_time': 0.5, 'promotion_rate': 0.7, 'next_role': 'HR Assistant'},
                'HR Assistant': {'avg_time': 1, 'promotion_rate': 0.65, 'next_role': 'HR Specialist'},
                'HR Specialist': {'avg_time': 2, 'promotion_rate': 0.5, 'next_role': 'Senior HR Specialist'},
                'Senior HR Specialist': {'avg_time': 2.5, 'promotion_rate': 0.4, 'next_role': 'Recruiter'},
                'Recruiter': {'avg_time': 3, 'promotion_rate': 0.35, 'next_role': 'HR Manager'},
                'HR Manager': {'avg_time': 4, 'promotion_rate': 0.2, 'next_role': 'Senior HR Manager'}
            }
        }
        
        # Add previous companies distribution
        self.prev_companies_probs = {
            '0': 0.15,  # First job
            '1-2': 0.35,  # Most common
            '3-4': 0.30,
            '5-6': 0.15,
            '7+': 0.05   # Rare but possible
        }
        
        # Add project completion base metrics
        self.project_completion_mean = 0.85  # 85% average completion rate
        self.project_completion_std = 0.12   # Standard deviation for normal distribution
        
       # Updated performance ratings to ensure consistency
        self.performance_ratings = {
            'Outstanding': 0.05,
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Needs Improvement': 0.20,
            'Performance Plan': 0.10
        }
        
        self.recruitment_sources = {
            'LinkedIn': 0.25,
            'Employee Referral': 0.20,
            'Company Website': 0.15,
            'Indeed': 0.12,
            'University Partnership': 0.10,
            'Recruitment Agency': 0.08,
            'Diversity Job Fair': 0.05,
            'Stack Overflow': 0.03,
            'GitHub': 0.02
        }
        
        self.termination_reasons = {
            'N/A-StillEmployed': 0.82,
            'Voluntary-Better Opportunity': 0.05,
            'Voluntary-Career Change': 0.02,
            'Voluntary-Relocation': 0.02,
            'Voluntary-Return to School': 0.01,
            'Voluntary-Retirement': 0.02,
            'Involuntary-Performance': 0.02,
            'Involuntary-Attendance': 0.01,
            'Involuntary-Policy Violation': 0.01,
            'Involuntary-Layoff': 0.02
        }
        
        self.education_levels = {
            'High School': 0.10,
            'Some College': 0.15,
            'Associates Degree': 0.10,
            'Bachelors Degree': 0.45,
            'Masters Degree': 0.18,
            'PhD': 0.02
        }
        
        self.remote_status = {
            'On-site': 0.40,
            'Hybrid-2 Days': 0.20,
            'Hybrid-3 Days': 0.25,
            'Full Remote': 0.15
        }
        
        self.ethnicities = {
            'White': 0.60,
            'Asian': 0.15,
            'Hispanic or Latino': 0.12,
            'Black or African American': 0.08,
            'Two or More Races': 0.03,
            'Native American': 0.01,
            'Pacific Islander': 0.01
        }

        self.engagement_modifiers = {
                'salary_satisfaction': (-0.5, 0.5),
                'years_in_role': (-0.2, -0.1),
                'performance': {
                    'Outstanding': 0.8,
                    'Exceeds Expectations': 0.5,
                    'Meets Expectations': 0.0,
                    'Needs Improvement': -0.5,
                    'Performance Plan': -0.8
                }
        }
        
        self.base_performance_ratings = {
            'Outstanding': 0.05,
            'Exceeds Expectations': 0.15,
            'Meets Expectations': 0.50,
            'Needs Improvement': 0.20,
            'Performance Plan': 0.10
        }
        
        self.dept_performance_biases = {
            'Engineering': {
                'Outstanding': 1.2,
                'Exceeds Expectations': 1.1
            },
            'Sales': {
                'Outstanding': 1.3,
                'Inconsistent': 1.2
            }
        }

        self.project_involvement_levels = {
            'Critical': 0.2,
            'High': 0.3,
            'Medium': 0.35,
            'Low': 0.15
        }

        self.feedback_categories = {
            'Technical Skills': {'weight': 0.3},
            'Communication': {'weight': 0.2},
            'Leadership': {'weight': 0.15},
            'Innovation': {'weight': 0.15},
            'Teamwork': {'weight': 0.2}
        }

        # Add this to existing engagement_modifiers
        self.engagement_modifiers.update({
            'promotion_recency': {'recent': 0.5, 'overdue': -0.3},
            'project_involvement': {
                'Critical': 0.4,
                'High': 0.2,
                'Medium': 0,
                'Low': -0.2
            },
            'feedback_score': (-0.4, 0.4)
        })

    def _generate_promotion_history(self, department, role, years_experience, performance):
        """Generate realistic promotion history"""
        promotions = []
        current_role = role
        time_in_company = years_experience
        
        while time_in_company > 0 and current_role in self.promotion_patterns.get(department, {}):
            pattern = self.promotion_patterns[department][current_role]
            prev_role = pattern.get('next_role')
            
            if prev_role:
                # Calculate promotion timing with some randomness
                avg_time = pattern['avg_time']
                actual_time = max(0.5, np.random.normal(avg_time, avg_time * 0.2))
                
                # Adjust timing based on performance
                if performance in ['Outstanding', 'Exceeds Expectations']:
                    actual_time *= 0.8
                
                promotion_date = self.TODAY - timedelta(days=int(time_in_company * 365))
                
                promotions.append({
                    'from_role': prev_role,
                    'to_role': current_role,
                    'date': promotion_date.strftime('%Y-%m-%d'),
                    'time_in_previous_role': round(actual_time, 1)
                })
                
                time_in_company -= actual_time
                current_role = prev_role
            else:
                break
                
        return list(reversed(promotions))  # Return in chronological order
    
    def _generate_project_metrics(self):
        """Generate project-related metrics including active projects and complexity."""
        active_projects = self._calculate_active_projects(self.current_level, self.current_department)
        
        # Generate project details
        projects = []
        for _ in range(active_projects):
            project = {
                'complexity': self._calculate_project_complexity(self.current_level, False),
                'status': random.choice(['Planning', 'In Progress', 'Final Stage'])
            }
            projects.append(project)
        
        return {
            'active_projects': active_projects,
            'avg_project_complexity': round(np.mean([p['complexity'] for p in projects]), 1),
            'project_satisfaction': self._generate_client_satisfaction(
                self.current_level, 
                self.current_department, 
                projects,
                self._generate_performance_score(self.current_level, False)
            )
        }

    def _generate_performance_metrics(self):
        """Generate performance-related metrics."""
        is_outlier = random.random() < 0.05
        performance_score = self._generate_performance_score(self.current_level, is_outlier)
        
        return {
            'performance_score': performance_score,
            'delivery_quality': self._generate_delivery_quality(self.current_level, is_outlier),
            'internal_contribution_count': self._generate_internal_contribution(self.current_level),
            'innovation_score': self._calculate_innovation_score(self.current_level, self.current_department)
        }

    def _generate_development_metrics(self):
        """Generate development and growth related metrics."""
        is_outlier = random.random() < 0.05
        return {
            'training_hours': self._calculate_training_hours(self.current_level, is_outlier),
            'mentorship_hours': self._calculate_mentorship_hours(self.current_level),
            'knowledge_sharing': self._calculate_knowledge_sharing(self.current_level, is_outlier),
            'promotion_readiness': self._calculate_promotion_readiness(
                self.current_level, 
                self.hire_date, 
                is_outlier
            )
        }

    def _generate_risk_metrics(self):
        """Generate risk and engagement related metrics."""
        is_outlier = random.random() < 0.05
        return {
            'flight_risk': self._calculate_flight_risk(is_outlier),
            'engagement_score': self._generate_engagement_score(is_outlier)
        }

    def _generate_engagement_score(self, is_outlier):
        """Generate employee engagement score."""
        if is_outlier:
            return random.choice([4.0, 9.5])
        
        base_score = np.random.normal(7.5, 1.0)
        return round(min(max(base_score, 1), 10), 1)

    def _generate_date(self, start_year_offset=-10, end_year_offset=0):
        """
        Generate a random date within specified year offsets from today.
        
        Parameters:
        start_year_offset (int): Number of years before today to start range (negative number)
        end_year_offset (int): Number of years before/after today to end range
        
        Returns:
        date: A random date within the specified range
        """
        start_date = self.TODAY + timedelta(days=365 * start_year_offset)
        end_date = self.TODAY + timedelta(days=365 * end_year_offset)
        
        # Generate random date between start and end
        days_between = (end_date - start_date).days
        random_days = np.random.randint(0, max(1, days_between))
        return start_date + timedelta(days=random_days)

    def _calculate_age(self, birth_date):
        """Calculate age based on birth date"""
        today = self.TODAY
        age = today.year - birth_date.year
        # Adjust age if birthday hasn't occurred this year
        if today.month < birth_date.month or (today.month == birth_date.month and today.day < birth_date.day):
            age -= 1
        return age
        
    def _generate_work_satisfaction_metrics(self, years_experience, performance, role):
        """Generate realistic work satisfaction metrics based on various factors"""
        base_satisfaction = np.random.normal(7, 2)  # Base satisfaction out of 10
        
        # Adjust based on performance
        performance_adjustments = {
            'Outstanding': 2,
            'Exceeds Expectations': 1,
            'Meets Expectations': 0,
            'Needs Improvement': -1.5,
            'Performance Plan': -2.5
        }
        
        # Adjust based on role level (more senior roles tend to be more satisfied)
        is_senior = any(senior_term in role.lower() for senior_term in ['senior', 'lead', 'manager'])
        role_adjustment = 1 if is_senior else 0
        
        # Adjust based on years of experience (slight burnout factor)
        experience_adjustment = -0.5 if years_experience > 5 else 0
        
        # Calculate final satisfaction score
        satisfaction = base_satisfaction + performance_adjustments[performance] + role_adjustment + experience_adjustment
        satisfaction = max(1, min(10, satisfaction))  # Clamp between 1 and 10
        
        return round(satisfaction, 1)

    def _generate_productivity_metrics(self, performance, years_experience):
        """Generate realistic productivity metrics"""
        base_productivity = np.random.normal(80, 10)  # Base productivity percentage
        
        # Adjust based on performance
        performance_adjustments = {
            'Outstanding': 15,
            'Exceeds Expectations': 10,
            'Meets Expectations': 0,
            'Needs Improvement': -10,
            'Performance Plan': -15
        }
        
        # Experience adjustment (diminishing returns after 5 years)
        experience_adjustment = min(years_experience * 2, 10)
        
        productivity = base_productivity + performance_adjustments[performance] + experience_adjustment
        productivity = max(40, min(120, productivity))  # Clamp between 40% and 120%
        
        return round(productivity, 1)

    def _get_performance_distribution(self, department, years_experience, role):
        """Get performance distribution adjusted for department and experience"""
        base_dist = self.base_performance_ratings.copy()
        
        # Apply department-specific biases
        if department in self.dept_performance_biases:
            for rating, bias in self.dept_performance_biases[department].items():
                base_dist[rating] *= bias
        
        # Experience adjustments
        if years_experience < 1:
            base_dist['Exceeds Expectations'] *= 0.5
            base_dist['Meets+ Expectations'] *= 0.7
            base_dist['Meets Expectations'] *= 1.3
        elif years_experience > 5:
            base_dist['Exceeds Expectations'] *= 1.2
            base_dist['Needs Improvement'] *= 0.8
        
        # Role-specific adjustments
        if 'Senior' in role or 'Lead' in role:
            base_dist['Exceeds Expectations'] *= 1.3
            base_dist['Performance Plan'] *= 0.7
        elif 'Intern' in role:
            base_dist['Exceeds Expectations'] *= 0.6
            base_dist['Meets Expectations'] *= 1.2
        
        # Normalize probabilities
        total = sum(base_dist.values())
        return {k: v/total for k, v in base_dist.items()}

    def _generate_experience_based_role(self, department):
        """Generate role based on experience distribution with realistic ratios"""
        dept_roles = self.departments[department]['roles']
        experience_reqs = self.departments[department]['experience_requirements']
        
        # Weight roles to favor more junior positions and maintain realistic ratios
        weights = []
        for role in dept_roles:
            if 'Intern' in role:
                weights.append(0.08)  # 8% interns
            elif 'Junior' in role or 'Coordinator' in role:
                weights.append(0.25)  # 25% junior positions
            elif 'Senior' in role or 'Lead' in role:
                weights.append(0.17)  # 17% senior positions
            else:
                weights.append(0.50)  # 50% mid-level positions
        
        # Normalize weights
        weights = np.array(weights) / sum(weights)
        return np.random.choice(dept_roles, p=weights)

    def _calculate_prev_companies(self, role, years_experience, age):
        """
        Calculate previous companies based on role, experience, and age.
        Returns a string indicating the range of previous companies.
        """
        if 'Intern' in role:
            # Interns typically have 0-1 previous companies (likely internships)
            return np.random.choice(['0', '1-2'], p=[0.8, 0.2])
        
        # Calculate maximum possible companies based on work years
        # Assume minimum 1 year at each company on average
        max_possible_work_years = age - 18  # Starting from age 18
        realistic_work_years = min(max_possible_work_years, years_experience + 2)  # Add buffer for gaps
        
        if realistic_work_years < 1:
            return '0'
        elif realistic_work_years < 3:
            return np.random.choice(['0', '1-2'], p=[0.4, 0.6])
        elif realistic_work_years < 5:
            return np.random.choice(['0', '1-2', '3-4'], p=[0.2, 0.5, 0.3])
        elif realistic_work_years < 8:
            return np.random.choice(['1-2', '3-4', '5-6'], p=[0.3, 0.5, 0.2])
        elif realistic_work_years < 12:
            return np.random.choice(['3-4', '5-6', '7+'], p=[0.4, 0.4, 0.2])
        else:
            return np.random.choice(['3-4', '5-6', '7+'], p=[0.3, 0.4, 0.3])

    def _generate_experience_years(self, role, department, age):
        """
        Generate years of experience based on role, department, and age.
        Returns a float representing years of experience.
        """
        min_exp, max_exp = self.departments[department]['experience_requirements'][role]
        
        if 'Intern' in role:
            # Interns typically have 0-1 years of experience
            return round(np.random.uniform(0, 1), 1)
        
        # Calculate maximum possible years of experience based on age
        max_possible_exp = age - 18  # Assuming work starts at 18
        
        # Adjust experience based on role seniority
        if 'Senior' in role or 'Lead' in role or 'Manager' in role:
            # Senior roles should have more experience
            base_exp = np.random.uniform(min_exp + 2, min(max_possible_exp, max_exp + 5))
        elif 'Junior' in role:
            # Junior roles should have less experience
            base_exp = np.random.uniform(min_exp, min(max_possible_exp, min_exp + 3))
        else:
            # Mid-level roles
            base_exp = np.random.uniform(min_exp, min(max_possible_exp, max_exp))
        
        # Add some random variation but ensure it stays realistic
        variation = np.random.normal(0, 0.5)
        experience = max(0, min(max_possible_exp, base_exp + variation))
        
        return round(experience, 1)

    def _generate_hire_date(self, role, department, years_experience):
        """
        Generate hire date based on role, department, and years of experience.
        Returns a date object.
        """
        if 'Intern' in role:
            # Interns typically start in summer or winter
            month = np.random.choice([6, 7, 1])  # June, July, or January
            year = self.BASE_YEAR
            day = np.random.randint(1, 28)
            return date(year, month, day)
        
        # Calculate hire date based on years of experience at the company
        max_company_experience = min(years_experience, 15)  # Cap at 15 years for realism
        
        # Add some randomness to company experience
        company_experience = np.random.uniform(0, max_company_experience)
        
        # Calculate days ago
        days_ago = int(company_experience * 365.25)
        hire_date = self.TODAY - timedelta(days=days_ago)
        
        # Adjust for common hiring months (more hiring in Q1 and Q3)
        month_weights = [1.2, 1.1, 1.2, 0.9, 0.8, 1.0, 1.1, 1.1, 1.2, 0.9, 0.8, 0.7]
        if np.random.random() < 0.7:  # 70% chance to adjust to preferred hiring months
            hire_date = hire_date.replace(month=np.random.choice(
                range(1, 13),
                p=np.array(month_weights) / sum(month_weights)
            ))
        
        return hire_date

    def _generate_age(self, role, department, hire_date):
        """Generate age based on role and realistic distributions"""
        min_exp, _ = self.departments[department]['experience_requirements'][role]
        years_since_hire = (self.TODAY - hire_date).days / 365.25
        
        if 'Intern' in role:
            return np.random.randint(18, 25)
        elif 'Junior' in role or 'Coordinator' in role:
            base_age = np.random.normal(25, 3)
        elif min_exp <= 2:
            base_age = np.random.normal(28, 4)
        elif min_exp <= 5:
            base_age = np.random.normal(32, 5)
        else:
            base_age = np.random.normal(38, 7)
        
        # Add some outliers
        if np.random.random() < 0.05:  # 5% chance of being an outlier
            if np.random.random() < 0.5:
                base_age += np.random.normal(10, 3)  # Older outlier
            else:
                base_age -= np.random.normal(5, 2)   # Younger outlier
        
        # Ensure age makes sense with hire date and experience
        min_possible_age = 18 + years_since_hire
        max_possible_age = 65
        
        age = max(min_possible_age, min(max_possible_age, base_age))
        return int(round(age))

    def _calculate_salary(self, role, department, performance, years_experience):
        """Calculate salary with realistic variations and market factors"""
        base_range = self.departments[department]['salary_ranges'][role]
        min_exp, max_exp = self.departments[department]['experience_requirements'][role]
        
        # Calculate base salary within range with slight random variation
        base = np.random.normal(
            (base_range[0] + base_range[1]) / 2,
            (base_range[1] - base_range[0]) / 6
        )
        
        # Ensure within range
        base = max(base_range[0], min(base_range[1], base))
        
        # Updated performance multipliers
        performance_multipliers = {
            'Outstanding': np.random.uniform(1.15, 1.20),
            'Exceeds Expectations': np.random.uniform(1.08, 1.14),
            'Meets Expectations': np.random.uniform(0.98, 1.02),
            'Needs Improvement': np.random.uniform(0.92, 0.97),
            'Performance Plan': np.random.uniform(0.85, 0.90)
        }

        exp_multiplier = 1 + (np.log1p(min(years_experience, max_exp)) * np.random.uniform(0.02, 0.04))
        
        market_adjustments = {
            'Engineering': np.random.uniform(1.05, 1.15),
            'IT': np.random.uniform(1.02, 1.10),
            'Sales': np.random.uniform(0.95, 1.05),
            'HR': np.random.uniform(0.98, 1.02)
        }
        
        # Role-specific adjustments
        role_adjustment = 1.0
        if 'Senior' in role:
            role_adjustment *= np.random.uniform(1.05, 1.15)
        elif 'Lead' in role:
            role_adjustment *= np.random.uniform(1.10, 1.20)
        elif 'Intern' in role:
            role_adjustment *= np.random.uniform(0.85, 0.95)
        
        # Calculate final salary with all factors
        final_salary = (base 
                    * performance_multipliers[performance]
                    * exp_multiplier
                    * market_adjustments[department]
                    * role_adjustment)
        
        # Add some random noise (±2%)
        noise = np.random.uniform(0.98, 1.02)
        final_salary *= noise
        
        return round(final_salary, 2)

    def _calculate_engagement_score(self, salary, years_experience, performance, role, termination_reason):
        """Calculate employee engagement score based on multiple factors"""
        base_score = np.random.normal(3.5, 0.5)
        
        # Salary satisfaction impact
        salary_mod = np.random.uniform(*self.engagement_modifiers['salary_satisfaction'])
        base_score += salary_mod
        
        # Years in role impact (stagnation effect)
        if years_experience > 2 and 'Senior' not in role and 'Lead' not in role:
            stagnation = np.random.uniform(*self.engagement_modifiers['years_in_role'])
            base_score += stagnation * (years_experience - 2)
        
        # Performance impact
        base_score += self.engagement_modifiers['performance'][performance]
        
        # Role-specific adjustments
        if 'Intern' in role:
            base_score += np.random.uniform(0.2, 0.5)  # Interns tend to be more engaged
        elif 'Senior' in role or 'Lead' in role:
            base_score += np.random.uniform(-0.2, 0.4)  # More variable engagement
        
        # Termination impact
        if termination_reason != 'N/A-StillEmployed':
            if 'Voluntary' in termination_reason:
                base_score -= np.random.uniform(0.5, 1.5)
            elif 'Involuntary' in termination_reason:
                base_score -= np.random.uniform(1.0, 2.0)
        
        # Ensure score is within realistic bounds (1-5)
        return round(max(1.0, min(5.0, base_score)), 2)

    def _generate_absence_days(self, performance, years_experience, role):
        """Generate realistic absence days based on multiple factors"""
        if 'Intern' in role:
            # Interns have fewer absence days due to shorter tenure
            base_days = np.random.negative_binomial(2, 0.5)
        else:
            # Base absence days following negative binomial distribution
            base_days = np.random.negative_binomial(5, 0.7)
        
        # Modify based on performance
        performance_multipliers = {
            'Exceeds Expectations': 0.7,
            'Meets+ Expectations': 0.85,
            'Meets Expectations': 1.0,
            'Needs Improvement': 1.3,
            'Performance Plan': 1.5
        }
        
        # Modify based on experience (slightly more absence with more experience)
        exp_modifier = 1 + (years_experience * 0.02)
        
        # Calculate final absence days
        absence_days = int(base_days * performance_multipliers[performance] * exp_modifier)
        
        # Add occasional outliers
        if np.random.random() < 0.05:  # 5% chance of unusual absence patterns
            absence_days += np.random.randint(10, 30)
        
        return absence_days

    def _generate_employee(self):
        """Generate a single employee record with enhanced realism"""
        # Select department and role with intern consideration
        department = np.random.choice(list(self.departments.keys()))
        
        # Determine if this will be an intern based on department's intern ratio
        is_intern = np.random.random() < self.departments[department]['intern_ratio']
        
        # Filter roles based on intern status
        available_roles = [role for role in self.departments[department]['roles'] 
                         if ('intern' in role.lower()) == is_intern]
        role = np.random.choice(available_roles)
        
        # Generate age first
        if 'Intern' in role:
            age = np.random.randint(18, 25)
        else:
            age = self._generate_age(role, department, None)  # Pass None for hire_date initially
        
        # Generate total years of experience based on role and age
        years_experience = self._generate_experience_years(role, department, age)
        
        # Generate hire date based on role and experience
        hire_date = self._generate_hire_date(role, department, years_experience)
        
        # Calculate previous companies based on role, experience, and age
        previous_companies = self._calculate_prev_companies(role, years_experience, age)
        
        # Calculate birth date from age
        birth_date = self.TODAY - timedelta(days=int(age * 365.25))
        
        # Generate performance rating (interns more likely to be "Meets Expectations")
        if is_intern:
            performance_probs = {'Meets Expectations': 0.7, 'Exceeds Expectations': 0.15, 
                               'Needs Improvement': 0.15}
            performance = np.random.choice(
                list(performance_probs.keys()),
                p=list(performance_probs.values())
            )
        else:
            performance = np.random.choice(
                list(self.performance_ratings.keys()),
                p=list(self.performance_ratings.values())
            )
        
        # Determine employment status and termination details
        term_reason = np.random.choice(
            list(self.termination_reasons.keys()),
            p=list(self.termination_reasons.values())
        )
        
        term_date = None
        if term_reason != 'N/A-StillEmployed':
            term_date = self._generate_date(
                start_year_offset=max(-3, int(-years_experience)),
                end_year_offset=0
            )
        
        # Generate last review date
        max_review_date = term_date if term_date else self.TODAY
        last_review_date = self._generate_date(
            start_year_offset=-1,
            end_year_offset=0
        )
        if last_review_date > max_review_date:
            last_review_date = max_review_date
        
        # Generate education level appropriate for role
        if 'intern' in role.lower():
            education_options = ['Some College', 'Bachelors Degree']
            education_probs = [0.3, 0.7]
        else:
            education_options = list(self.education_levels.keys())
            education_probs = list(self.education_levels.values())
        
        education = np.random.choice(education_options, p=education_probs)
        
        # Calculate work satisfaction and productivity
        satisfaction = self._generate_work_satisfaction_metrics(years_experience, performance, role)
        productivity = self._generate_productivity_metrics(performance, years_experience)
        
        # Generate base employee record
        employee = {
            "EmployeeID": self.fake.unique.random_int(min=10000, max=99999),
            "Name": self.fake.name(),
            "Department": department,
            "Position": role,
            "HireDate": hire_date.strftime('%Y-%m-%d'),
            "TerminationDate": term_date.strftime('%Y-%m-%d') if term_date else None,
            "TerminationReason": term_reason,
            "EmploymentStatus": "Terminated" if term_date else "Active",
            "DateOfBirth": birth_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Gender": np.random.choice(['M', 'F'], p=[0.51, 0.49]),
            "Ethnicity": np.random.choice(
                list(self.ethnicities.keys()),
                p=list(self.ethnicities.values())
            ),
            "Salary": self._calculate_salary(role, department, performance, years_experience),
            "PerformanceRating": performance,
            "RecruitmentSource": np.random.choice(
                list(self.recruitment_sources.keys()),
                p=list(self.recruitment_sources.values())
            ),
            "EducationLevel": education,
            "YearsExperience": round(years_experience, 1),
            "PreviousCompanies": np.random.choice(
                list(self.prev_companies_probs.keys()),
                p=list(self.prev_companies_probs.values())
            ),
            "RemoteWorkStatus": np.random.choice(
                list(self.remote_status.keys()),
                p=list(self.remote_status.values())
            ),
            "ProjectCompletionRate": min(100, max(0, np.random.normal(
                self.project_completion_mean * 100,
                self.project_completion_std * 100
            ))),
            "WorkSatisfaction": satisfaction,
            "Productivity": productivity,
            "EngagementScore": round(np.random.normal(3.5, 0.5), 2),
            "AbsenceDays": int(np.random.negative_binomial(5, 0.7)),
            "LastReviewDate": last_review_date.strftime('%Y-%m-%d'),
            "OvertimeHours": int(np.random.exponential(10)) if not is_intern else int(np.random.exponential(3)),
            "TrainingHoursCompleted": int(np.random.normal(40, 15)) if not is_intern else int(np.random.normal(60, 10)),
            "TeamSize": int(np.random.normal(8, 3)),
            "ProjectsCompleted": int(np.random.normal(12, 4) * years_experience) if years_experience > 0 else 0,
            "CertificationsCount": int(np.random.exponential(2)),
        }
        
        # Add department-specific metrics
        if department == 'Sales':
            employee.update({
                "QuotaAttainment": round(min(150, max(50, np.random.normal(100, 20))), 1),
                "LeadConversionRate": round(min(100, max(0, np.random.normal(30, 10))), 1),
                "ClientRetentionRate": round(min(100, max(60, np.random.normal(85, 8))), 1),
                "DealsClosed": int(np.random.normal(15, 5) * years_experience) if years_experience > 0 else 0,
                "AverageContractValue": round(np.random.normal(50000, 20000), 2),
                "ClientSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "SalesTargetPercent": round(min(200, max(0, np.random.normal(95, 25))), 1),
                "ProspectingHours": int(np.random.normal(15, 5)),
                "RepeatBusinessPercent": round(min(100, max(0, np.random.normal(65, 15))), 1)
            })

        elif department == 'Engineering':
            # Calculate bug metrics based on experience and performance
            base_bug_rate = np.random.normal(5, 2)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/5)))  # More experience = fewer bugs
            performance_factors = {
                'Outstanding': 0.6,
                'Exceeds Expectations': 0.8,
                'Meets Expectations': 1.0,
                'Inconsistent': 1.3,
                'Needs Improvement': 1.5,
                'Performance Plan': 1.8
            }
            
            # Calculate code metrics
            code_review_scores = []
            for _ in range(3):  # Last 3 code reviews
                base_score = np.random.normal(8, 1)
                exp_modifier = min(2, years_experience/3)  # Experience improves code quality
                score = min(10, max(1, base_score + exp_modifier))
                code_review_scores.append(round(score, 1))
            
            employee.update({
                "CodeQualityScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "BugsPerProject": round(max(0, base_bug_rate * experience_factor * performance_factors[performance]), 1),
                "CodeReviewScores": code_review_scores,
                "CommitsPerWeek": int(np.random.normal(12, 4)),
                "PullRequestsOpened": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "PullRequestsAccepted": int(np.random.normal(7, 3) * years_experience) if years_experience > 0 else 0,
                "TechnicalDebtScore": round(min(10, max(1, np.random.normal(4, 2))), 1),
                "DocumentationContribution": round(min(100, max(0, np.random.normal(70, 20))), 1),
                "SystemUptime": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "OnCallIncidents": int(np.random.exponential(3)),
                "DeploymentFrequency": int(np.random.normal(5, 2)),
                "CodeCoverage": round(min(100, max(60, np.random.normal(85, 8))), 1)
            })

        elif department == 'IT':
            # Calculate incident resolution metrics based on experience
            base_resolution_time = np.random.normal(4, 1)
            experience_factor = max(0.5, min(2, 1 / (1 + years_experience/3)))
            
            employee.update({
                "TicketsResolved": int(np.random.normal(120, 30) * years_experience) if years_experience > 0 else 0,
                "AverageResolutionTime": round(max(0.5, base_resolution_time * experience_factor), 1),
                "FirstCallResolutionRate": round(min(100, max(50, np.random.normal(75, 10))), 1),
                "SystemUptimeManaged": round(min(100, max(90, np.random.normal(99.5, 0.3))), 2),
                "SecurityIncidentsHandled": int(np.random.exponential(5)),
                "CustomerSatisfactionScore": round(min(10, max(1, np.random.normal(8, 1))), 1),
                "SystemsManaged": int(np.random.normal(15, 5)),
                "PreventiveMaintenance": round(min(100, max(0, np.random.normal(80, 15))), 1),
                "NetworkPerformanceScore": round(min(100, max(70, np.random.normal(92, 5))), 1),
                "IncidentResponseTime": round(max(0.1, np.random.exponential(0.5)), 1),
                "BackupSuccessRate": round(min(100, max(90, np.random.normal(98, 2))), 1),
                "ComplianceScore": round(min(100, max(80, np.random.normal(95, 3))), 1)
            })

        elif department == 'HR':
            # Calculate recruiting effectiveness based on experience
            base_time_to_fill = np.random.normal(30, 8)
            experience_factor = max(0.7, min(1.3, 1 / (1 + years_experience/4)))
            
            employee.update({
                "TimeToFill": round(max(10, base_time_to_fill * experience_factor), 1),
                "CandidatesHired": int(np.random.normal(8, 3) * years_experience) if years_experience > 0 else 0,
                "RetentionRate": round(min(100, max(70, np.random.normal(85, 8))), 1),
                "EmployeeSatisfactionScore": round(min(10, max(1, np.random.normal(7.5, 1.2))), 1),
                "TrainingProgramsManaged": int(np.random.normal(6, 2)),
                "PolicyComplianceRate": round(min(100, max(90, np.random.normal(97, 2))), 1),
                "DisputesResolved": int(np.random.exponential(5)),
                "OnboardingEffectivenessScore": round(min(100, max(60, np.random.normal(85, 10))), 1),
                "ExitInterviewsCompleted": int(np.random.normal(5, 2)),
                "BenefitsAdministrationScore": round(min(100, max(70, np.random.normal(90, 5))), 1),
                "RecruitmentCostPerHire": round(np.random.normal(4000, 1000), 2),
                "EmployeeRelationsScore": round(min(10, max(1, np.random.normal(8, 1))), 1)
            })

        # Add learning and development metrics
        certifications = []
        if np.random.random() < 0.7:  # 70% chance of having certifications
            cert_count = employee['CertificationsCount']
            dept_certs = {
                'Engineering': ['AWS Certified Developer', 'CISSP', 'PMP', 'Azure Developer', 'Google Cloud Engineer'],
                'IT': ['CompTIA A+', 'CCNA', 'CISSP', 'AWS SysOps', 'Microsoft Certified'],
                'Sales': ['Certified Sales Professional', 'Salesforce Certified', 'HubSpot Sales', 'Customer Success'],
                'HR': ['PHR', 'SHRM-CP', 'HRCI', 'Talent Management', 'DEI Certification']
            }
            available_certs = dept_certs.get(department, [])
            if available_certs and cert_count > 0:
                certifications = np.random.choice(available_certs, size=min(cert_count, len(available_certs)), replace=False).tolist()
        
        employee["Certifications"] = certifications
        
        # Add collaboration metrics
        employee.update({
            "CrossTeamProjects": int(np.random.normal(3, 1) * years_experience) if years_experience > 0 else 0,
            "PeerReviewsCompleted": int(np.random.normal(10, 3) * years_experience) if years_experience > 0 else 0,
            "MentoringHours": int(np.random.exponential(20)) if years_experience > 2 else 0,
            "TeamCollaborationScore": round(min(10, max(1, np.random.normal(7.5, 1.5))), 1)
        })
        
        employee = self()._generate_employee()  # Generate base employee data
        
        # Add promotion history
        employee['promotion_history'] = self._generate_promotion_history(
            employee['Department'],
            employee['Position'],
            employee['YearsExperience'],
            employee['PerformanceRating']
        )
        
        # Add feedback scores
        employee['feedback_scores'] = self._generate_feedback_scores(
            employee['PerformanceRating'],
            employee['YearsExperience'],
            employee['Position']
        )
        
        # Add project metrics
        employee['project_metrics'] = self._generate_project_metrics(
            employee['Department'],
            employee['Position'],
            employee['YearsExperience'],
            employee['PerformanceRating']
        )
        
        # Calculate additional metrics
        employee.update({
            'TimeInCurrentRole': round(max(0.1, employee['YearsExperience'] - 
                sum(p['time_in_previous_role'] for p in employee['promotion_history'])), 1),
            'PromotionReadiness': round(min(100, max(0, np.random.normal(
                70 + (10 if employee['PerformanceRating'] in ['Outstanding', 'Exceeds Expectations'] else 0),
                15))), 1),
            'AverageFeedbackScore': round(np.mean([
                score * self.feedback_categories[category]['weight']
                for category, score in employee['feedback_scores'].items()
            ]), 2),
            'ProjectSuccessRate': round(np.mean([
                project['success_score'] for project in employee['project_metrics']
            ]) if employee['project_metrics'] else 0, 1)
        })
        
        return employee

    def generate_dataset(self, num_records=1000):
        """Generate a complete HR dataset"""
        employees = [self._generate_employee() for _ in range(num_records)]
        return pd.DataFrame(employees)
    
    def save_dataset(self, num_records=1000, xlsx_path='hr_data.xlsx', json_path='hr_data.json'):
        """Generate and save dataset in both Excel and JSON formats with auto-adjusted column widths"""
        df = self.generate_dataset(num_records)
        
        # Save the DataFrame to Excel first
        df.to_excel(xlsx_path, index=False)
        
        # Adjust column widths
        workbook = load_workbook(xlsx_path)
        worksheet = workbook.active
        
        for column_cells in worksheet.columns:
            # Get the length of the longest cell in the column (including header)
            max_length = max(len(str(cell.value)) for cell in column_cells)
            column_letter = column_cells[0].column_letter  # Get column letter
            worksheet.column_dimensions[column_letter].width = max_length + 2  # Add padding
        
        # Save the adjusted workbook
        workbook.save(xlsx_path)
        
        # Also save JSON as before
        df.to_json(json_path, orient='records', date_format='iso')
            
        return df


# Create an instance of the HRDataGenerator class
generator = HRDataGenerator()

# Define paths for output files
current_dir = os.getcwd()
xlsx_path = os.path.join(current_dir, 'hr_data.xlsx')
json_path = os.path.join(current_dir, 'hr_data.json')

# Generate and save data
df = generator.save_dataset(num_records=900, xlsx_path=xlsx_path, json_path=json_path)

# Check if files are created and print a message
if os.path.exists(xlsx_path):
    print(f"Excel file created successfully at: {xlsx_path}")
else:
    print("Excel file was not created!")

if os.path.exists(json_path):
    print(f"JSON file created successfully at: {json_path}")
else:
    print("JSON file was not created!")

TypeError: unsupported operand type(s) for -: 'datetime.date' and 'NoneType'

### V1

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta, date
import json

class HRDataGenerator:
    def __init__(self, seed=42):
        self.fake = Faker()
        Faker.seed(seed)
        np.random.seed(seed)
        
        # Constants that affect data generation
        self.MIN_AGE = 22
        self.MAX_AGE = 65
        self.MIN_YEARS_HISTORY = 10
        self.TODAY = date.today()
        self.BASE_YEAR = self.TODAY.year
        
        self._initialize_lookup_data()
    
    def _initialize_lookup_data(self):
        """Initialize all lookup data with realistic values"""
        self.departments = {
            'Engineering': {
                'roles': ['Software Engineer', 'Senior Software Engineer', 'QA Engineer'],
                'salary_ranges': {
                    'Software Engineer': (75000, 120000),
                    'Senior Software Engineer': (110000, 180000),
                    'QA Engineer': (65000, 110000)
                }
            },
            'IT': {
                'roles': ['System Administrator', 'Network Engineer', 'Security Analyst'],
                'salary_ranges': {
                    'System Administrator': (65000, 115000),
                    'Network Engineer': (80000, 140000),
                    'Security Analyst': (90000, 150000)
                }
            },
            'Sales': {
                'roles': ['Sales Representative', 'Account Executive', 'Sales Manager'],
                'salary_ranges': {
                    'Sales Representative': (45000, 90000),
                    'Account Executive': (70000, 130000),
                    'Sales Manager': (100000, 160000)
                }
            },
            'HR': {
                'roles': ['HR Specialist', 'Recruiter', 'HR Manager'],
                'salary_ranges': {
                    'HR Specialist': (50000, 85000),
                    'Recruiter': (55000, 95000),
                    'HR Manager': (90000, 140000)
                }
            }
        }
        
        self.performance_ratings = {
            'Exceeds Expectations': 0.15,  # 15% of employees
            'Meets Expectations': 0.65,    # 65% of employees
            'Needs Improvement': 0.15,     # 15% of employees
            'Performance Plan': 0.05       # 5% of employees
        }
        
        self.recruitment_sources = {
            'LinkedIn': 0.30,
            'Employee Referral': 0.25,
            'Company Website': 0.20,
            'Indeed': 0.15,
            'Recruitment Agency': 0.05,
            'Diversity Job Fair': 0.05
        }
        
        self.termination_reasons = {
            'N/A-StillEmployed': 0.85,    # 85% still employed
            'Voluntary-Other Opportunity': 0.06,
            'Voluntary-Relocation': 0.03,
            'Voluntary-Retirement': 0.02,
            'Involuntary-Performance': 0.02,
            'Involuntary-Layoff': 0.02
        }
        
        self.ethnicities = {
            'White': 0.60,
            'Asian': 0.15,
            'Hispanic or Latino': 0.12,
            'Black or African American': 0.08,
            'Two or More Races': 0.03,
            'Native American': 0.01,
            'Pacific Islander': 0.01
        }
    
    def _generate_date(self, start_year_offset=-10, end_year_offset=0):
        """Generate a date between start and end year offsets from current year"""
        start_date = date(self.BASE_YEAR + start_year_offset, 1, 1)
        end_date = date(self.BASE_YEAR + end_year_offset, 12, 31)
        fake_date = self.fake.date_between(start_date=start_date, end_date=end_date)
        return fake_date
    
    def _calculate_salary(self, role, department, performance, years_experience):
        """Calculate salary based on role, performance, and experience"""
        base_range = self.departments[department]['salary_ranges'][role]
        
        # Calculate base salary within range
        base = np.random.uniform(base_range[0], base_range[1])
        
        # Apply performance multiplier
        performance_multipliers = {
            'Exceeds Expectations': 1.15,
            'Meets Expectations': 1.0,
            'Needs Improvement': 0.9,
            'Performance Plan': 0.85
        }
        
        # Apply experience multiplier (1-3% increase per year)
        experience_multiplier = 1 + (years_experience * np.random.uniform(0.01, 0.03))
        
        final_salary = base * performance_multipliers[performance] * experience_multiplier
        return round(final_salary, 2)
    
    def _calculate_age(self, birth_date):
        """Calculate age from birth date"""
        today = self.TODAY
        return today.year - birth_date.year - ((today.month, today.day) < (birth_date.month, birth_date.day))
    
    def _calculate_years_experience(self, hire_date):
        """Calculate years of experience from hire date"""
        days_employed = (self.TODAY - hire_date).days
        return days_employed / 365.25
    
    def _generate_employee(self):
        """Generate a single employee record"""
        # Select department and role
        department = np.random.choice(list(self.departments.keys()))
        role = np.random.choice(self.departments[department]['roles'])
        
        # Generate dates
        hire_date = self._generate_date()
        birth_date = self._generate_date(
            start_year_offset=-(self.MAX_AGE),
            end_year_offset=-(self.MIN_AGE)
        )
        
        # Calculate age and experience
        age = self._calculate_age(birth_date)
        years_experience = self._calculate_years_experience(hire_date)
        
        # Generate performance rating
        performance = np.random.choice(
            list(self.performance_ratings.keys()),
            p=list(self.performance_ratings.values())
        )
        
        # Determine employment status and termination details
        term_reason = np.random.choice(
            list(self.termination_reasons.keys()),
            p=list(self.termination_reasons.values())
        )
        
        term_date = None
        if term_reason != 'N/A-StillEmployed':
            term_date = self._generate_date(
                start_year_offset=max(-3, int(-years_experience)),
                end_year_offset=0
            )
        
        # Generate last review date (within the past year, before termination if applicable)
        max_review_date = term_date if term_date else self.TODAY
        last_review_date = self._generate_date(
            start_year_offset=-1,
            end_year_offset=0
        )
        if last_review_date > max_review_date:
            last_review_date = max_review_date
        
        return {
            "EmployeeID": self.fake.unique.random_int(min=10000, max=99999),
            "Name": self.fake.name(),
            "Department": department,
            "Position": role,
            "HireDate": hire_date.strftime('%Y-%m-%d'),
            "TerminationDate": term_date.strftime('%Y-%m-%d') if term_date else None,
            "TerminationReason": term_reason,
            "EmploymentStatus": "Terminated" if term_date else "Active",
            "DateOfBirth": birth_date.strftime('%Y-%m-%d'),
            "Age": age,
            "Gender": np.random.choice(['M', 'F'], p=[0.51, 0.49]),
            "Ethnicity": np.random.choice(
                list(self.ethnicities.keys()),
                p=list(self.ethnicities.values())
            ),
            "Salary": self._calculate_salary(role, department, performance, years_experience),
            "PerformanceRating": performance,
            "RecruitmentSource": np.random.choice(
                list(self.recruitment_sources.keys()),
                p=list(self.recruitment_sources.values())
            ),
            "EngagementScore": round(np.random.normal(3.5, 0.5), 2),
            "AbsenceDays": int(np.random.negative_binomial(5, 0.7)),
            "LastReviewDate": last_review_date.strftime('%Y-%m-%d')
        }
    
    def generate_dataset(self, num_records=1000):
        """Generate a complete HR dataset"""
        employees = [self._generate_employee() for _ in range(num_records)]
        return pd.DataFrame(employees)
    
    def save_dataset(self, num_records=1000, csv_path='hr_data.csv', json_path='hr_data.json'):
        """Generate and save dataset in both CSV and JSON formats"""
        df = self.generate_dataset(num_records)
        
        # Save as CSV
        df.to_csv(csv_path, index=False)
        
        # Save as JSON with proper formatting
        df.to_json(json_path, orient='records', date_format='iso')
        
        return df

# Usage example
if __name__ == "__main__":
    generator = HRDataGenerator(seed=42)
    df = generator.save_dataset(1000)
    print(f"Generated {len(df)} employee records")

Generated 1000 employee records


In [6]:
# Create generator instance
generator = HRDataGenerator(seed=42)

# Generate and save dataset
df = generator.save_dataset(
    num_records=720,
    csv_path='hr_data.csv',
    json_path='hr_data.json'
)

TypeError: unsupported operand type(s) for -: 'datetime.datetime' and 'datetime.date'

### Converting it into desired format

import json
from datetime import datetime
import os
from pathlib import Path

def transform_json(input_json):
    """
    Transform JSON from current format to desired format, adding missing fields
    with default values where necessary.
    """
    # Create default values for missing fields
    default_values = {
        "ManagerName": None,
        "ManagerID": None,
        "RecruitmentSource": None,
        "PerformanceScore": None,
        "EngagementSurvey": None,
        "EmpSatisfaction": None,
        "SpecialProjectsCount": 0,
        "LastPerformanceReview_Date": None,
        "DaysLateLast30": 0,
        "Absences": 0
    }
    
    # Transform date formats
    def transform_date(date_str):
        if not date_str:
            return None
        try:
            # Parse the input date
            if "/" in date_str:
                date_obj = datetime.strptime(date_str.strip(), '%m/%d/%Y')
            else:
                return date_str  # Return as-is if format is unknown
            # Format to desired output
            return date_obj.strftime('%-m/%-d/%y')
        except ValueError:
            return date_str  # Return original if parsing fails
    
    # Transform existing fields and add new ones
    transformed = {
        # Existing fields with transformations
        "Employee_Name": f"{input_json['Employee_Name'].split()[-1]}, {' '.join(input_json['Employee_Name'].split()[:-1])}",
        "EmpID": input_json["EmpID"],
        "MarriedID": input_json["MarriedID"],
        "MaritalStatusID": input_json["MaritalStatusID"],
        "GenderID": input_json["GenderID"],
        "EmpStatusID": input_json["EmpStatusID"],
        "DeptID": input_json["DeptID"],
        "PerfScoreID": input_json["PerfScoreID"],
        "FromDiversityJobFairID": input_json["FromDiversityJobFairID"],
        "Salary": input_json["Salary"],
        "Termd": input_json["Termd"],
        "PositionID": input_json["PositionID"],
        "Position": input_json["Position"],
        "State": input_json["State"],
        "Zip": input_json["Zip"],
        "DOB": transform_date(input_json["DOB"]),
        "Sex": input_json["Sex"] + " " if len(input_json["Sex"]) == 1 else input_json["Sex"],
        "MaritalDesc": input_json["MaritalDesc"],
        "CitizenDesc": input_json["CitizenDesc"],
        "HispanicLatino": input_json["HispanicLatino"],
        "RaceDesc": input_json["RaceDesc"],
        "DateofHire": transform_date(input_json["DateofHire"]),
        "DateofTermination": transform_date(input_json["DateofTermination"]) if input_json["DateofTermination"] != "N/A-StillEmployed" else None,
        "TermReason": input_json["TermReason"],
        "EmploymentStatus": input_json["EmploymentStatus"],
        "Department": input_json["Department"].ljust(15)
    }
    
    # Add missing fields with default values
    transformed.update(default_values)
    
    return transformed

def process_json_file(input_path, output_path):
    """
    Process a JSON file from input path and save the transformed result to output path.
    
    Args:
        input_path (str): Path to the input JSON file
        output_path (str): Path where the transformed JSON should be saved
    
    Returns:
        bool: True if successful, False if an error occurred
    """
    try:
        # Create output directory if it doesn't exist
        output_dir = os.path.dirname(output_path)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        # Read input file
        with open(input_path, 'r', encoding='utf-8') as file:
            input_data = json.load(file)
        
        # Handle both single object and list of objects
        if isinstance(input_data, list):
            transformed_data = [transform_json(item) for item in input_data]
        else:
            transformed_data = transform_json(input_data)
        
        # Write transformed data to output file
        with open(output_path, 'w', encoding='utf-8') as file:
            json.dump(transformed_data, file, indent=4)
            
        print(f"Successfully transformed JSON data from {input_path}")
        print(f"Transformed data saved to {output_path}")
        return True
        
    except FileNotFoundError:
        print(f"Error: Input file not found at {input_path}")
        return False
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in input file {input_path}")
        return False
    except Exception as e:
        print(f"Error occurred while processing the file: {str(e)}")
        return False


if __name__ == "__main__":
    input_file_path = "C:\\Users\\govar\\OneDrive\\Documents\\HRM\\data\\processed\\expanded_hr_dataset.json"
    output_file_path = "C:\\Users\\govar\\OneDrive\\Documents\\HRM\\data\\processed\\transformed.json"
    
    # Process the file
    success = process_json_file(input_file_path, output_file_path)
    
    if success:
        print("JSON transformation completed successfully!")
    else:
        print("JSON transformation failed. Please check the error messages above.")